In [1]:
import os
import glob

from IPython.display import display, Markdown
import pandas as pd

# Group types of models (experiment type and model type) and pick best performing in terms of f1-score per unit of analysis and report them in a table


In [2]:
df_filepaths = glob.glob('./experiment_results/*.csv')

In [3]:
exps_df = pd.concat([pd.read_csv(df_filepath, index_col=['run_uuid', 'runName']) for df_filepath in df_filepaths])

In [4]:
exps_df.model_type = exps_df.model_type.str.strip('¶')

In [5]:
exps_df.shape

(2474, 48)

In [6]:
def get_best_models(df, grouping_criterion):
    return df.reset_index().loc[df.reset_index().groupby(grouping_criterion)['test_f1_micro_mean'].idxmax().dropna()]\
        .set_index(grouping_criterion)

### Generate the tables to report

In [7]:
def display_performance_table(df, metric, index_cols=['model_type'], display_=True):
    report_table = df.reset_index().copy()
    report_table['result'] = report_table[f'test_{metric}_mean'].map(lambda x: f'{x:.2f}') + \
    ' $\pm$ ' + report_table[f'test_{metric}_std'].map(lambda x: f'{x:.2f}')
    report_table['col_title'] = report_table.unit_of_analysis.str.split('_').str.join(' ') 
    report_table['col_title'] = pd.Categorical(
        report_table.col_title,
        categories=['title', 'title and first paragraph', 'title and 5 sentences', 'title and 10 sentences',
                    'title and first sentence each paragraph', 'raw text'],
        ordered=True)
    report_table = report_table[index_cols + ['col_title', 'result']]\
        .pivot_table(index=index_cols, columns=['col_title'], values=['result'], aggfunc='first', fill_value=0)\
        .droplevel(0, axis=1)

    report_table.columns.names = [None]

    # Highlight best scoring models according to their average
    mean_perf_arr = report_table.applymap(lambda x: float(str(x).split(' ')[0])).to_numpy()
    highlight_mask = mean_perf_arr == mean_perf_arr.max()
    report_table_arr = report_table.to_numpy()  # Note it passes the array by reference
    report_table_arr[highlight_mask] = '**' + report_table_arr[highlight_mask] + '**'

    if display_:
        display(Markdown(report_table.to_markdown()))
    
    return report_table

### Generate tables for all languages

In [8]:
metrics_to_report = ['f1_micro', 'recall_micro', 'precision_micro', 'accuracy'] 

In [9]:
language_dict = {'en': 'English', 'it': 'Italian', 'fr': 'French', 'po': 'Polish', 'ru': 'Russian', 'ge': 'German'}

In [10]:
def display_metrics_and_write_to_file(df, grouping_criterion, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    best_model_dfs_list = []
    report_tables_dfs_dict = {metric: [] for metric in metrics_to_report}

    for language, results_df in df.groupby('language'):
        best_models_df = get_best_models(results_df, grouping_criterion + ['unit_of_analysis'])
        best_model_dfs_list.append(best_models_df.copy())

        display(Markdown(f'# {language_dict[language]}'))
        
        for metric in metrics_to_report:
            os.makedirs(os.path.join(output_dir, metric), exist_ok=True)

            output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
            output_dir_latex = os.path.join(output_dir, metric, 'latex')
            output_dir_csv = os.path.join(output_dir, metric, 'csv')

            os.makedirs(output_dir_markdown, exist_ok=True)
            os.makedirs(output_dir_latex, exist_ok=True)
            os.makedirs(output_dir_csv, exist_ok=True)

            display(Markdown(f'## {metric}'))

            report_table = display_performance_table(df=best_models_df, index_cols=grouping_criterion, metric=metric, display_=True)

            # Export as markdown
            markdown_file = open(os.path.join(output_dir_markdown, f"{language_dict[language]}_{metric}.md"), "w")
            report_table.reset_index().to_markdown(markdown_file, index=False)
            markdown_file.close()

            # Export as latex table
            latex_file = open(os.path.join(output_dir_latex, f"{language_dict[language]}_{metric}.tex"), "w")
            report_table.reset_index().to_latex(latex_file, index=False)
            latex_file.close()

            # Export as csv
            report_table.to_csv(os.path.join(output_dir_csv, f"{language_dict[language]}_{metric}.csv"))

            # Stack all languages into single table
            report_table['language'] = language
            report_table = report_table.reset_index().set_index(['language'] + grouping_criterion)

            report_tables_dfs_dict[metric].append(report_table)

    # Store a csv with all the metrics and parameters of the best runs per language per grouping_criterion
    pd.concat(best_model_dfs_list)\
        .reset_index().set_index(['language'] + grouping_criterion).sort_index()\
        .to_csv(os.path.join(output_dir, 'best_exp_params_metrics_per_language.csv'))

    # Report or store unified table
    display(Markdown(f'# All 6 Languages'))
    for metric in metrics_to_report:
        display(Markdown(f'## {metric}'))
        multi_language_report_table_metric = pd.concat(report_tables_dfs_dict[metric])
        display(Markdown(multi_language_report_table_metric.reset_index().to_markdown(index=False)))

        output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
        output_dir_latex = os.path.join(output_dir, metric, 'latex')
        output_dir_csv = os.path.join(output_dir, metric, 'csv')

        # Export as markdown
        markdown_file = open(os.path.join(output_dir_markdown, f"all_6_languages_{metric}.md"), "w")
        multi_language_report_table_metric.reset_index().to_markdown(markdown_file, index=False)
        markdown_file.close()

        # Export as latex table
        latex_file = open(os.path.join(output_dir_latex, f"all_6_languages_{metric}.tex"), "w")
        multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
        latex_file.close()

        # Export as csv
        multi_language_report_table_metric.to_csv(os.path.join(output_dir_csv, f"all_6_languages_{metric}.csv"))

# Per model type

In [11]:
display_metrics_and_write_to_file(df=exps_df, grouping_criterion=['model_type'], output_dir='per_model_type_tables')

# English

## f1_micro

| model_type                      | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:--------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| Binary Relevance kNN            | 0.25 $\pm$ 0.05 | 0.31 $\pm$ 0.04             | 0.28 $\pm$ 0.09         | 0.28 $\pm$ 0.02          | 0.33 $\pm$ 0.05                           | 0.33 $\pm$ 0.04     |
| ComplementNB                    | 0.55 $\pm$ 0.00 | 0.62 $\pm$ 0.02             | 0.63 $\pm$ 0.00         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.65 $\pm$ 0.01     |
| Dummy Classifier                | 0.42 $\pm$ 0.02 | 0.43 $\pm$ 0.01             | 0.41 $\pm$ 0.01         | 0.43 $\pm$ 0.01          | 0.43 $\pm$ 0.01                           | 0.42 $\pm$ 0.01     |
| KNN                             | 0.41 $\pm$ 0.02 | 0.47 $\pm$ 0.06             | 0.41 $\pm$ 0.07         | 0.43 $\pm$ 0.04          | 0.41 $\pm$ 0.06                           | 0.52 $\pm$ 0.06     |
| LinearSVM                       | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| LogisticRegression              | 0.57 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| LogisticRegressionElasticNet    | 0.58 $\pm$ 0.01 | 0.63 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.63 $\pm$ 0.01          | 0.65 $\pm$ 0.01                           | 0.67 $\pm$ 0.00     |
| LogisticRegressionLasso         | 0.53 $\pm$ 0.02 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.03         | 0.60 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| LogisticRegressionRidge         | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.02     |
| Multi-label ARAM                | 0.31 $\pm$ 0.02 | 0.31 $\pm$ 0.03             | 0.31 $\pm$ 0.02         | 0.33 $\pm$ 0.03          | 0.33 $\pm$ 0.03                           | 0.26 $\pm$ 0.08     |
| Multilabel k Nearest Neighbours | 0.47 $\pm$ 0.04 | 0.49 $\pm$ 0.04             | 0.51 $\pm$ 0.01         | 0.54 $\pm$ 0.02          | 0.55 $\pm$ 0.01                           | 0.59 $\pm$ 0.00     |
| NaiveBayes                      | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.68 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.69 $\pm$ 0.01** |
| RandomForest                    | 0.56 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.60 $\pm$ 0.03         | 0.62 $\pm$ 0.02          | 0.63 $\pm$ 0.02                           | 0.64 $\pm$ 0.02     |
| RidgeClassifier                 | 0.57 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| SVM                             | 0.50 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.40 $\pm$ 0.01         | 0.41 $\pm$ 0.02          | 0.41 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| XGBoost                         | 0.53 $\pm$ 0.02 | 0.57 $\pm$ 0.01             | 0.57 $\pm$ 0.02         | 0.59 $\pm$ 0.01          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_type                      | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:--------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | Binary Relevance kNN            | 0.23 $\pm$ 0.06 | 0.42 $\pm$ 0.03             | 0.45 $\pm$ 0.08         | 0.43 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | 0.68 $\pm$ 0.02     |
| en         | ComplementNB                    | 0.55 $\pm$ 0.02 | 0.63 $\pm$ 0.01             | 0.65 $\pm$ 0.01         | 0.64 $\pm$ 0.01          | 0.65 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | Dummy Classifier                | 0.39 $\pm$ 0.01 | 0.40 $\pm$ 0.01             | 0.39 $\pm$ 0.01         | 0.41 $\pm$ 0.01          | 0.41 $\pm$ 0.01                           | 0.40 $\pm$ 0.01     |
| en         | KNN                             | 0.36 $\pm$ 0.02 | 0.48 $\pm$ 0.01             | 0.53 $\pm$ 0.06         | 0.52 $\pm$ 0.01          | 0.56 $\pm$ 0.06                           | 0.61 $\pm$ 0.02     |
| en         | LinearSVM                       | 0.56 $\pm$ 0.00 | 0.64 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | 0.58 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionElasticNet    | 0.59 $\pm$ 0.01 | 0.65 $\pm$ 0.01             | 0.66 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.65 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | 0.62 $\pm$ 0.00 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.03         | 0.63 $\pm$ 0.02          | 0.63 $\pm$ 0.02                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionRidge         | 0.60 $\pm$ 0.00 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.69 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | Multi-label ARAM                | 0.23 $\pm$ 0.02 | 0.24 $\pm$ 0.02             | 0.23 $\pm$ 0.02         | 0.25 $\pm$ 0.02          | 0.25 $\pm$ 0.02                           | 0.19 $\pm$ 0.06     |
| en         | Multilabel k Nearest Neighbours | 0.52 $\pm$ 0.01 | 0.52 $\pm$ 0.01             | 0.56 $\pm$ 0.03         | 0.55 $\pm$ 0.01          | 0.59 $\pm$ 0.02                           | 0.63 $\pm$ 0.02     |
| en         | NaiveBayes                      | 0.59 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.64 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| en         | RandomForest                    | 0.67 $\pm$ 0.01 | **0.75 $\pm$ 0.01**         | 0.74 $\pm$ 0.02         | 0.73 $\pm$ 0.01          | 0.74 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RidgeClassifier                 | 0.58 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | SVM                             | 0.52 $\pm$ 0.03 | 0.63 $\pm$ 0.03             | 0.61 $\pm$ 0.06         | 0.63 $\pm$ 0.04          | 0.56 $\pm$ 0.02                           | 0.65 $\pm$ 0.02     |
| en         | XGBoost                         | 0.58 $\pm$ 0.02 | 0.64 $\pm$ 0.02             | 0.63 $\pm$ 0.02         | 0.66 $\pm$ 0.01          | 0.68 $\pm$ 0.00                           | 0.71 $\pm$ 0.02     |
| fr         | Binary Relevance kNN            | 0.08 $\pm$ 0.04 | 0.14 $\pm$ 0.06             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| fr         | ComplementNB                    | 0.35 $\pm$ 0.03 | 0.37 $\pm$ 0.01             | 0.43 $\pm$ 0.01         | 0.45 $\pm$ 0.03          | 0.42 $\pm$ 0.01                           | 0.48 $\pm$ 0.04     |
| fr         | Dummy Classifier                | 0.25 $\pm$ 0.02 | 0.28 $\pm$ 0.02             | 0.27 $\pm$ 0.03         | 0.27 $\pm$ 0.01          | 0.28 $\pm$ 0.01                           | 0.29 $\pm$ 0.01     |
| fr         | KNN                             | 0.28 $\pm$ 0.01 | 0.28 $\pm$ 0.01             | 0.28 $\pm$ 0.01         | 0.28 $\pm$ 0.01          | 0.27 $\pm$ 0.05                           | 0.28 $\pm$ 0.01     |
| fr         | LinearSVM                       | 0.42 $\pm$ 0.01 | 0.44 $\pm$ 0.03             | 0.47 $\pm$ 0.06         | 0.52 $\pm$ 0.06          | 0.46 $\pm$ 0.05                           | 0.52 $\pm$ 0.05     |
| fr         | LogisticRegression              | 0.41 $\pm$ 0.01 | 0.43 $\pm$ 0.02             | 0.50 $\pm$ 0.07         | 0.52 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.06     |
| fr         | LogisticRegressionElasticNet    | 0.43 $\pm$ 0.03 | 0.44 $\pm$ 0.04             | 0.48 $\pm$ 0.03         | 0.50 $\pm$ 0.02          | 0.45 $\pm$ 0.04                           | **0.58 $\pm$ 0.06** |
| fr         | LogisticRegressionLasso         | 0.46 $\pm$ 0.02 | 0.42 $\pm$ 0.02             | 0.45 $\pm$ 0.02         | 0.41 $\pm$ 0.05          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.04     |
| fr         | LogisticRegressionRidge         | 0.42 $\pm$ 0.04 | 0.45 $\pm$ 0.02             | 0.53 $\pm$ 0.09         | 0.55 $\pm$ 0.03          | 0.54 $\pm$ 0.08                           | 0.53 $\pm$ 0.08     |
| fr         | Multi-label ARAM                | 0.11 $\pm$ 0.01 | 0.10 $\pm$ 0.00             | 0.10 $\pm$ 0.00         | 0.06 $\pm$ 0.03          | 0.08 $\pm$ 0.03                           | 0.08 $\pm$ 0.03     |
| fr         | Multilabel k Nearest Neighbours | 0.39 $\pm$ 0.03 | 0.34 $\pm$ 0.02             | 0.33 $\pm$ 0.03         | 0.40 $\pm$ 0.01          | 0.33 $\pm$ 0.08                           | 0.43 $\pm$ 0.05     |
| fr         | NaiveBayes                      | 0.38 $\pm$ 0.02 | 0.41 $\pm$ 0.03             | 0.44 $\pm$ 0.02         | 0.45 $\pm$ 0.02          | 0.44 $\pm$ 0.01                           | 0.49 $\pm$ 0.02     |
| fr         | RandomForest                    | 0.53 $\pm$ 0.07 | 0.41 $\pm$ 0.06             | 0.52 $\pm$ 0.07         | 0.48 $\pm$ 0.04          | 0.44 $\pm$ 0.06                           | 0.57 $\pm$ 0.07     |
| fr         | RidgeClassifier                 | 0.40 $\pm$ 0.03 | 0.45 $\pm$ 0.01             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | SVM                             | 0.29 $\pm$ 0.04 | 0.05 $\pm$ 0.06             | 0.20 $\pm$ 0.13         | 0.06 $\pm$ 0.05          | 0.13 $\pm$ 0.05                           | 0.19 $\pm$ 0.03     |
| fr         | XGBoost                         | 0.48 $\pm$ 0.01 | 0.38 $\pm$ 0.01             | 0.39 $\pm$ 0.03         | 0.44 $\pm$ 0.05          | 0.42 $\pm$ 0.03                           | 0.51 $\pm$ 0.02     |
| ge         | Binary Relevance kNN            | 0.31 $\pm$ 0.07 | 0.18 $\pm$ 0.02             | 0.19 $\pm$ 0.05         | 0.17 $\pm$ 0.05          | 0.18 $\pm$ 0.06                           | 0.17 $\pm$ 0.12     |
| ge         | ComplementNB                    | 0.44 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.42 $\pm$ 0.03         | 0.48 $\pm$ 0.02          | 0.44 $\pm$ 0.06                           | 0.43 $\pm$ 0.02     |
| ge         | Dummy Classifier                | 0.39 $\pm$ 0.02 | 0.37 $\pm$ 0.03             | 0.38 $\pm$ 0.04         | 0.39 $\pm$ 0.02          | 0.37 $\pm$ 0.04                           | 0.38 $\pm$ 0.03     |
| ge         | KNN                             | 0.37 $\pm$ 0.05 | 0.38 $\pm$ 0.05             | 0.40 $\pm$ 0.09         | 0.32 $\pm$ 0.07          | 0.28 $\pm$ 0.06                           | 0.24 $\pm$ 0.02     |
| ge         | LinearSVM                       | 0.42 $\pm$ 0.03 | 0.49 $\pm$ 0.01             | 0.48 $\pm$ 0.06         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.52 $\pm$ 0.03     |
| ge         | LogisticRegression              | 0.41 $\pm$ 0.03 | 0.48 $\pm$ 0.03             | 0.45 $\pm$ 0.03         | 0.49 $\pm$ 0.03          | 0.46 $\pm$ 0.04                           | 0.50 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.02             | 0.45 $\pm$ 0.04         | 0.53 $\pm$ 0.05          | 0.49 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionLasso         | 0.48 $\pm$ 0.02 | 0.48 $\pm$ 0.04             | 0.46 $\pm$ 0.00         | 0.50 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.58 $\pm$ 0.03     |
| ge         | LogisticRegressionRidge         | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.04             | 0.43 $\pm$ 0.05         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | Multi-label ARAM                | 0.08 $\pm$ 0.03 | 0.12 $\pm$ 0.07             | 0.12 $\pm$ 0.07         | 0.12 $\pm$ 0.07          | 0.13 $\pm$ 0.07                           | 0.12 $\pm$ 0.07     |
| ge         | Multilabel k Nearest Neighbours | 0.41 $\pm$ 0.04 | 0.43 $\pm$ 0.04             | 0.41 $\pm$ 0.02         | 0.44 $\pm$ 0.04          | 0.40 $\pm$ 0.08                           | 0.45 $\pm$ 0.02     |
| ge         | NaiveBayes                      | 0.44 $\pm$ 0.02 | 0.48 $\pm$ 0.02             | 0.47 $\pm$ 0.01         | 0.50 $\pm$ 0.02          | 0.50 $\pm$ 0.04                           | 0.54 $\pm$ 0.03     |
| ge         | RandomForest                    | 0.46 $\pm$ 0.01 | 0.47 $\pm$ 0.03             | 0.45 $\pm$ 0.03         | 0.49 $\pm$ 0.04          | 0.58 $\pm$ 0.13                           | **0.64 $\pm$ 0.11** |
| ge         | RidgeClassifier                 | 0.42 $\pm$ 0.03 | 0.48 $\pm$ 0.05             | 0.44 $\pm$ 0.04         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.03                           | 0.55 $\pm$ 0.01     |
| ge         | SVM                             | 0.39 $\pm$ 0.04 | 0.55 $\pm$ 0.08             | 0.41 $\pm$ 0.06         | 0.35 $\pm$ 0.08          | 0.25 $\pm$ 0.03                           | 0.33 $\pm$ 0.02     |
| ge         | XGBoost                         | 0.48 $\pm$ 0.03 | 0.48 $\pm$ 0.03             | 0.47 $\pm$ 0.02         | 0.46 $\pm$ 0.01          | 0.50 $\pm$ 0.04                           | 0.56 $\pm$ 0.06     |
| it         | Binary Relevance kNN            | 0.19 $\pm$ 0.04 | 0.18 $\pm$ 0.01             | 0.12 $\pm$ 0.03         | 0.15 $\pm$ 0.07          | 0.12 $\pm$ 0.05                           | 0.13 $\pm$ 0.06     |
| it         | ComplementNB                    | 0.40 $\pm$ 0.00 | 0.43 $\pm$ 0.02             | 0.45 $\pm$ 0.02         | 0.50 $\pm$ 0.02          | 0.40 $\pm$ 0.03                           | 0.46 $\pm$ 0.02     |
| it         | Dummy Classifier                | 0.34 $\pm$ 0.03 | 0.32 $\pm$ 0.03             | 0.31 $\pm$ 0.03         | 0.31 $\pm$ 0.04          | 0.32 $\pm$ 0.03                           | 0.33 $\pm$ 0.03     |
| it         | KNN                             | 0.33 $\pm$ 0.01 | 0.33 $\pm$ 0.02             | 0.32 $\pm$ 0.02         | 0.32 $\pm$ 0.02          | 0.35 $\pm$ 0.00                           | 0.33 $\pm$ 0.01     |
| it         | LinearSVM                       | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.01             | 0.46 $\pm$ 0.02         | 0.51 $\pm$ 0.04          | 0.52 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegression              | 0.42 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.47 $\pm$ 0.02         | 0.51 $\pm$ 0.02          | 0.50 $\pm$ 0.02                           | 0.56 $\pm$ 0.02     |
| it         | LogisticRegressionElasticNet    | 0.44 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.57 $\pm$ 0.04          | 0.50 $\pm$ 0.01                           | 0.55 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | 0.46 $\pm$ 0.01 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.04         | 0.51 $\pm$ 0.03          | 0.48 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| it         | LogisticRegressionRidge         | 0.43 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.59 $\pm$ 0.07          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.07     |
| it         | Multi-label ARAM                | 0.06 $\pm$ 0.00 | 0.06 $\pm$ 0.00             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.01                           | 0.06 $\pm$ 0.00     |
| it         | Multilabel k Nearest Neighbours | 0.39 $\pm$ 0.01 | 0.40 $\pm$ 0.03             | 0.39 $\pm$ 0.02         | 0.39 $\pm$ 0.02          | 0.38 $\pm$ 0.02                           | 0.40 $\pm$ 0.02     |
| it         | NaiveBayes                      | 0.40 $\pm$ 0.01 | 0.46 $\pm$ 0.01             | 0.49 $\pm$ 0.02         | 0.51 $\pm$ 0.01          | 0.48 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| it         | RandomForest                    | 0.47 $\pm$ 0.03 | 0.55 $\pm$ 0.03             | 0.49 $\pm$ 0.04         | 0.55 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | **0.62 $\pm$ 0.06** |
| it         | RidgeClassifier                 | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.04             | 0.47 $\pm$ 0.01         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | SVM                             | 0.36 $\pm$ 0.03 | 0.43 $\pm$ 0.07             | 0.30 $\pm$ 0.04         | 0.33 $\pm$ 0.05          | 0.31 $\pm$ 0.03                           | 0.31 $\pm$ 0.03     |
| it         | XGBoost                         | 0.42 $\pm$ 0.03 | 0.44 $\pm$ 0.04             | 0.45 $\pm$ 0.02         | 0.49 $\pm$ 0.04          | 0.52 $\pm$ 0.05                           | 0.54 $\pm$ 0.04     |
| po         | Binary Relevance kNN            | 0.38 $\pm$ 0.07 | 0.34 $\pm$ 0.05             | 0.24 $\pm$ 0.03         | 0.19 $\pm$ 0.04          | 0.15 $\pm$ 0.05                           | 0.15 $\pm$ 0.05     |
| po         | ComplementNB                    | 0.46 $\pm$ 0.04 | 0.47 $\pm$ 0.04             | 0.53 $\pm$ 0.01         | 0.57 $\pm$ 0.02          | 0.51 $\pm$ 0.05                           | 0.53 $\pm$ 0.04     |
| po         | Dummy Classifier                | 0.43 $\pm$ 0.02 | 0.42 $\pm$ 0.03             | 0.43 $\pm$ 0.03         | 0.41 $\pm$ 0.02          | 0.40 $\pm$ 0.00                           | 0.42 $\pm$ 0.03     |
| po         | KNN                             | 0.48 $\pm$ 0.05 | 0.45 $\pm$ 0.03             | 0.50 $\pm$ 0.04         | 0.27 $\pm$ 0.04          | 0.23 $\pm$ 0.05                           | 0.32 $\pm$ 0.10     |
| po         | LinearSVM                       | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.57 $\pm$ 0.04         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | 0.48 $\pm$ 0.04 | 0.53 $\pm$ 0.04             | 0.57 $\pm$ 0.04         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegressionElasticNet    | 0.49 $\pm$ 0.05 | 0.54 $\pm$ 0.03             | 0.58 $\pm$ 0.05         | 0.60 $\pm$ 0.07          | 0.58 $\pm$ 0.04                           | 0.62 $\pm$ 0.05     |
| po         | LogisticRegressionLasso         | 0.51 $\pm$ 0.04 | 0.52 $\pm$ 0.04             | 0.52 $\pm$ 0.03         | 0.57 $\pm$ 0.04          | 0.57 $\pm$ 0.02                           | 0.58 $\pm$ 0.06     |
| po         | LogisticRegressionRidge         | 0.50 $\pm$ 0.05 | 0.54 $\pm$ 0.05             | 0.60 $\pm$ 0.05         | 0.59 $\pm$ 0.01          | 0.61 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | Multi-label ARAM                | 0.06 $\pm$ 0.00 | 0.06 $\pm$ 0.00             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.00                           | 0.06 $\pm$ 0.00     |
| po         | Multilabel k Nearest Neighbours | 0.46 $\pm$ 0.03 | 0.47 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.51 $\pm$ 0.03          | 0.53 $\pm$ 0.01                           | 0.55 $\pm$ 0.05     |
| po         | NaiveBayes                      | 0.50 $\pm$ 0.04 | 0.55 $\pm$ 0.03             | 0.58 $\pm$ 0.02         | 0.62 $\pm$ 0.04          | 0.59 $\pm$ 0.04                           | 0.59 $\pm$ 0.04     |
| po         | RandomForest                    | 0.57 $\pm$ 0.03 | 0.60 $\pm$ 0.02             | 0.58 $\pm$ 0.07         | 0.59 $\pm$ 0.01          | 0.65 $\pm$ 0.03                           | **0.70 $\pm$ 0.03** |
| po         | RidgeClassifier                 | 0.49 $\pm$ 0.05 | 0.54 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | SVM                             | 0.46 $\pm$ 0.04 | 0.44 $\pm$ 0.06             | 0.45 $\pm$ 0.09         | 0.38 $\pm$ 0.04          | 0.36 $\pm$ 0.04                           | 0.39 $\pm$ 0.03     |
| po         | XGBoost                         | 0.54 $\pm$ 0.03 | 0.52 $\pm$ 0.03             | 0.54 $\pm$ 0.04         | 0.57 $\pm$ 0.04          | 0.59 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ru         | Binary Relevance kNN            | 0.06 $\pm$ 0.00 | 0.07 $\pm$ 0.01             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.00                           | 0.06 $\pm$ 0.00     |
| ru         | ComplementNB                    | 0.30 $\pm$ 0.01 | 0.34 $\pm$ 0.05             | 0.45 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.43 $\pm$ 0.06                           | 0.39 $\pm$ 0.02     |
| ru         | Dummy Classifier                | 0.23 $\pm$ 0.01 | 0.22 $\pm$ 0.02             | 0.19 $\pm$ 0.01         | 0.21 $\pm$ 0.00          | 0.20 $\pm$ 0.02                           | 0.23 $\pm$ 0.01     |
| ru         | KNN                             | 0.22 $\pm$ 0.01 | 0.24 $\pm$ 0.01             | 0.22 $\pm$ 0.01         | 0.22 $\pm$ 0.01          | 0.23 $\pm$ 0.01                           | 0.22 $\pm$ 0.01     |
| ru         | LinearSVM                       | 0.34 $\pm$ 0.05 | 0.40 $\pm$ 0.03             | 0.48 $\pm$ 0.07         | **0.59 $\pm$ 0.03**      | 0.48 $\pm$ 0.06                           | 0.52 $\pm$ 0.05     |
| ru         | LogisticRegression              | 0.35 $\pm$ 0.02 | 0.42 $\pm$ 0.04             | 0.49 $\pm$ 0.03         | 0.56 $\pm$ 0.02          | 0.52 $\pm$ 0.03                           | 0.53 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | 0.37 $\pm$ 0.02 | 0.43 $\pm$ 0.01             | 0.49 $\pm$ 0.01         | 0.57 $\pm$ 0.02          | 0.47 $\pm$ 0.03                           | 0.52 $\pm$ 0.03     |
| ru         | LogisticRegressionLasso         | 0.38 $\pm$ 0.04 | 0.41 $\pm$ 0.02             | 0.41 $\pm$ 0.04         | 0.47 $\pm$ 0.02          | 0.51 $\pm$ 0.07                           | 0.47 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | 0.38 $\pm$ 0.02 | 0.45 $\pm$ 0.02             | 0.53 $\pm$ 0.03         | 0.56 $\pm$ 0.04          | 0.54 $\pm$ 0.01                           | 0.56 $\pm$ 0.07     |
| ru         | Multi-label ARAM                | 0.06 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.00         | 0.05 $\pm$ 0.00          | 0.06 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ru         | Multilabel k Nearest Neighbours | 0.31 $\pm$ 0.06 | 0.32 $\pm$ 0.02             | 0.35 $\pm$ 0.04         | 0.34 $\pm$ 0.05          | 0.36 $\pm$ 0.08                           | 0.39 $\pm$ 0.04     |
| ru         | NaiveBayes                      | 0.31 $\pm$ 0.00 | 0.37 $\pm$ 0.03             | 0.40 $\pm$ 0.01         | 0.44 $\pm$ 0.03          | 0.43 $\pm$ 0.02                           | 0.45 $\pm$ 0.02     |
| ru         | RandomForest                    | 0.44 $\pm$ 0.01 | 0.54 $\pm$ 0.05             | 0.43 $\pm$ 0.02         | 0.56 $\pm$ 0.03          | 0.50 $\pm$ 0.02                           | 0.57 $\pm$ 0.04     |
| ru         | RidgeClassifier                 | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | SVM                             | 0.43 $\pm$ 0.08 | 0.17 $\pm$ 0.03             | 0.22 $\pm$ 0.12         | 0.28 $\pm$ 0.20          | 0.21 $\pm$ 0.14                           | 0.15 $\pm$ 0.07     |
| ru         | XGBoost                         | 0.36 $\pm$ 0.05 | 0.39 $\pm$ 0.07             | 0.40 $\pm$ 0.07         | 0.46 $\pm$ 0.04          | 0.42 $\pm$ 0.01                           | 0.51 $\pm$ 0.03     |

/tmp/ipykernel_58875/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_type                      | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:--------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | Binary Relevance kNN            | 0.02 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| en         | ComplementNB                    | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.08 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.09 $\pm$ 0.01     |
| en         | Dummy Classifier                | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.03 $\pm$ 0.01                           | 0.01 $\pm$ 0.00     |
| en         | LinearSVM                       | 0.02 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | 0.03 $\pm$ 0.00 | 0.06 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.03     |
| en         | LogisticRegressionElasticNet    | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionRidge         | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.01                           | 0.10 $\pm$ 0.02     |
| en         | Multi-label ARAM                | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| en         | Multilabel k Nearest Neighbours | 0.03 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| en         | NaiveBayes                      | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.00         | 0.02 $\pm$ 0.00          | 0.04 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| en         | RandomForest                    | 0.07 $\pm$ 0.02 | 0.09 $\pm$ 0.01             | **0.11 $\pm$ 0.02**     | **0.11 $\pm$ 0.00**      | **0.11 $\pm$ 0.01**                       | **0.11 $\pm$ 0.02** |
| en         | RidgeClassifier                 | 0.03 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | SVM                             | 0.03 $\pm$ 0.02 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.03 $\pm$ 0.02     |
| en         | XGBoost                         | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.07 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| fr         | Binary Relevance kNN            | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | ComplementNB                    | 0.00 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| fr         | Dummy Classifier                | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | KNN                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | LinearSVM                       | 0.02 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.07 $\pm$ 0.03     |
| fr         | LogisticRegression              | 0.02 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionElasticNet    | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.04                           | **0.09 $\pm$ 0.02** |
| fr         | LogisticRegressionLasso         | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | Multi-label ARAM                | 0.04 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | Multilabel k Nearest Neighbours | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.03 $\pm$ 0.02     |
| fr         | NaiveBayes                      | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| fr         | RandomForest                    | 0.04 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| fr         | RidgeClassifier                 | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | SVM                             | 0.00 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| fr         | XGBoost                         | 0.03 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| ge         | Binary Relevance kNN            | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | ComplementNB                    | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | Dummy Classifier                | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | LinearSVM                       | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LogisticRegression              | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | 0.02 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | Multi-label ARAM                | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ge         | Multilabel k Nearest Neighbours | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.03                           | 0.01 $\pm$ 0.01     |
| ge         | NaiveBayes                      | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.02     |
| ge         | RandomForest                    | 0.01 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | **0.06 $\pm$ 0.02**      | 0.03 $\pm$ 0.04                           | 0.02 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | SVM                             | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | XGBoost                         | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| it         | Binary Relevance kNN            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | ComplementNB                    | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| it         | Dummy Classifier                | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | LinearSVM                       | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegression              | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.05 $\pm$ 0.03          | 0.05 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| it         | LogisticRegressionElasticNet    | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.03                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionLasso         | 0.05 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.03 $\pm$ 0.00     |
| it         | LogisticRegressionRidge         | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.05 $\pm$ 0.02                           | 0.08 $\pm$ 0.01     |
| it         | Multi-label ARAM                | 0.06 $\pm$ 0.02 | 0.06 $\pm$ 0.02             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| it         | Multilabel k Nearest Neighbours | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| it         | NaiveBayes                      | 0.02 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.00     |
| it         | RandomForest                    | 0.03 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | **0.09 $\pm$ 0.02**      | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.02     |
| it         | RidgeClassifier                 | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | SVM                             | 0.02 $\pm$ 0.00 | 0.01 $\pm$ 0.00             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.00     |
| it         | XGBoost                         | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.00         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | Binary Relevance kNN            | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | ComplementNB                    | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.05 $\pm$ 0.00          | 0.02 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| po         | Dummy Classifier                | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | LinearSVM                       | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegression              | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.03 $\pm$ 0.00          | 0.04 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.05 $\pm$ 0.00                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.01     |
| po         | LogisticRegressionRidge         | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | Multi-label ARAM                | 0.05 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | Multilabel k Nearest Neighbours | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.04 $\pm$ 0.02     |
| po         | NaiveBayes                      | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| po         | RandomForest                    | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.03             | 0.03 $\pm$ 0.03         | 0.04 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | **0.09 $\pm$ 0.02** |
| po         | RidgeClassifier                 | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | SVM                             | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | XGBoost                         | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.03 $\pm$ 0.03     |
| ru         | Binary Relevance kNN            | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| ru         | ComplementNB                    | 0.01 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| ru         | Dummy Classifier                | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | LinearSVM                       | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.09 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.12 $\pm$ 0.02     |
| ru         | LogisticRegression              | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.00         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.03                           | 0.10 $\pm$ 0.01     |
| ru         | LogisticRegressionElasticNet    | 0.03 $\pm$ 0.00 | 0.04 $\pm$ 0.01             | 0.07 $\pm$ 0.01         | 0.09 $\pm$ 0.04          | 0.09 $\pm$ 0.04                           | 0.11 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.04          | 0.08 $\pm$ 0.03                           | 0.07 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | 0.04 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| ru         | Multi-label ARAM                | 0.07 $\pm$ 0.01 | 0.08 $\pm$ 0.02             | 0.08 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| ru         | Multilabel k Nearest Neighbours | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| ru         | NaiveBayes                      | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.05 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| ru         | RandomForest                    | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.07 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| ru         | RidgeClassifier                 | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | SVM                             | 0.06 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ru         | XGBoost                         | 0.04 $\pm$ 0.03 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.04                           | **0.13 $\pm$ 0.05** |

/tmp/ipykernel_58875/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)



# Per model sub-type

In [12]:
display_metrics_and_write_to_file(df=exps_df, grouping_criterion=['model_type','model_subtype'], output_dir='per_model_subtype_tables')

# English

## f1_micro

|                                                              | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-------------------------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| ('Binary Relevance kNN', 'Natively Multilabel')              | 0.25 $\pm$ 0.05 | 0.31 $\pm$ 0.04             | 0.28 $\pm$ 0.09         | 0.28 $\pm$ 0.02          | 0.33 $\pm$ 0.05                           | 0.33 $\pm$ 0.04     |
| ('ComplementNB', 'RakelD Partitioning of labels')            | 0.55 $\pm$ 0.00 | 0.62 $\pm$ 0.02             | 0.63 $\pm$ 0.00         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.65 $\pm$ 0.01     |
| ('Dummy Classifier', 'No Upsampling')                        | 0.42 $\pm$ 0.02 | 0.43 $\pm$ 0.01             | 0.41 $\pm$ 0.01         | 0.43 $\pm$ 0.01          | 0.43 $\pm$ 0.01                           | 0.42 $\pm$ 0.01     |
| ('KNN', 'BorderlineSMOTE')                                   | 0.41 $\pm$ 0.02 | 0.33 $\pm$ 0.06             | 0.28 $\pm$ 0.01         | 0.29 $\pm$ 0.00          | 0.31 $\pm$ 0.04                           | 0.33 $\pm$ 0.03     |
| ('KNN', 'No Upsampling')                                     | 0.34 $\pm$ 0.06 | 0.40 $\pm$ 0.05             | 0.38 $\pm$ 0.06         | 0.35 $\pm$ 0.04          | 0.41 $\pm$ 0.06                           | 0.50 $\pm$ 0.06     |
| ('KNN', 'Random Oversampling')                               | 0.38 $\pm$ 0.06 | 0.47 $\pm$ 0.06             | 0.41 $\pm$ 0.07         | 0.43 $\pm$ 0.04          | 0.40 $\pm$ 0.07                           | 0.52 $\pm$ 0.06     |
| ('KNN', 'SMOTE')                                             | 0.41 $\pm$ 0.03 | 0.34 $\pm$ 0.04             | 0.30 $\pm$ 0.01         | 0.30 $\pm$ 0.02          | 0.31 $\pm$ 0.04                           | 0.33 $\pm$ 0.04     |
| ('KNN', 'SVMSMOTE')                                          | 0               | 0.38 $\pm$ 0.03             | 0                       | 0                        | 0.34 $\pm$ 0.03                           | 0                   |
| ('LinearSVM', 'BorderlineSMOTE')                             | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LinearSVM', 'No Upsampling')                               | 0.56 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LinearSVM', 'RakelD Partitioning of labels')               | 0.55 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| ('LinearSVM', 'Random Oversampling')                         | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LinearSVM', 'SMOTE')                                       | 0.56 $\pm$ 0.02 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'BorderlineSMOTE')                    | 0.55 $\pm$ 0.00 | 0.60 $\pm$ 0.02             | 0.60 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'No Upsampling')                      | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.60 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegression', 'Random Oversampling')                | 0.56 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'SMOTE')                              | 0.56 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'SVMSMOTE')                           | 0.57 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.61 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegressionElasticNet', 'BorderlineSMOTE')          | 0.57 $\pm$ 0.02 | 0.62 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.63 $\pm$ 0.02          | 0.65 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegressionElasticNet', 'No Upsampling')            | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegressionElasticNet', 'Random Oversampling')      | 0.57 $\pm$ 0.02 | 0.62 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.63 $\pm$ 0.02          | 0.65 $\pm$ 0.01                           | 0.67 $\pm$ 0.00     |
| ('LogisticRegressionElasticNet', 'SMOTE')                    | 0.58 $\pm$ 0.02 | 0.62 $\pm$ 0.02             | 0.62 $\pm$ 0.02         | 0.63 $\pm$ 0.02          | 0.65 $\pm$ 0.01                           | 0.67 $\pm$ 0.00     |
| ('LogisticRegressionElasticNet', 'SVMSMOTE')                 | 0.58 $\pm$ 0.01 | 0.63 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.63 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.67 $\pm$ 0.01     |
| ('LogisticRegressionLasso', 'BorderlineSMOTE')               | 0.53 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.59 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.60 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| ('LogisticRegressionLasso', 'No Upsampling')                 | 0.53 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.59 $\pm$ 0.02         | 0.57 $\pm$ 0.02          | 0.60 $\pm$ 0.01                           | 0.62 $\pm$ 0.00     |
| ('LogisticRegressionLasso', 'Random Oversampling')           | 0.53 $\pm$ 0.02 | 0.59 $\pm$ 0.02             | 0.59 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.60 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('LogisticRegressionLasso', 'SMOTE')                         | 0.53 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.59 $\pm$ 0.02         | 0.58 $\pm$ 0.02          | 0.61 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('LogisticRegressionLasso', 'SVMSMOTE')                      | 0.53 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.03         | 0.60 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('LogisticRegressionRidge', 'BorderlineSMOTE')               | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'No Upsampling')                 | 0.55 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'RakelD Partitioning of labels') | 0.55 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.61 $\pm$ 0.01         | 0.61 $\pm$ 0.02          | 0.64 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'Random Oversampling')           | 0.56 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.02          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'SMOTE')                         | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'SVMSMOTE')                      | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.66 $\pm$ 0.02     |
| ('Multi-label ARAM', 'Natively Multilabel')                  | 0.31 $\pm$ 0.02 | 0.31 $\pm$ 0.03             | 0.31 $\pm$ 0.02         | 0.33 $\pm$ 0.03          | 0.33 $\pm$ 0.03                           | 0.26 $\pm$ 0.08     |
| ('Multilabel k Nearest Neighbours', 'Natively Multilabel')   | 0.47 $\pm$ 0.04 | 0.49 $\pm$ 0.04             | 0.51 $\pm$ 0.01         | 0.54 $\pm$ 0.02          | 0.55 $\pm$ 0.01                           | 0.59 $\pm$ 0.00     |
| ('NaiveBayes', 'BorderlineSMOTE')                            | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | 0.68 $\pm$ 0.01     |
| ('NaiveBayes', 'No Upsampling')                              | 0.61 $\pm$ 0.01 | 0.66 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.65 $\pm$ 0.02          | 0.64 $\pm$ 0.03                           | 0.64 $\pm$ 0.02     |
| ('NaiveBayes', 'Random Oversampling')                        | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.68 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.69 $\pm$ 0.01** |
| ('NaiveBayes', 'SMOTE')                                      | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.69 $\pm$ 0.01** |
| ('RandomForest', 'BorderlineSMOTE')                          | 0.52 $\pm$ 0.02 | 0.54 $\pm$ 0.01             | 0.56 $\pm$ 0.03         | 0.59 $\pm$ 0.01          | 0.61 $\pm$ 0.02                           | 0.64 $\pm$ 0.01     |
| ('RandomForest', 'No Upsampling')                            | 0.50 $\pm$ 0.02 | 0.53 $\pm$ 0.02             | 0.56 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.60 $\pm$ 0.02                           | 0.61 $\pm$ 0.01     |
| ('RandomForest', 'Random Oversampling')                      | 0.56 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.60 $\pm$ 0.03         | 0.62 $\pm$ 0.02          | 0.63 $\pm$ 0.02                           | 0.64 $\pm$ 0.02     |
| ('RandomForest', 'SMOTE')                                    | 0.52 $\pm$ 0.03 | 0.55 $\pm$ 0.02             | 0.57 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.02     |
| ('RandomForest', 'SVMSMOTE')                                 | 0.53 $\pm$ 0.02 | 0.55 $\pm$ 0.01             | 0.57 $\pm$ 0.03         | 0.60 $\pm$ 0.02          | 0.61 $\pm$ 0.02                           | 0.64 $\pm$ 0.01     |
| ('RidgeClassifier', 'BorderlineSMOTE')                       | 0.55 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'No Upsampling')                         | 0.55 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'Random Oversampling')                   | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'SMOTE')                                 | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'SVMSMOTE')                              | 0.57 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| ('SVM', 'BorderlineSMOTE')                                   | 0.46 $\pm$ 0.01 | 0.37 $\pm$ 0.01             | 0.35 $\pm$ 0.01         | 0.36 $\pm$ 0.02          | 0.37 $\pm$ 0.02                           | 0.37 $\pm$ 0.01     |
| ('SVM', 'No Upsampling')                                     | 0.38 $\pm$ 0.01 | 0.38 $\pm$ 0.02             | 0.34 $\pm$ 0.01         | 0.37 $\pm$ 0.01          | 0.40 $\pm$ 0.02                           | 0.41 $\pm$ 0.02     |
| ('SVM', 'RakelD Partitioning of labels')                     | 0.41 $\pm$ 0.03 | 0.36 $\pm$ 0.02             | 0.31 $\pm$ 0.00         | 0.37 $\pm$ 0.03          | 0.40 $\pm$ 0.02                           | 0.31 $\pm$ 0.04     |
| ('SVM', 'Random Oversampling')                               | 0.49 $\pm$ 0.00 | 0.44 $\pm$ 0.02             | 0.40 $\pm$ 0.01         | 0.41 $\pm$ 0.02          | 0.41 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| ('SVM', 'SMOTE')                                             | 0.48 $\pm$ 0.01 | 0.37 $\pm$ 0.01             | 0.35 $\pm$ 0.01         | 0.35 $\pm$ 0.01          | 0.38 $\pm$ 0.03                           | 0.37 $\pm$ 0.01     |
| ('SVM', 'SVMSMOTE')                                          | 0.50 $\pm$ 0.01 | 0.42 $\pm$ 0.00             | 0.39 $\pm$ 0.02         | 0.38 $\pm$ 0.01          | 0.34 $\pm$ 0.01                           | 0.35 $\pm$ 0.02     |
| ('XGBoost', 'BorderlineSMOTE')                               | 0.53 $\pm$ 0.01 | 0.56 $\pm$ 0.01             | 0.57 $\pm$ 0.03         | 0.57 $\pm$ 0.01          | 0.60 $\pm$ 0.01                           | 0.64 $\pm$ 0.00     |
| ('XGBoost', 'No Upsampling')                                 | 0.52 $\pm$ 0.02 | 0.57 $\pm$ 0.02             | 0.56 $\pm$ 0.02         | 0.58 $\pm$ 0.00          | 0.60 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('XGBoost', 'Random Oversampling')                           | 0.53 $\pm$ 0.02 | 0.57 $\pm$ 0.01             | 0.57 $\pm$ 0.02         | 0.59 $\pm$ 0.01          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |
| ('XGBoost', 'SMOTE')                                         | 0.53 $\pm$ 0.02 | 0.57 $\pm$ 0.01             | 0.57 $\pm$ 0.02         | 0.58 $\pm$ 0.00          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.01     |
| ('XGBoost', 'SVMSMOTE')                                      | 0.53 $\pm$ 0.02 | 0.56 $\pm$ 0.01             | 0.56 $\pm$ 0.02         | 0.58 $\pm$ 0.01          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_type                      | model_subtype                 | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:--------------------------------|:------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | Binary Relevance kNN            | Natively Multilabel           | 0.23 $\pm$ 0.06 | 0.42 $\pm$ 0.03             | 0.45 $\pm$ 0.08         | 0.43 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | 0.68 $\pm$ 0.02     |
| en         | ComplementNB                    | RakelD Partitioning of labels | 0.55 $\pm$ 0.02 | 0.63 $\pm$ 0.01             | 0.65 $\pm$ 0.01         | 0.64 $\pm$ 0.01          | 0.65 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | Dummy Classifier                | No Upsampling                 | 0.39 $\pm$ 0.01 | 0.40 $\pm$ 0.01             | 0.39 $\pm$ 0.01         | 0.41 $\pm$ 0.01          | 0.41 $\pm$ 0.01                           | 0.40 $\pm$ 0.01     |
| en         | KNN                             | BorderlineSMOTE               | 0.36 $\pm$ 0.02 | 0.39 $\pm$ 0.08             | 0.36 $\pm$ 0.07         | 0.45 $\pm$ 0.08          | 0.40 $\pm$ 0.11                           | 0.50 $\pm$ 0.05     |
| en         | KNN                             | No Upsampling                 | 0.42 $\pm$ 0.08 | 0.51 $\pm$ 0.05             | 0.54 $\pm$ 0.04         | 0.55 $\pm$ 0.04          | 0.56 $\pm$ 0.06                           | 0.66 $\pm$ 0.02     |
| en         | KNN                             | Random Oversampling           | 0.45 $\pm$ 0.05 | 0.48 $\pm$ 0.01             | 0.53 $\pm$ 0.06         | 0.52 $\pm$ 0.01          | 0.54 $\pm$ 0.02                           | 0.61 $\pm$ 0.02     |
| en         | KNN                             | SMOTE                         | 0.36 $\pm$ 0.06 | 0.33 $\pm$ 0.11             | 0.42 $\pm$ 0.08         | 0.51 $\pm$ 0.05          | 0.45 $\pm$ 0.07                           | 0.50 $\pm$ 0.05     |
| en         | KNN                             | SVMSMOTE                      | 0               | 0.36 $\pm$ 0.09             | 0                       | 0                        | 0.42 $\pm$ 0.08                           | 0                   |
| en         | LinearSVM                       | BorderlineSMOTE               | 0.56 $\pm$ 0.00 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LinearSVM                       | No Upsampling                 | 0.58 $\pm$ 0.00 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LinearSVM                       | RakelD Partitioning of labels | 0.57 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.66 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.67 $\pm$ 0.01     |
| en         | LinearSVM                       | Random Oversampling           | 0.58 $\pm$ 0.00 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LinearSVM                       | SMOTE                         | 0.55 $\pm$ 0.02 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | BorderlineSMOTE               | 0.56 $\pm$ 0.00 | 0.64 $\pm$ 0.01             | 0.67 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | No Upsampling                 | 0.58 $\pm$ 0.00 | 0.64 $\pm$ 0.02             | 0.67 $\pm$ 0.03         | 0.66 $\pm$ 0.02          | 0.67 $\pm$ 0.03                           | 0.68 $\pm$ 0.02     |
| en         | LogisticRegression              | Random Oversampling           | 0.58 $\pm$ 0.00 | 0.64 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | SMOTE                         | 0.56 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.03                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | SVMSMOTE                      | 0.58 $\pm$ 0.01 | 0.62 $\pm$ 0.01             | 0.65 $\pm$ 0.02         | 0.66 $\pm$ 0.01          | 0.66 $\pm$ 0.03                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.59 $\pm$ 0.02 | 0.66 $\pm$ 0.02             | 0.66 $\pm$ 0.03         | 0.66 $\pm$ 0.01          | 0.66 $\pm$ 0.02                           | 0.67 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | No Upsampling                 | 0.60 $\pm$ 0.02 | 0.67 $\pm$ 0.01             | 0.67 $\pm$ 0.03         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | Random Oversampling           | 0.59 $\pm$ 0.02 | 0.65 $\pm$ 0.01             | 0.66 $\pm$ 0.02         | 0.65 $\pm$ 0.02          | 0.65 $\pm$ 0.02                           | 0.67 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | SMOTE                         | 0.59 $\pm$ 0.02 | 0.66 $\pm$ 0.02             | 0.66 $\pm$ 0.03         | 0.66 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.59 $\pm$ 0.01 | 0.65 $\pm$ 0.01             | 0.66 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.68 $\pm$ 0.02     |
| en         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.60 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.64 $\pm$ 0.03         | 0.61 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | No Upsampling                 | 0.62 $\pm$ 0.00 | 0.64 $\pm$ 0.03             | 0.65 $\pm$ 0.02         | 0.61 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | Random Oversampling           | 0.60 $\pm$ 0.01 | 0.64 $\pm$ 0.03             | 0.64 $\pm$ 0.03         | 0.61 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | SMOTE                         | 0.59 $\pm$ 0.01 | 0.64 $\pm$ 0.03             | 0.64 $\pm$ 0.02         | 0.61 $\pm$ 0.01          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | SVMSMOTE                      | 0.62 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.03         | 0.63 $\pm$ 0.02          | 0.63 $\pm$ 0.02                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.59 $\pm$ 0.00 | 0.67 $\pm$ 0.01             | 0.69 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.68 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | No Upsampling                 | 0.62 $\pm$ 0.01 | 0.68 $\pm$ 0.01             | 0.69 $\pm$ 0.03         | 0.68 $\pm$ 0.03          | 0.68 $\pm$ 0.03                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.61 $\pm$ 0.01 | 0.66 $\pm$ 0.02             | 0.67 $\pm$ 0.03         | 0.66 $\pm$ 0.03          | 0.69 $\pm$ 0.02                           | 0.69 $\pm$ 0.03     |
| en         | LogisticRegressionRidge         | Random Oversampling           | 0.61 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SMOTE                         | 0.59 $\pm$ 0.00 | 0.68 $\pm$ 0.01             | 0.69 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.68 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SVMSMOTE                      | 0.60 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.68 $\pm$ 0.03          | 0.68 $\pm$ 0.02                           | 0.70 $\pm$ 0.02     |
| en         | Multi-label ARAM                | Natively Multilabel           | 0.23 $\pm$ 0.02 | 0.24 $\pm$ 0.02             | 0.23 $\pm$ 0.02         | 0.25 $\pm$ 0.02          | 0.25 $\pm$ 0.02                           | 0.19 $\pm$ 0.06     |
| en         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.52 $\pm$ 0.01 | 0.52 $\pm$ 0.01             | 0.56 $\pm$ 0.03         | 0.55 $\pm$ 0.01          | 0.59 $\pm$ 0.02                           | 0.63 $\pm$ 0.02     |
| en         | NaiveBayes                      | BorderlineSMOTE               | 0.59 $\pm$ 0.02 | 0.65 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.64 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.64 $\pm$ 0.02     |
| en         | NaiveBayes                      | No Upsampling                 | 0.57 $\pm$ 0.01 | 0.68 $\pm$ 0.03             | 0.73 $\pm$ 0.04         | 0.74 $\pm$ 0.01          | 0.74 $\pm$ 0.05                           | 0.75 $\pm$ 0.02     |
| en         | NaiveBayes                      | Random Oversampling           | 0.58 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.02         | 0.64 $\pm$ 0.02          | 0.65 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| en         | NaiveBayes                      | SMOTE                         | 0.59 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.65 $\pm$ 0.01          | 0.66 $\pm$ 0.02                           | 0.65 $\pm$ 0.02     |
| en         | RandomForest                    | BorderlineSMOTE               | 0.71 $\pm$ 0.01 | 0.73 $\pm$ 0.03             | 0.71 $\pm$ 0.05         | 0.73 $\pm$ 0.01          | 0.75 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RandomForest                    | No Upsampling                 | 0.71 $\pm$ 0.01 | 0.73 $\pm$ 0.01             | 0.74 $\pm$ 0.04         | 0.75 $\pm$ 0.01          | **0.77 $\pm$ 0.01**                       | 0.74 $\pm$ 0.00     |
| en         | RandomForest                    | Random Oversampling           | 0.67 $\pm$ 0.01 | 0.75 $\pm$ 0.01             | 0.74 $\pm$ 0.02         | 0.73 $\pm$ 0.01          | 0.74 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RandomForest                    | SMOTE                         | 0.70 $\pm$ 0.00 | 0.74 $\pm$ 0.02             | 0.72 $\pm$ 0.02         | 0.73 $\pm$ 0.01          | 0.76 $\pm$ 0.02                           | 0.73 $\pm$ 0.03     |
| en         | RandomForest                    | SVMSMOTE                      | 0.72 $\pm$ 0.02 | 0.75 $\pm$ 0.03             | 0.72 $\pm$ 0.03         | 0.73 $\pm$ 0.01          | 0.74 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RidgeClassifier                 | BorderlineSMOTE               | 0.59 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | No Upsampling                 | 0.59 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | Random Oversampling           | 0.60 $\pm$ 0.01 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SMOTE                         | 0.59 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SVMSMOTE                      | 0.58 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.68 $\pm$ 0.02          | 0.67 $\pm$ 0.03                           | 0.69 $\pm$ 0.02     |
| en         | SVM                             | BorderlineSMOTE               | 0.60 $\pm$ 0.02 | 0.60 $\pm$ 0.06             | 0.54 $\pm$ 0.04         | 0.54 $\pm$ 0.05          | 0.45 $\pm$ 0.03                           | 0.54 $\pm$ 0.04     |
| en         | SVM                             | No Upsampling                 | 0.54 $\pm$ 0.02 | 0.54 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.04          | 0.48 $\pm$ 0.01                           | 0.54 $\pm$ 0.04     |
| en         | SVM                             | RakelD Partitioning of labels | 0.48 $\pm$ 0.02 | 0.49 $\pm$ 0.03             | 0.41 $\pm$ 0.11         | 0.48 $\pm$ 0.03          | 0.43 $\pm$ 0.07                           | 0.53 $\pm$ 0.03     |
| en         | SVM                             | Random Oversampling           | 0.62 $\pm$ 0.01 | 0.63 $\pm$ 0.03             | 0.61 $\pm$ 0.06         | 0.63 $\pm$ 0.04          | 0.56 $\pm$ 0.02                           | 0.65 $\pm$ 0.02     |
| en         | SVM                             | SMOTE                         | 0.58 $\pm$ 0.01 | 0.61 $\pm$ 0.04             | 0.58 $\pm$ 0.05         | 0.53 $\pm$ 0.04          | 0.47 $\pm$ 0.02                           | 0.55 $\pm$ 0.04     |
| en         | SVM                             | SVMSMOTE                      | 0.52 $\pm$ 0.03 | 0.61 $\pm$ 0.02             | 0.55 $\pm$ 0.05         | 0.60 $\pm$ 0.04          | 0.55 $\pm$ 0.04                           | 0.61 $\pm$ 0.05     |
| en         | XGBoost                         | BorderlineSMOTE               | 0.60 $\pm$ 0.02 | 0.64 $\pm$ 0.01             | 0.63 $\pm$ 0.01         | 0.64 $\pm$ 0.01          | 0.66 $\pm$ 0.02                           | 0.70 $\pm$ 0.02     |
| en         | XGBoost                         | No Upsampling                 | 0.59 $\pm$ 0.02 | 0.65 $\pm$ 0.02             | 0.64 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.69 $\pm$ 0.01                           | 0.74 $\pm$ 0.01     |
| en         | XGBoost                         | Random Oversampling           | 0.58 $\pm$ 0.02 | 0.64 $\pm$ 0.02             | 0.63 $\pm$ 0.02         | 0.66 $\pm$ 0.01          | 0.68 $\pm$ 0.00                           | 0.71 $\pm$ 0.01     |
| en         | XGBoost                         | SMOTE                         | 0.60 $\pm$ 0.02 | 0.63 $\pm$ 0.02             | 0.64 $\pm$ 0.01         | 0.65 $\pm$ 0.00          | 0.67 $\pm$ 0.01                           | 0.69 $\pm$ 0.02     |
| en         | XGBoost                         | SVMSMOTE                      | 0.60 $\pm$ 0.02 | 0.64 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.65 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.71 $\pm$ 0.02     |
| fr         | Binary Relevance kNN            | Natively Multilabel           | 0.08 $\pm$ 0.04 | 0.14 $\pm$ 0.06             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| fr         | ComplementNB                    | RakelD Partitioning of labels | 0.35 $\pm$ 0.03 | 0.37 $\pm$ 0.01             | 0.43 $\pm$ 0.01         | 0.45 $\pm$ 0.03          | 0.42 $\pm$ 0.01                           | 0.48 $\pm$ 0.04     |
| fr         | Dummy Classifier                | No Upsampling                 | 0.25 $\pm$ 0.02 | 0.28 $\pm$ 0.02             | 0.27 $\pm$ 0.03         | 0.27 $\pm$ 0.01          | 0.28 $\pm$ 0.01                           | 0.29 $\pm$ 0.01     |
| fr         | KNN                             | BorderlineSMOTE               | 0.28 $\pm$ 0.01 | 0.28 $\pm$ 0.01             | 0.28 $\pm$ 0.01         | 0.27 $\pm$ 0.01          | 0.26 $\pm$ 0.03                           | 0.27 $\pm$ 0.02     |
| fr         | KNN                             | No Upsampling                 | 0.30 $\pm$ 0.01 | 0.19 $\pm$ 0.03             | 0.18 $\pm$ 0.03         | 0.26 $\pm$ 0.03          | 0.08 $\pm$ 0.08                           | 0.17 $\pm$ 0.15     |
| fr         | KNN                             | Random Oversampling           | 0.35 $\pm$ 0.02 | 0.31 $\pm$ 0.01             | 0.30 $\pm$ 0.04         | 0.28 $\pm$ 0.08          | 0.14 $\pm$ 0.03                           | 0.40 $\pm$ 0.06     |
| fr         | KNN                             | SMOTE                         | 0.28 $\pm$ 0.01 | 0.28 $\pm$ 0.01             | 0.28 $\pm$ 0.01         | 0.28 $\pm$ 0.01          | 0.27 $\pm$ 0.05                           | 0.28 $\pm$ 0.01     |
| fr         | LinearSVM                       | BorderlineSMOTE               | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LinearSVM                       | No Upsampling                 | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LinearSVM                       | RakelD Partitioning of labels | 0.39 $\pm$ 0.01 | 0.44 $\pm$ 0.03             | 0.47 $\pm$ 0.06         | 0.52 $\pm$ 0.06          | 0.44 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| fr         | LinearSVM                       | Random Oversampling           | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LinearSVM                       | SMOTE                         | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LogisticRegression              | BorderlineSMOTE               | 0.41 $\pm$ 0.01 | 0.46 $\pm$ 0.03             | 0.49 $\pm$ 0.05         | 0.53 $\pm$ 0.03          | 0.46 $\pm$ 0.05                           | 0.51 $\pm$ 0.07     |
| fr         | LogisticRegression              | No Upsampling                 | 0.41 $\pm$ 0.02 | 0.43 $\pm$ 0.02             | 0.50 $\pm$ 0.07         | 0.50 $\pm$ 0.05          | 0.45 $\pm$ 0.05                           | 0.55 $\pm$ 0.09     |
| fr         | LogisticRegression              | Random Oversampling           | 0.41 $\pm$ 0.02 | 0.48 $\pm$ 0.03             | 0.52 $\pm$ 0.08         | 0.55 $\pm$ 0.04          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.06     |
| fr         | LogisticRegression              | SMOTE                         | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.03             | 0.51 $\pm$ 0.05         | 0.54 $\pm$ 0.05          | 0.46 $\pm$ 0.04                           | 0.52 $\pm$ 0.06     |
| fr         | LogisticRegression              | SVMSMOTE                      | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.47 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.08     |
| fr         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.43 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.48 $\pm$ 0.03         | 0.50 $\pm$ 0.01          | 0.46 $\pm$ 0.04                           | 0.57 $\pm$ 0.07     |
| fr         | LogisticRegressionElasticNet    | No Upsampling                 | 0.44 $\pm$ 0.03 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.03         | 0.55 $\pm$ 0.01          | 0.47 $\pm$ 0.04                           | 0.57 $\pm$ 0.08     |
| fr         | LogisticRegressionElasticNet    | Random Oversampling           | 0.42 $\pm$ 0.01 | 0.41 $\pm$ 0.04             | 0.46 $\pm$ 0.03         | 0.50 $\pm$ 0.04          | 0.45 $\pm$ 0.04                           | 0.58 $\pm$ 0.06     |
| fr         | LogisticRegressionElasticNet    | SMOTE                         | 0.43 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.47 $\pm$ 0.03         | 0.50 $\pm$ 0.02          | 0.47 $\pm$ 0.04                           | **0.59 $\pm$ 0.07** |
| fr         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.43 $\pm$ 0.03 | 0.44 $\pm$ 0.04             | 0.50 $\pm$ 0.05         | 0.53 $\pm$ 0.02          | 0.49 $\pm$ 0.05                           | 0.56 $\pm$ 0.08     |
| fr         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.46 $\pm$ 0.02 | 0.39 $\pm$ 0.01             | 0.42 $\pm$ 0.04         | 0.41 $\pm$ 0.04          | 0.46 $\pm$ 0.03                           | 0.48 $\pm$ 0.05     |
| fr         | LogisticRegressionLasso         | No Upsampling                 | 0.43 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.41 $\pm$ 0.06          | 0.45 $\pm$ 0.03                           | 0.47 $\pm$ 0.04     |
| fr         | LogisticRegressionLasso         | Random Oversampling           | 0.46 $\pm$ 0.02 | 0.40 $\pm$ 0.02             | 0.43 $\pm$ 0.02         | 0.41 $\pm$ 0.05          | 0.46 $\pm$ 0.02                           | 0.48 $\pm$ 0.04     |
| fr         | LogisticRegressionLasso         | SMOTE                         | 0.43 $\pm$ 0.02 | 0.39 $\pm$ 0.01             | 0.43 $\pm$ 0.04         | 0.41 $\pm$ 0.04          | 0.45 $\pm$ 0.02                           | 0.45 $\pm$ 0.03     |
| fr         | LogisticRegressionLasso         | SVMSMOTE                      | 0.49 $\pm$ 0.01 | 0.42 $\pm$ 0.02             | 0.45 $\pm$ 0.02         | 0.40 $\pm$ 0.03          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.41 $\pm$ 0.02 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.08         | 0.54 $\pm$ 0.04          | 0.52 $\pm$ 0.07                           | 0.52 $\pm$ 0.09     |
| fr         | LogisticRegressionRidge         | No Upsampling                 | 0.40 $\pm$ 0.04 | 0.43 $\pm$ 0.03             | 0.47 $\pm$ 0.07         | 0.54 $\pm$ 0.05          | 0.52 $\pm$ 0.04                           | 0.54 $\pm$ 0.09     |
| fr         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.42 $\pm$ 0.05 | 0.41 $\pm$ 0.02             | 0.44 $\pm$ 0.04         | 0.52 $\pm$ 0.05          | 0.53 $\pm$ 0.05                           | 0.53 $\pm$ 0.07     |
| fr         | LogisticRegressionRidge         | Random Oversampling           | 0.44 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.53 $\pm$ 0.09         | 0.55 $\pm$ 0.03          | 0.54 $\pm$ 0.08                           | 0.53 $\pm$ 0.08     |
| fr         | LogisticRegressionRidge         | SMOTE                         | 0.40 $\pm$ 0.04 | 0.45 $\pm$ 0.02             | 0.53 $\pm$ 0.09         | 0.53 $\pm$ 0.02          | 0.51 $\pm$ 0.07                           | 0.53 $\pm$ 0.09     |
| fr         | LogisticRegressionRidge         | SVMSMOTE                      | 0.42 $\pm$ 0.04 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.09         | 0.54 $\pm$ 0.04          | 0.49 $\pm$ 0.09                           | 0.50 $\pm$ 0.08     |
| fr         | Multi-label ARAM                | Natively Multilabel           | 0.11 $\pm$ 0.01 | 0.10 $\pm$ 0.00             | 0.10 $\pm$ 0.00         | 0.06 $\pm$ 0.03          | 0.08 $\pm$ 0.03                           | 0.08 $\pm$ 0.03     |
| fr         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.39 $\pm$ 0.03 | 0.34 $\pm$ 0.02             | 0.33 $\pm$ 0.03         | 0.40 $\pm$ 0.01          | 0.33 $\pm$ 0.08                           | 0.43 $\pm$ 0.05     |
| fr         | NaiveBayes                      | BorderlineSMOTE               | 0.38 $\pm$ 0.02 | 0.41 $\pm$ 0.02             | 0.43 $\pm$ 0.00         | 0.45 $\pm$ 0.02          | 0.44 $\pm$ 0.01                           | 0.49 $\pm$ 0.03     |
| fr         | NaiveBayes                      | No Upsampling                 | 0.35 $\pm$ 0.03 | 0.40 $\pm$ 0.03             | 0.44 $\pm$ 0.02         | 0.51 $\pm$ 0.06          | 0.51 $\pm$ 0.01                           | 0.51 $\pm$ 0.04     |
| fr         | NaiveBayes                      | Random Oversampling           | 0.38 $\pm$ 0.03 | 0.40 $\pm$ 0.02             | 0.44 $\pm$ 0.02         | 0.45 $\pm$ 0.02          | 0.44 $\pm$ 0.00                           | 0.47 $\pm$ 0.01     |
| fr         | NaiveBayes                      | SMOTE                         | 0.38 $\pm$ 0.02 | 0.41 $\pm$ 0.03             | 0.44 $\pm$ 0.01         | 0.46 $\pm$ 0.03          | 0.44 $\pm$ 0.02                           | 0.49 $\pm$ 0.02     |
| fr         | RandomForest                    | BorderlineSMOTE               | 0.48 $\pm$ 0.07 | 0.49 $\pm$ 0.12             | 0.48 $\pm$ 0.12         | 0.57 $\pm$ 0.09          | 0.45 $\pm$ 0.03                           | 0.49 $\pm$ 0.04     |
| fr         | RandomForest                    | No Upsampling                 | 0.47 $\pm$ 0.08 | 0.34 $\pm$ 0.09             | 0.40 $\pm$ 0.09         | 0.41 $\pm$ 0.05          | 0.44 $\pm$ 0.08                           | 0.47 $\pm$ 0.04     |
| fr         | RandomForest                    | Random Oversampling           | 0.53 $\pm$ 0.07 | 0.41 $\pm$ 0.06             | 0.52 $\pm$ 0.07         | 0.48 $\pm$ 0.04          | 0.44 $\pm$ 0.06                           | 0.57 $\pm$ 0.07     |
| fr         | RandomForest                    | SMOTE                         | 0.47 $\pm$ 0.03 | 0.38 $\pm$ 0.03             | 0.47 $\pm$ 0.06         | 0.57 $\pm$ 0.02          | 0.50 $\pm$ 0.07                           | 0.47 $\pm$ 0.04     |
| fr         | RandomForest                    | SVMSMOTE                      | 0.50 $\pm$ 0.05 | 0.45 $\pm$ 0.07             | 0.47 $\pm$ 0.08         | 0.49 $\pm$ 0.12          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.05     |
| fr         | RidgeClassifier                 | BorderlineSMOTE               | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | No Upsampling                 | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | Random Oversampling           | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | SMOTE                         | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | SVMSMOTE                      | 0.40 $\pm$ 0.03 | 0.45 $\pm$ 0.01             | 0.52 $\pm$ 0.10         | 0.54 $\pm$ 0.04          | 0.51 $\pm$ 0.07                           | 0.51 $\pm$ 0.09     |
| fr         | SVM                             | BorderlineSMOTE               | 0.35 $\pm$ 0.07 | 0.00 $\pm$ 0.00             | 0.07 $\pm$ 0.05         | 0.03 $\pm$ 0.05          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | No Upsampling                 | 0.14 $\pm$ 0.03 | 0.00 $\pm$ 0.00             | 0.10 $\pm$ 0.07         | 0.00 $\pm$ 0.00          | 0.09 $\pm$ 0.02                           | 0.05 $\pm$ 0.04     |
| fr         | SVM                             | RakelD Partitioning of labels | 0.12 $\pm$ 0.06 | 0.05 $\pm$ 0.06             | 0.07 $\pm$ 0.06         | 0.06 $\pm$ 0.05          | 0.13 $\pm$ 0.05                           | 0.12 $\pm$ 0.04     |
| fr         | SVM                             | Random Oversampling           | 0.30 $\pm$ 0.03 | 0.17 $\pm$ 0.10             | 0.16 $\pm$ 0.06         | 0.19 $\pm$ 0.06          | 0.16 $\pm$ 0.05                           | 0.19 $\pm$ 0.03     |
| fr         | SVM                             | SMOTE                         | 0.30 $\pm$ 0.05 | 0.05 $\pm$ 0.07             | 0.12 $\pm$ 0.02         | 0.05 $\pm$ 0.04          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | SVMSMOTE                      | 0.29 $\pm$ 0.04 | 0.15 $\pm$ 0.11             | 0.20 $\pm$ 0.13         | 0.15 $\pm$ 0.08          | 0.05 $\pm$ 0.07                           | 0.00 $\pm$ 0.00     |
| fr         | XGBoost                         | BorderlineSMOTE               | 0.50 $\pm$ 0.01 | 0.38 $\pm$ 0.02             | 0.37 $\pm$ 0.05         | 0.43 $\pm$ 0.04          | 0.42 $\pm$ 0.03                           | 0.53 $\pm$ 0.07     |
| fr         | XGBoost                         | No Upsampling                 | 0.44 $\pm$ 0.04 | 0.36 $\pm$ 0.04             | 0.38 $\pm$ 0.04         | 0.41 $\pm$ 0.04          | 0.37 $\pm$ 0.01                           | 0.52 $\pm$ 0.08     |
| fr         | XGBoost                         | Random Oversampling           | 0.44 $\pm$ 0.03 | 0.36 $\pm$ 0.05             | 0.39 $\pm$ 0.03         | 0.44 $\pm$ 0.05          | 0.39 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| fr         | XGBoost                         | SMOTE                         | 0.48 $\pm$ 0.01 | 0.38 $\pm$ 0.01             | 0.37 $\pm$ 0.00         | 0.41 $\pm$ 0.03          | 0.42 $\pm$ 0.03                           | 0.51 $\pm$ 0.04     |
| fr         | XGBoost                         | SVMSMOTE                      | 0.47 $\pm$ 0.02 | 0.38 $\pm$ 0.01             | 0.39 $\pm$ 0.04         | 0.38 $\pm$ 0.05          | 0.42 $\pm$ 0.04                           | 0.50 $\pm$ 0.05     |
| ge         | Binary Relevance kNN            | Natively Multilabel           | 0.31 $\pm$ 0.07 | 0.18 $\pm$ 0.02             | 0.19 $\pm$ 0.05         | 0.17 $\pm$ 0.05          | 0.18 $\pm$ 0.06                           | 0.17 $\pm$ 0.12     |
| ge         | ComplementNB                    | RakelD Partitioning of labels | 0.44 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.42 $\pm$ 0.03         | 0.48 $\pm$ 0.02          | 0.44 $\pm$ 0.06                           | 0.43 $\pm$ 0.02     |
| ge         | Dummy Classifier                | No Upsampling                 | 0.39 $\pm$ 0.02 | 0.37 $\pm$ 0.03             | 0.38 $\pm$ 0.04         | 0.39 $\pm$ 0.02          | 0.37 $\pm$ 0.04                           | 0.38 $\pm$ 0.03     |
| ge         | KNN                             | BorderlineSMOTE               | 0.42 $\pm$ 0.04 | 0.38 $\pm$ 0.05             | 0.40 $\pm$ 0.09         | 0.27 $\pm$ 0.05          | 0.32 $\pm$ 0.06                           | 0.24 $\pm$ 0.01     |
| ge         | KNN                             | No Upsampling                 | 0.41 $\pm$ 0.01 | 0.34 $\pm$ 0.06             | 0.39 $\pm$ 0.07         | 0.28 $\pm$ 0.09          | 0.19 $\pm$ 0.01                           | 0.15 $\pm$ 0.04     |
| ge         | KNN                             | Random Oversampling           | 0.47 $\pm$ 0.02 | 0.41 $\pm$ 0.08             | 0.45 $\pm$ 0.07         | 0.34 $\pm$ 0.06          | 0.24 $\pm$ 0.06                           | 0.20 $\pm$ 0.06     |
| ge         | KNN                             | SMOTE                         | 0.35 $\pm$ 0.02 | 0.35 $\pm$ 0.06             | 0.32 $\pm$ 0.01         | 0.32 $\pm$ 0.07          | 0.28 $\pm$ 0.06                           | 0.24 $\pm$ 0.02     |
| ge         | KNN                             | SVMSMOTE                      | 0.37 $\pm$ 0.05 | 0.35 $\pm$ 0.03             | 0.43 $\pm$ 0.06         | 0                        | 0.32 $\pm$ 0.10                           | 0                   |
| ge         | LinearSVM                       | BorderlineSMOTE               | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LinearSVM                       | No Upsampling                 | 0.42 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LinearSVM                       | RakelD Partitioning of labels | 0.42 $\pm$ 0.03 | 0.46 $\pm$ 0.03             | 0.48 $\pm$ 0.06         | 0.48 $\pm$ 0.06          | 0.46 $\pm$ 0.02                           | 0.52 $\pm$ 0.03     |
| ge         | LinearSVM                       | Random Oversampling           | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LinearSVM                       | SMOTE                         | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LogisticRegression              | BorderlineSMOTE               | 0.41 $\pm$ 0.03 | 0.50 $\pm$ 0.02             | 0.45 $\pm$ 0.03         | 0.46 $\pm$ 0.05          | 0.47 $\pm$ 0.05                           | 0.51 $\pm$ 0.06     |
| ge         | LogisticRegression              | No Upsampling                 | 0.41 $\pm$ 0.03 | 0.48 $\pm$ 0.03             | 0.43 $\pm$ 0.04         | 0.45 $\pm$ 0.04          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.03     |
| ge         | LogisticRegression              | Random Oversampling           | 0.41 $\pm$ 0.04 | 0.48 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.05                           | 0.50 $\pm$ 0.02     |
| ge         | LogisticRegression              | SMOTE                         | 0.41 $\pm$ 0.03 | 0.50 $\pm$ 0.02             | 0.45 $\pm$ 0.03         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LogisticRegression              | SVMSMOTE                      | 0.41 $\pm$ 0.03 | 0.48 $\pm$ 0.04             | 0.43 $\pm$ 0.03         | 0.49 $\pm$ 0.03          | 0.45 $\pm$ 0.06                           | 0.48 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.42 $\pm$ 0.05 | 0.47 $\pm$ 0.02             | 0.45 $\pm$ 0.04         | 0.52 $\pm$ 0.05          | 0.48 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | No Upsampling                 | 0.42 $\pm$ 0.04 | 0.47 $\pm$ 0.02             | 0.48 $\pm$ 0.06         | 0.51 $\pm$ 0.05          | 0.48 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | Random Oversampling           | 0.43 $\pm$ 0.03 | 0.47 $\pm$ 0.01             | 0.45 $\pm$ 0.04         | 0.53 $\pm$ 0.05          | 0.49 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | SMOTE                         | 0.43 $\pm$ 0.05 | 0.47 $\pm$ 0.01             | 0.45 $\pm$ 0.05         | 0.51 $\pm$ 0.05          | 0.49 $\pm$ 0.04                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.02             | 0.46 $\pm$ 0.05         | 0.53 $\pm$ 0.05          | 0.47 $\pm$ 0.03                           | 0.55 $\pm$ 0.04     |
| ge         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.45 $\pm$ 0.02 | 0.44 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.49 $\pm$ 0.05                           | 0.57 $\pm$ 0.03     |
| ge         | LogisticRegressionLasso         | No Upsampling                 | 0.48 $\pm$ 0.02 | 0.45 $\pm$ 0.04             | 0.45 $\pm$ 0.04         | 0.50 $\pm$ 0.03          | 0.49 $\pm$ 0.05                           | 0.55 $\pm$ 0.04     |
| ge         | LogisticRegressionLasso         | Random Oversampling           | 0.46 $\pm$ 0.02 | 0.46 $\pm$ 0.04             | 0.45 $\pm$ 0.01         | 0.50 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.57 $\pm$ 0.03     |
| ge         | LogisticRegressionLasso         | SMOTE                         | 0.45 $\pm$ 0.01 | 0.46 $\pm$ 0.05             | 0.46 $\pm$ 0.00         | 0.49 $\pm$ 0.04          | 0.50 $\pm$ 0.05                           | 0.58 $\pm$ 0.03     |
| ge         | LogisticRegressionLasso         | SVMSMOTE                      | 0.45 $\pm$ 0.02 | 0.48 $\pm$ 0.04             | 0.46 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.48 $\pm$ 0.04                           | 0.55 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.45 $\pm$ 0.04                           | 0.55 $\pm$ 0.03     |
| ge         | LogisticRegressionRidge         | No Upsampling                 | 0.38 $\pm$ 0.05 | 0.48 $\pm$ 0.03             | 0.40 $\pm$ 0.03         | 0.45 $\pm$ 0.02          | 0.45 $\pm$ 0.04                           | 0.54 $\pm$ 0.04     |
| ge         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.42 $\pm$ 0.03 | 0.47 $\pm$ 0.05             | 0.43 $\pm$ 0.03         | 0.46 $\pm$ 0.04          | 0.44 $\pm$ 0.01                           | 0.54 $\pm$ 0.07     |
| ge         | LogisticRegressionRidge         | Random Oversampling           | 0.42 $\pm$ 0.05 | 0.49 $\pm$ 0.04             | 0.42 $\pm$ 0.04         | 0.45 $\pm$ 0.02          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | LogisticRegressionRidge         | SMOTE                         | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.45 $\pm$ 0.02          | 0.46 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | LogisticRegressionRidge         | SVMSMOTE                      | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.05             | 0.43 $\pm$ 0.05         | 0.47 $\pm$ 0.00          | 0.46 $\pm$ 0.01                           | 0.55 $\pm$ 0.01     |
| ge         | Multi-label ARAM                | Natively Multilabel           | 0.08 $\pm$ 0.03 | 0.12 $\pm$ 0.07             | 0.12 $\pm$ 0.07         | 0.12 $\pm$ 0.07          | 0.13 $\pm$ 0.07                           | 0.12 $\pm$ 0.07     |
| ge         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.41 $\pm$ 0.04 | 0.43 $\pm$ 0.04             | 0.41 $\pm$ 0.02         | 0.44 $\pm$ 0.04          | 0.40 $\pm$ 0.08                           | 0.45 $\pm$ 0.02     |
| ge         | NaiveBayes                      | BorderlineSMOTE               | 0.46 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.03         | 0.51 $\pm$ 0.02          | 0.50 $\pm$ 0.05                           | 0.53 $\pm$ 0.02     |
| ge         | NaiveBayes                      | No Upsampling                 | 0.44 $\pm$ 0.02 | 0.48 $\pm$ 0.02             | 0.47 $\pm$ 0.01         | 0.51 $\pm$ 0.02          | 0.48 $\pm$ 0.03                           | 0.52 $\pm$ 0.03     |
| ge         | NaiveBayes                      | Random Oversampling           | 0.45 $\pm$ 0.02 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.50 $\pm$ 0.02          | 0.49 $\pm$ 0.05                           | 0.52 $\pm$ 0.03     |
| ge         | NaiveBayes                      | SMOTE                         | 0.46 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.51 $\pm$ 0.03          | 0.50 $\pm$ 0.04                           | 0.54 $\pm$ 0.03     |
| ge         | RandomForest                    | BorderlineSMOTE               | 0.44 $\pm$ 0.07 | 0.44 $\pm$ 0.03             | 0.43 $\pm$ 0.03         | 0.45 $\pm$ 0.05          | 0.57 $\pm$ 0.10                           | **0.64 $\pm$ 0.07** |
| ge         | RandomForest                    | No Upsampling                 | 0.43 $\pm$ 0.01 | 0.40 $\pm$ 0.02             | 0.44 $\pm$ 0.00         | 0.45 $\pm$ 0.03          | 0.46 $\pm$ 0.03                           | 0.52 $\pm$ 0.04     |
| ge         | RandomForest                    | Random Oversampling           | 0.46 $\pm$ 0.01 | 0.47 $\pm$ 0.03             | 0.45 $\pm$ 0.03         | 0.49 $\pm$ 0.04          | 0.49 $\pm$ 0.04                           | 0.54 $\pm$ 0.04     |
| ge         | RandomForest                    | SMOTE                         | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.04             | 0.44 $\pm$ 0.03         | 0.45 $\pm$ 0.02          | 0.58 $\pm$ 0.13                           | **0.64 $\pm$ 0.11** |
| ge         | RandomForest                    | SVMSMOTE                      | 0.41 $\pm$ 0.03 | 0.45 $\pm$ 0.03             | 0.43 $\pm$ 0.02         | 0.49 $\pm$ 0.03          | 0.52 $\pm$ 0.09                           | 0.60 $\pm$ 0.08     |
| ge         | RidgeClassifier                 | BorderlineSMOTE               | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | No Upsampling                 | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | Random Oversampling           | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | SMOTE                         | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | SVMSMOTE                      | 0.42 $\pm$ 0.03 | 0.48 $\pm$ 0.05             | 0.44 $\pm$ 0.04         | 0.48 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.54 $\pm$ 0.02     |
| ge         | SVM                             | BorderlineSMOTE               | 0.36 $\pm$ 0.05 | 0.43 $\pm$ 0.07             | 0.34 $\pm$ 0.05         | 0.17 $\pm$ 0.03          | 0.28 $\pm$ 0.08                           | 0.26 $\pm$ 0.02     |
| ge         | SVM                             | No Upsampling                 | 0.33 $\pm$ 0.10 | 0.25 $\pm$ 0.04             | 0.31 $\pm$ 0.05         | 0.31 $\pm$ 0.06          | 0.25 $\pm$ 0.03                           | 0.30 $\pm$ 0.02     |
| ge         | SVM                             | RakelD Partitioning of labels | 0.27 $\pm$ 0.03 | 0.23 $\pm$ 0.04             | 0.17 $\pm$ 0.02         | 0.19 $\pm$ 0.02          | 0.30 $\pm$ 0.10                           | 0.30 $\pm$ 0.06     |
| ge         | SVM                             | Random Oversampling           | 0.39 $\pm$ 0.09 | 0.40 $\pm$ 0.03             | 0.41 $\pm$ 0.06         | 0.35 $\pm$ 0.08          | 0.25 $\pm$ 0.03                           | 0.33 $\pm$ 0.02     |
| ge         | SVM                             | SMOTE                         | 0.38 $\pm$ 0.10 | 0.41 $\pm$ 0.09             | 0.37 $\pm$ 0.09         | 0.20 $\pm$ 0.06          | 0.31 $\pm$ 0.10                           | 0.26 $\pm$ 0.02     |
| ge         | SVM                             | SVMSMOTE                      | 0.39 $\pm$ 0.04 | 0.55 $\pm$ 0.08             | 0.32 $\pm$ 0.11         | 0.24 $\pm$ 0.05          | 0.23 $\pm$ 0.07                           | 0.24 $\pm$ 0.01     |
| ge         | XGBoost                         | BorderlineSMOTE               | 0.45 $\pm$ 0.01 | 0.45 $\pm$ 0.01             | 0.47 $\pm$ 0.02         | 0.47 $\pm$ 0.03          | 0.49 $\pm$ 0.03                           | 0.55 $\pm$ 0.06     |
| ge         | XGBoost                         | No Upsampling                 | 0.44 $\pm$ 0.05 | 0.45 $\pm$ 0.03             | 0.49 $\pm$ 0.02         | 0.45 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.53 $\pm$ 0.03     |
| ge         | XGBoost                         | Random Oversampling           | 0.48 $\pm$ 0.03 | 0.48 $\pm$ 0.03             | 0.46 $\pm$ 0.03         | 0.46 $\pm$ 0.01          | 0.47 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | XGBoost                         | SMOTE                         | 0.47 $\pm$ 0.02 | 0.47 $\pm$ 0.01             | 0.46 $\pm$ 0.03         | 0.48 $\pm$ 0.05          | 0.50 $\pm$ 0.04                           | 0.56 $\pm$ 0.06     |
| ge         | XGBoost                         | SVMSMOTE                      | 0.44 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.46 $\pm$ 0.01          | 0.50 $\pm$ 0.05                           | 0.55 $\pm$ 0.04     |
| it         | Binary Relevance kNN            | Natively Multilabel           | 0.19 $\pm$ 0.04 | 0.18 $\pm$ 0.01             | 0.12 $\pm$ 0.03         | 0.15 $\pm$ 0.07          | 0.12 $\pm$ 0.05                           | 0.13 $\pm$ 0.06     |
| it         | ComplementNB                    | RakelD Partitioning of labels | 0.40 $\pm$ 0.00 | 0.43 $\pm$ 0.02             | 0.45 $\pm$ 0.02         | 0.50 $\pm$ 0.02          | 0.40 $\pm$ 0.03                           | 0.46 $\pm$ 0.02     |
| it         | Dummy Classifier                | No Upsampling                 | 0.34 $\pm$ 0.03 | 0.32 $\pm$ 0.03             | 0.31 $\pm$ 0.03         | 0.31 $\pm$ 0.04          | 0.32 $\pm$ 0.03                           | 0.33 $\pm$ 0.03     |
| it         | KNN                             | BorderlineSMOTE               | 0.33 $\pm$ 0.01 | 0.33 $\pm$ 0.01             | 0.34 $\pm$ 0.02         | 0.32 $\pm$ 0.02          | 0.32 $\pm$ 0.02                           | 0.33 $\pm$ 0.01     |
| it         | KNN                             | No Upsampling                 | 0.36 $\pm$ 0.07 | 0.20 $\pm$ 0.05             | 0.17 $\pm$ 0.01         | 0.22 $\pm$ 0.14          | 0.14 $\pm$ 0.08                           | 0.11 $\pm$ 0.05     |
| it         | KNN                             | Random Oversampling           | 0.44 $\pm$ 0.03 | 0.34 $\pm$ 0.03             | 0.36 $\pm$ 0.09         | 0.32 $\pm$ 0.02          | 0.30 $\pm$ 0.01                           | 0.33 $\pm$ 0.09     |
| it         | KNN                             | SMOTE                         | 0.33 $\pm$ 0.01 | 0.33 $\pm$ 0.02             | 0.32 $\pm$ 0.02         | 0.32 $\pm$ 0.02          | 0.35 $\pm$ 0.00                           | 0.30 $\pm$ 0.04     |
| it         | KNN                             | SVMSMOTE                      | 0.34 $\pm$ 0.02 | 0                           | 0.30 $\pm$ 0.04         | 0.32 $\pm$ 0.02          | 0                                         | 0.31 $\pm$ 0.04     |
| it         | LinearSVM                       | BorderlineSMOTE               | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | No Upsampling                 | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | RakelD Partitioning of labels | 0.41 $\pm$ 0.00 | 0.48 $\pm$ 0.01             | 0.46 $\pm$ 0.02         | 0.51 $\pm$ 0.04          | 0.52 $\pm$ 0.02                           | 0.52 $\pm$ 0.02     |
| it         | LinearSVM                       | Random Oversampling           | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | SMOTE                         | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegression              | BorderlineSMOTE               | 0.42 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.55 $\pm$ 0.06          | 0.49 $\pm$ 0.01                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegression              | No Upsampling                 | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.01             | 0.47 $\pm$ 0.02         | 0.54 $\pm$ 0.05          | 0.46 $\pm$ 0.02                           | 0.57 $\pm$ 0.02     |
| it         | LogisticRegression              | Random Oversampling           | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.54 $\pm$ 0.06          | 0.50 $\pm$ 0.02                           | 0.56 $\pm$ 0.02     |
| it         | LogisticRegression              | SMOTE                         | 0.42 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.56 $\pm$ 0.06          | 0.50 $\pm$ 0.02                           | 0.57 $\pm$ 0.03     |
| it         | LogisticRegression              | SVMSMOTE                      | 0.41 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.47 $\pm$ 0.02         | 0.51 $\pm$ 0.02          | 0.49 $\pm$ 0.02                           | 0.55 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.43 $\pm$ 0.01 | 0.49 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.57 $\pm$ 0.05          | 0.52 $\pm$ 0.03                           | 0.55 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | No Upsampling                 | 0.44 $\pm$ 0.01 | 0.52 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.59 $\pm$ 0.04          | 0.53 $\pm$ 0.02                           | 0.56 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | Random Oversampling           | 0.43 $\pm$ 0.02 | 0.48 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.57 $\pm$ 0.04          | 0.50 $\pm$ 0.01                           | 0.55 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | SMOTE                         | 0.43 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.05          | 0.51 $\pm$ 0.02                           | 0.54 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.44 $\pm$ 0.01 | 0.49 $\pm$ 0.03             | 0.48 $\pm$ 0.02         | 0.56 $\pm$ 0.05          | 0.52 $\pm$ 0.01                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.46 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.43 $\pm$ 0.04         | 0.51 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | No Upsampling                 | 0.46 $\pm$ 0.02 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.05         | 0.52 $\pm$ 0.04          | 0.48 $\pm$ 0.02                           | 0.51 $\pm$ 0.04     |
| it         | LogisticRegressionLasso         | Random Oversampling           | 0.46 $\pm$ 0.01 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.05         | 0.51 $\pm$ 0.04          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | SMOTE                         | 0.45 $\pm$ 0.01 | 0.45 $\pm$ 0.02             | 0.44 $\pm$ 0.04         | 0.50 $\pm$ 0.04          | 0.49 $\pm$ 0.03                           | 0.51 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | SVMSMOTE                      | 0.45 $\pm$ 0.02 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.04         | 0.50 $\pm$ 0.03          | 0.48 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| it         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.45 $\pm$ 0.01 | 0.51 $\pm$ 0.03             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.57 $\pm$ 0.04                           | **0.62 $\pm$ 0.08** |
| it         | LogisticRegressionRidge         | No Upsampling                 | 0.46 $\pm$ 0.01 | 0.52 $\pm$ 0.02             | 0.51 $\pm$ 0.04         | 0.59 $\pm$ 0.08          | 0.55 $\pm$ 0.02                           | **0.62 $\pm$ 0.06** |
| it         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.46 $\pm$ 0.02 | 0.49 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.59 $\pm$ 0.05          | 0.53 $\pm$ 0.05                           | 0.58 $\pm$ 0.04     |
| it         | LogisticRegressionRidge         | Random Oversampling           | 0.45 $\pm$ 0.02 | 0.51 $\pm$ 0.03             | 0.50 $\pm$ 0.02         | 0.59 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | LogisticRegressionRidge         | SMOTE                         | 0.45 $\pm$ 0.01 | 0.52 $\pm$ 0.03             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.57 $\pm$ 0.06                           | 0.61 $\pm$ 0.07     |
| it         | LogisticRegressionRidge         | SVMSMOTE                      | 0.43 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.09          | 0.54 $\pm$ 0.03                           | 0.61 $\pm$ 0.05     |
| it         | Multi-label ARAM                | Natively Multilabel           | 0.06 $\pm$ 0.00 | 0.06 $\pm$ 0.00             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.01                           | 0.06 $\pm$ 0.00     |
| it         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.39 $\pm$ 0.01 | 0.40 $\pm$ 0.03             | 0.39 $\pm$ 0.02         | 0.39 $\pm$ 0.02          | 0.38 $\pm$ 0.02                           | 0.40 $\pm$ 0.02     |
| it         | NaiveBayes                      | BorderlineSMOTE               | 0.41 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.49 $\pm$ 0.02         | 0.52 $\pm$ 0.02          | 0.49 $\pm$ 0.02                           | 0.51 $\pm$ 0.03     |
| it         | NaiveBayes                      | No Upsampling                 | 0.40 $\pm$ 0.01 | 0.48 $\pm$ 0.01             | 0.51 $\pm$ 0.02         | 0.58 $\pm$ 0.00          | 0.58 $\pm$ 0.02                           | 0.58 $\pm$ 0.02     |
| it         | NaiveBayes                      | Random Oversampling           | 0.41 $\pm$ 0.01 | 0.46 $\pm$ 0.01             | 0.49 $\pm$ 0.02         | 0.51 $\pm$ 0.01          | 0.48 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| it         | NaiveBayes                      | SMOTE                         | 0.41 $\pm$ 0.01 | 0.45 $\pm$ 0.01             | 0.49 $\pm$ 0.02         | 0.52 $\pm$ 0.02          | 0.48 $\pm$ 0.02                           | 0.51 $\pm$ 0.03     |
| it         | RandomForest                    | BorderlineSMOTE               | 0.50 $\pm$ 0.05 | 0.51 $\pm$ 0.02             | 0.57 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.57 $\pm$ 0.06                           | 0.61 $\pm$ 0.07     |
| it         | RandomForest                    | No Upsampling                 | 0.44 $\pm$ 0.01 | 0.53 $\pm$ 0.03             | 0.53 $\pm$ 0.09         | 0.56 $\pm$ 0.09          | 0.52 $\pm$ 0.02                           | 0.59 $\pm$ 0.03     |
| it         | RandomForest                    | Random Oversampling           | 0.47 $\pm$ 0.03 | 0.55 $\pm$ 0.03             | 0.49 $\pm$ 0.04         | 0.55 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | **0.62 $\pm$ 0.06** |
| it         | RandomForest                    | SMOTE                         | 0.51 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.49 $\pm$ 0.00         | 0.54 $\pm$ 0.07          | 0.60 $\pm$ 0.04                           | 0.59 $\pm$ 0.05     |
| it         | RandomForest                    | SVMSMOTE                      | 0.52 $\pm$ 0.02 | 0.51 $\pm$ 0.03             | 0.48 $\pm$ 0.06         | 0.52 $\pm$ 0.06          | 0.57 $\pm$ 0.07                           | **0.62 $\pm$ 0.02** |
| it         | RidgeClassifier                 | BorderlineSMOTE               | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.03             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | No Upsampling                 | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | Random Oversampling           | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | SMOTE                         | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | SVMSMOTE                      | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.04             | 0.47 $\pm$ 0.01         | 0.56 $\pm$ 0.07          | 0.51 $\pm$ 0.02                           | 0.59 $\pm$ 0.05     |
| it         | SVM                             | BorderlineSMOTE               | 0.44 $\pm$ 0.04 | 0.42 $\pm$ 0.07             | 0.22 $\pm$ 0.06         | 0.25 $\pm$ 0.06          | 0.21 $\pm$ 0.06                           | 0.29 $\pm$ 0.13     |
| it         | SVM                             | No Upsampling                 | 0.28 $\pm$ 0.03 | 0.23 $\pm$ 0.08             | 0.18 $\pm$ 0.07         | 0.23 $\pm$ 0.04          | 0.18 $\pm$ 0.05                           | 0.24 $\pm$ 0.02     |
| it         | SVM                             | RakelD Partitioning of labels | 0.13 $\pm$ 0.05 | 0.12 $\pm$ 0.04             | 0.10 $\pm$ 0.03         | 0.13 $\pm$ 0.04          | 0.14 $\pm$ 0.03                           | 0.14 $\pm$ 0.05     |
| it         | SVM                             | Random Oversampling           | 0.48 $\pm$ 0.04 | 0.35 $\pm$ 0.06             | 0.30 $\pm$ 0.04         | 0.33 $\pm$ 0.05          | 0.31 $\pm$ 0.03                           | 0.31 $\pm$ 0.03     |
| it         | SVM                             | SMOTE                         | 0.45 $\pm$ 0.03 | 0.39 $\pm$ 0.05             | 0.22 $\pm$ 0.06         | 0.24 $\pm$ 0.06          | 0.21 $\pm$ 0.07                           | 0.25 $\pm$ 0.10     |
| it         | SVM                             | SVMSMOTE                      | 0.36 $\pm$ 0.03 | 0.43 $\pm$ 0.07             | 0.32 $\pm$ 0.02         | 0.26 $\pm$ 0.05          | 0.17 $\pm$ 0.03                           | 0.17 $\pm$ 0.04     |
| it         | XGBoost                         | BorderlineSMOTE               | 0.44 $\pm$ 0.03 | 0.44 $\pm$ 0.04             | 0.44 $\pm$ 0.02         | 0.47 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.53 $\pm$ 0.03     |
| it         | XGBoost                         | No Upsampling                 | 0.43 $\pm$ 0.05 | 0.43 $\pm$ 0.03             | 0.43 $\pm$ 0.02         | 0.46 $\pm$ 0.02          | 0.51 $\pm$ 0.05                           | 0.55 $\pm$ 0.03     |
| it         | XGBoost                         | Random Oversampling           | 0.42 $\pm$ 0.03 | 0.42 $\pm$ 0.04             | 0.45 $\pm$ 0.02         | 0.49 $\pm$ 0.04          | 0.48 $\pm$ 0.05                           | 0.55 $\pm$ 0.04     |
| it         | XGBoost                         | SMOTE                         | 0.43 $\pm$ 0.02 | 0.43 $\pm$ 0.04             | 0.46 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.52 $\pm$ 0.05                           | 0.54 $\pm$ 0.04     |
| it         | XGBoost                         | SVMSMOTE                      | 0.43 $\pm$ 0.05 | 0.44 $\pm$ 0.03             | 0.45 $\pm$ 0.03         | 0.49 $\pm$ 0.04          | 0.52 $\pm$ 0.07                           | 0.57 $\pm$ 0.03     |
| po         | Binary Relevance kNN            | Natively Multilabel           | 0.38 $\pm$ 0.07 | 0.34 $\pm$ 0.05             | 0.24 $\pm$ 0.03         | 0.19 $\pm$ 0.04          | 0.15 $\pm$ 0.05                           | 0.15 $\pm$ 0.05     |
| po         | ComplementNB                    | RakelD Partitioning of labels | 0.46 $\pm$ 0.04 | 0.47 $\pm$ 0.04             | 0.53 $\pm$ 0.01         | 0.57 $\pm$ 0.02          | 0.51 $\pm$ 0.05                           | 0.53 $\pm$ 0.04     |
| po         | Dummy Classifier                | No Upsampling                 | 0.43 $\pm$ 0.02 | 0.42 $\pm$ 0.03             | 0.43 $\pm$ 0.03         | 0.41 $\pm$ 0.02          | 0.40 $\pm$ 0.00                           | 0.42 $\pm$ 0.03     |
| po         | KNN                             | BorderlineSMOTE               | 0.39 $\pm$ 0.06 | 0.38 $\pm$ 0.09             | 0.27 $\pm$ 0.02         | 0.27 $\pm$ 0.04          | 0.23 $\pm$ 0.09                           | 0.32 $\pm$ 0.10     |
| po         | KNN                             | No Upsampling                 | 0.46 $\pm$ 0.05 | 0.38 $\pm$ 0.04             | 0.47 $\pm$ 0.06         | 0.46 $\pm$ 0.02          | 0.29 $\pm$ 0.04                           | 0.30 $\pm$ 0.09     |
| po         | KNN                             | Random Oversampling           | 0.48 $\pm$ 0.05 | 0.45 $\pm$ 0.03             | 0.50 $\pm$ 0.04         | 0.54 $\pm$ 0.04          | 0.33 $\pm$ 0.04                           | 0.38 $\pm$ 0.08     |
| po         | KNN                             | SMOTE                         | 0.39 $\pm$ 0.06 | 0.37 $\pm$ 0.11             | 0.27 $\pm$ 0.02         | 0.20 $\pm$ 0.03          | 0.23 $\pm$ 0.05                           | 0.19 $\pm$ 0.03     |
| po         | KNN                             | SVMSMOTE                      | 0.42 $\pm$ 0.03 | 0.38 $\pm$ 0.04             | 0.34 $\pm$ 0.04         | 0.25 $\pm$ 0.06          | 0                                         | 0                   |
| po         | LinearSVM                       | BorderlineSMOTE               | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LinearSVM                       | No Upsampling                 | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LinearSVM                       | RakelD Partitioning of labels | 0.46 $\pm$ 0.05 | 0.52 $\pm$ 0.04             | 0.57 $\pm$ 0.04         | 0.60 $\pm$ 0.03          | 0.55 $\pm$ 0.03                           | 0.59 $\pm$ 0.04     |
| po         | LinearSVM                       | Random Oversampling           | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LinearSVM                       | SMOTE                         | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | BorderlineSMOTE               | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.03             | 0.61 $\pm$ 0.05         | 0.59 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | No Upsampling                 | 0.48 $\pm$ 0.04 | 0.53 $\pm$ 0.04             | 0.57 $\pm$ 0.04         | 0.58 $\pm$ 0.04          | 0.56 $\pm$ 0.04                           | 0.60 $\pm$ 0.04     |
| po         | LogisticRegression              | Random Oversampling           | 0.48 $\pm$ 0.04 | 0.53 $\pm$ 0.03             | 0.58 $\pm$ 0.04         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | SMOTE                         | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.04         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | SVMSMOTE                      | 0.48 $\pm$ 0.04 | 0.52 $\pm$ 0.04             | 0.55 $\pm$ 0.04         | 0.59 $\pm$ 0.03          | 0.58 $\pm$ 0.03                           | 0.63 $\pm$ 0.06     |
| po         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.49 $\pm$ 0.04 | 0.52 $\pm$ 0.03             | 0.55 $\pm$ 0.05         | 0.61 $\pm$ 0.08          | 0.58 $\pm$ 0.03                           | 0.63 $\pm$ 0.05     |
| po         | LogisticRegressionElasticNet    | No Upsampling                 | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.02             | 0.55 $\pm$ 0.05         | 0.60 $\pm$ 0.08          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegressionElasticNet    | Random Oversampling           | 0.49 $\pm$ 0.05 | 0.52 $\pm$ 0.03             | 0.54 $\pm$ 0.05         | 0.60 $\pm$ 0.07          | 0.58 $\pm$ 0.04                           | 0.62 $\pm$ 0.05     |
| po         | LogisticRegressionElasticNet    | SMOTE                         | 0.49 $\pm$ 0.04 | 0.52 $\pm$ 0.03             | 0.55 $\pm$ 0.05         | 0.61 $\pm$ 0.08          | 0.58 $\pm$ 0.03                           | 0.62 $\pm$ 0.04     |
| po         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.50 $\pm$ 0.05 | 0.54 $\pm$ 0.03             | 0.58 $\pm$ 0.05         | 0.61 $\pm$ 0.04          | 0.58 $\pm$ 0.04                           | 0.63 $\pm$ 0.06     |
| po         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.51 $\pm$ 0.04 | 0.51 $\pm$ 0.04             | 0.52 $\pm$ 0.04         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.02                           | 0.59 $\pm$ 0.04     |
| po         | LogisticRegressionLasso         | No Upsampling                 | 0.53 $\pm$ 0.06 | 0.51 $\pm$ 0.03             | 0.53 $\pm$ 0.04         | 0.57 $\pm$ 0.04          | 0.57 $\pm$ 0.02                           | 0.58 $\pm$ 0.03     |
| po         | LogisticRegressionLasso         | Random Oversampling           | 0.50 $\pm$ 0.05 | 0.52 $\pm$ 0.03             | 0.51 $\pm$ 0.03         | 0.57 $\pm$ 0.03          | 0.57 $\pm$ 0.02                           | 0.58 $\pm$ 0.04     |
| po         | LogisticRegressionLasso         | SMOTE                         | 0.51 $\pm$ 0.06 | 0.52 $\pm$ 0.04             | 0.52 $\pm$ 0.03         | 0.57 $\pm$ 0.04          | 0.57 $\pm$ 0.01                           | 0.59 $\pm$ 0.04     |
| po         | LogisticRegressionLasso         | SVMSMOTE                      | 0.52 $\pm$ 0.04 | 0.53 $\pm$ 0.04             | 0.52 $\pm$ 0.06         | 0.56 $\pm$ 0.02          | 0.56 $\pm$ 0.02                           | 0.58 $\pm$ 0.06     |
| po         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.50 $\pm$ 0.04 | 0.55 $\pm$ 0.05             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.63 $\pm$ 0.03                           | 0.64 $\pm$ 0.04     |
| po         | LogisticRegressionRidge         | No Upsampling                 | 0.48 $\pm$ 0.04 | 0.55 $\pm$ 0.04             | 0.60 $\pm$ 0.06         | 0.56 $\pm$ 0.04          | 0.64 $\pm$ 0.04                           | 0.65 $\pm$ 0.04     |
| po         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.49 $\pm$ 0.05 | 0.56 $\pm$ 0.06             | 0.59 $\pm$ 0.02         | 0.56 $\pm$ 0.02          | 0.61 $\pm$ 0.07                           | 0.61 $\pm$ 0.03     |
| po         | LogisticRegressionRidge         | Random Oversampling           | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.05             | 0.60 $\pm$ 0.05         | 0.58 $\pm$ 0.02          | 0.61 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | LogisticRegressionRidge         | SMOTE                         | 0.49 $\pm$ 0.04 | 0.55 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.59 $\pm$ 0.01          | 0.62 $\pm$ 0.02                           | 0.64 $\pm$ 0.03     |
| po         | LogisticRegressionRidge         | SVMSMOTE                      | 0.50 $\pm$ 0.05 | 0.54 $\pm$ 0.02             | 0.59 $\pm$ 0.03         | 0.59 $\pm$ 0.03          | 0.60 $\pm$ 0.05                           | 0.63 $\pm$ 0.03     |
| po         | Multi-label ARAM                | Natively Multilabel           | 0.06 $\pm$ 0.00 | 0.06 $\pm$ 0.00             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.00                           | 0.06 $\pm$ 0.00     |
| po         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.46 $\pm$ 0.03 | 0.47 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.51 $\pm$ 0.03          | 0.53 $\pm$ 0.01                           | 0.55 $\pm$ 0.05     |
| po         | NaiveBayes                      | BorderlineSMOTE               | 0.50 $\pm$ 0.04 | 0.55 $\pm$ 0.03             | 0.57 $\pm$ 0.02         | 0.60 $\pm$ 0.03          | 0.59 $\pm$ 0.04                           | 0.59 $\pm$ 0.04     |
| po         | NaiveBayes                      | No Upsampling                 | 0.49 $\pm$ 0.02 | 0.55 $\pm$ 0.03             | 0.58 $\pm$ 0.04         | 0.64 $\pm$ 0.06          | 0.60 $\pm$ 0.05                           | 0.61 $\pm$ 0.04     |
| po         | NaiveBayes                      | Random Oversampling           | 0.50 $\pm$ 0.03 | 0.54 $\pm$ 0.03             | 0.58 $\pm$ 0.02         | 0.62 $\pm$ 0.04          | 0.59 $\pm$ 0.04                           | 0.59 $\pm$ 0.03     |
| po         | NaiveBayes                      | SMOTE                         | 0.50 $\pm$ 0.04 | 0.54 $\pm$ 0.03             | 0.57 $\pm$ 0.02         | 0.60 $\pm$ 0.04          | 0.59 $\pm$ 0.04                           | 0.59 $\pm$ 0.04     |
| po         | RandomForest                    | BorderlineSMOTE               | 0.57 $\pm$ 0.02 | 0.60 $\pm$ 0.05             | 0.59 $\pm$ 0.07         | 0.57 $\pm$ 0.03          | 0.70 $\pm$ 0.04                           | 0.70 $\pm$ 0.02     |
| po         | RandomForest                    | No Upsampling                 | 0.48 $\pm$ 0.06 | 0.54 $\pm$ 0.08             | 0.56 $\pm$ 0.06         | 0.56 $\pm$ 0.05          | 0.52 $\pm$ 0.02                           | 0.60 $\pm$ 0.01     |
| po         | RandomForest                    | Random Oversampling           | 0.57 $\pm$ 0.03 | 0.60 $\pm$ 0.02             | 0.58 $\pm$ 0.07         | 0.59 $\pm$ 0.01          | 0.65 $\pm$ 0.03                           | 0.70 $\pm$ 0.03     |
| po         | RandomForest                    | SMOTE                         | 0.55 $\pm$ 0.03 | 0.59 $\pm$ 0.04             | 0.55 $\pm$ 0.03         | 0.59 $\pm$ 0.07          | 0.66 $\pm$ 0.01                           | **0.72 $\pm$ 0.04** |
| po         | RandomForest                    | SVMSMOTE                      | 0.58 $\pm$ 0.02 | 0.56 $\pm$ 0.05             | 0.58 $\pm$ 0.08         | 0.59 $\pm$ 0.08          | 0.67 $\pm$ 0.07                           | 0.71 $\pm$ 0.05     |
| po         | RidgeClassifier                 | BorderlineSMOTE               | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | No Upsampling                 | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.05         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | Random Oversampling           | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | SMOTE                         | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | SVMSMOTE                      | 0.49 $\pm$ 0.05 | 0.54 $\pm$ 0.04             | 0.57 $\pm$ 0.03         | 0.59 $\pm$ 0.04          | 0.63 $\pm$ 0.06                           | 0.62 $\pm$ 0.05     |
| po         | SVM                             | BorderlineSMOTE               | 0.52 $\pm$ 0.05 | 0.39 $\pm$ 0.05             | 0.39 $\pm$ 0.06         | 0.30 $\pm$ 0.02          | 0.26 $\pm$ 0.03                           | 0.32 $\pm$ 0.08     |
| po         | SVM                             | No Upsampling                 | 0.35 $\pm$ 0.04 | 0.34 $\pm$ 0.03             | 0.28 $\pm$ 0.04         | 0.30 $\pm$ 0.02          | 0.32 $\pm$ 0.03                           | 0.34 $\pm$ 0.03     |
| po         | SVM                             | RakelD Partitioning of labels | 0.34 $\pm$ 0.10 | 0.32 $\pm$ 0.01             | 0.28 $\pm$ 0.04         | 0.22 $\pm$ 0.07          | 0.25 $\pm$ 0.01                           | 0.26 $\pm$ 0.05     |
| po         | SVM                             | Random Oversampling           | 0.52 $\pm$ 0.05 | 0.42 $\pm$ 0.03             | 0.45 $\pm$ 0.09         | 0.38 $\pm$ 0.04          | 0.36 $\pm$ 0.04                           | 0.39 $\pm$ 0.03     |
| po         | SVM                             | SMOTE                         | 0.50 $\pm$ 0.05 | 0.39 $\pm$ 0.04             | 0.38 $\pm$ 0.05         | 0.31 $\pm$ 0.01          | 0.27 $\pm$ 0.04                           | 0.29 $\pm$ 0.04     |
| po         | SVM                             | SVMSMOTE                      | 0.46 $\pm$ 0.04 | 0.44 $\pm$ 0.06             | 0.35 $\pm$ 0.04         | 0.32 $\pm$ 0.01          | 0.23 $\pm$ 0.01                           | 0.23 $\pm$ 0.01     |
| po         | XGBoost                         | BorderlineSMOTE               | 0.54 $\pm$ 0.03 | 0.52 $\pm$ 0.03             | 0.52 $\pm$ 0.05         | 0.58 $\pm$ 0.05          | 0.59 $\pm$ 0.03                           | 0.66 $\pm$ 0.02     |
| po         | XGBoost                         | No Upsampling                 | 0.52 $\pm$ 0.01 | 0.52 $\pm$ 0.02             | 0.53 $\pm$ 0.05         | 0.56 $\pm$ 0.06          | 0.59 $\pm$ 0.03                           | 0.67 $\pm$ 0.01     |
| po         | XGBoost                         | Random Oversampling           | 0.51 $\pm$ 0.01 | 0.52 $\pm$ 0.03             | 0.53 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.59 $\pm$ 0.02                           | 0.63 $\pm$ 0.02     |
| po         | XGBoost                         | SMOTE                         | 0.54 $\pm$ 0.03 | 0.51 $\pm$ 0.03             | 0.52 $\pm$ 0.04         | 0.56 $\pm$ 0.06          | 0.57 $\pm$ 0.02                           | 0.65 $\pm$ 0.03     |
| po         | XGBoost                         | SVMSMOTE                      | 0.56 $\pm$ 0.03 | 0.51 $\pm$ 0.03             | 0.54 $\pm$ 0.04         | 0.57 $\pm$ 0.04          | 0.59 $\pm$ 0.03                           | 0.66 $\pm$ 0.01     |
| ru         | Binary Relevance kNN            | Natively Multilabel           | 0.06 $\pm$ 0.00 | 0.07 $\pm$ 0.01             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.00                           | 0.06 $\pm$ 0.00     |
| ru         | ComplementNB                    | RakelD Partitioning of labels | 0.30 $\pm$ 0.01 | 0.34 $\pm$ 0.05             | 0.45 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.43 $\pm$ 0.06                           | 0.39 $\pm$ 0.02     |
| ru         | Dummy Classifier                | No Upsampling                 | 0.23 $\pm$ 0.01 | 0.22 $\pm$ 0.02             | 0.19 $\pm$ 0.01         | 0.21 $\pm$ 0.00          | 0.20 $\pm$ 0.02                           | 0.23 $\pm$ 0.01     |
| ru         | KNN                             | BorderlineSMOTE               | 0.22 $\pm$ 0.01 | 0.25 $\pm$ 0.02             | 0.22 $\pm$ 0.00         | 0.22 $\pm$ 0.00          | 0.22 $\pm$ 0.00                           | 0.21 $\pm$ 0.01     |
| ru         | KNN                             | No Upsampling                 | 0.26 $\pm$ 0.04 | 0.12 $\pm$ 0.07             | 0.20 $\pm$ 0.05         | 0.24 $\pm$ 0.03          | 0.11 $\pm$ 0.08                           | 0.20 $\pm$ 0.12     |
| ru         | KNN                             | Random Oversampling           | 0.34 $\pm$ 0.01 | 0.36 $\pm$ 0.09             | 0.32 $\pm$ 0.01         | 0.36 $\pm$ 0.06          | 0.28 $\pm$ 0.01                           | 0.39 $\pm$ 0.05     |
| ru         | KNN                             | SMOTE                         | 0.22 $\pm$ 0.01 | 0.24 $\pm$ 0.01             | 0.22 $\pm$ 0.01         | 0.22 $\pm$ 0.01          | 0.23 $\pm$ 0.01                           | 0.22 $\pm$ 0.01     |
| ru         | LinearSVM                       | BorderlineSMOTE               | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LinearSVM                       | No Upsampling                 | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LinearSVM                       | RakelD Partitioning of labels | 0.34 $\pm$ 0.05 | 0.40 $\pm$ 0.03             | 0.48 $\pm$ 0.07         | 0.59 $\pm$ 0.03          | 0.48 $\pm$ 0.06                           | 0.51 $\pm$ 0.05     |
| ru         | LinearSVM                       | Random Oversampling           | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LinearSVM                       | SMOTE                         | 0.34 $\pm$ 0.02 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LogisticRegression              | BorderlineSMOTE               | 0.35 $\pm$ 0.01 | 0.42 $\pm$ 0.05             | 0.51 $\pm$ 0.05         | 0.60 $\pm$ 0.04          | 0.49 $\pm$ 0.03                           | 0.53 $\pm$ 0.04     |
| ru         | LogisticRegression              | No Upsampling                 | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.04             | 0.49 $\pm$ 0.03         | 0.56 $\pm$ 0.02          | 0.46 $\pm$ 0.02                           | 0.47 $\pm$ 0.01     |
| ru         | LogisticRegression              | Random Oversampling           | 0.35 $\pm$ 0.02 | 0.43 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.52 $\pm$ 0.03                           | 0.53 $\pm$ 0.02     |
| ru         | LogisticRegression              | SMOTE                         | 0.34 $\pm$ 0.03 | 0.43 $\pm$ 0.04             | 0.49 $\pm$ 0.04         | 0.58 $\pm$ 0.02          | 0.49 $\pm$ 0.05                           | 0.52 $\pm$ 0.06     |
| ru         | LogisticRegression              | SVMSMOTE                      | 0               | 0.42 $\pm$ 0.05             | 0                       | 0                        | 0                                         | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.37 $\pm$ 0.02 | 0.42 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.56 $\pm$ 0.02          | 0.48 $\pm$ 0.01                           | 0.56 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | No Upsampling                 | 0.37 $\pm$ 0.03 | 0.46 $\pm$ 0.01             | 0.48 $\pm$ 0.01         | 0.57 $\pm$ 0.01          | 0.53 $\pm$ 0.04                           | 0.58 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | Random Oversampling           | 0.37 $\pm$ 0.02 | 0.43 $\pm$ 0.01             | 0.49 $\pm$ 0.01         | 0.57 $\pm$ 0.02          | 0.47 $\pm$ 0.03                           | 0.52 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | SMOTE                         | 0.37 $\pm$ 0.02 | 0.43 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.58 $\pm$ 0.02          | 0.49 $\pm$ 0.03                           | 0.57 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0               | 0.42 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.53 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.38 $\pm$ 0.04 | 0.43 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.48 $\pm$ 0.02          | 0.51 $\pm$ 0.07                           | 0.48 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | No Upsampling                 | 0.37 $\pm$ 0.06 | 0.43 $\pm$ 0.03             | 0.41 $\pm$ 0.01         | 0.48 $\pm$ 0.08          | 0.49 $\pm$ 0.04                           | 0.45 $\pm$ 0.01     |
| ru         | LogisticRegressionLasso         | Random Oversampling           | 0.34 $\pm$ 0.04 | 0.41 $\pm$ 0.02             | 0.41 $\pm$ 0.04         | 0.47 $\pm$ 0.02          | 0.50 $\pm$ 0.10                           | 0.46 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | SMOTE                         | 0.39 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.40 $\pm$ 0.02         | 0.45 $\pm$ 0.03          | 0.48 $\pm$ 0.07                           | 0.46 $\pm$ 0.01     |
| ru         | LogisticRegressionLasso         | SVMSMOTE                      | 0               | 0.40 $\pm$ 0.02             | 0                       | 0                        | 0                                         | 0.47 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.38 $\pm$ 0.02 | 0.44 $\pm$ 0.00             | 0.56 $\pm$ 0.04         | 0.57 $\pm$ 0.03          | 0.56 $\pm$ 0.02                           | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionRidge         | No Upsampling                 | 0.37 $\pm$ 0.03 | 0.43 $\pm$ 0.02             | 0.56 $\pm$ 0.04         | 0.54 $\pm$ 0.07          | 0.52 $\pm$ 0.05                           | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.33 $\pm$ 0.03 | 0.43 $\pm$ 0.03             | 0.49 $\pm$ 0.01         | 0.46 $\pm$ 0.03          | 0.51 $\pm$ 0.09                           | 0.53 $\pm$ 0.06     |
| ru         | LogisticRegressionRidge         | Random Oversampling           | 0.38 $\pm$ 0.02 | 0.44 $\pm$ 0.02             | 0.53 $\pm$ 0.03         | 0.56 $\pm$ 0.04          | 0.54 $\pm$ 0.01                           | 0.56 $\pm$ 0.07     |
| ru         | LogisticRegressionRidge         | SMOTE                         | 0.39 $\pm$ 0.03 | 0.45 $\pm$ 0.02             | 0.53 $\pm$ 0.03         | 0.55 $\pm$ 0.04          | 0.55 $\pm$ 0.02                           | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionRidge         | SVMSMOTE                      | 0               | 0.41 $\pm$ 0.03             | 0                       | 0                        | 0                                         | 0.50 $\pm$ 0.02     |
| ru         | Multi-label ARAM                | Natively Multilabel           | 0.06 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.00         | 0.05 $\pm$ 0.00          | 0.06 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ru         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.31 $\pm$ 0.06 | 0.32 $\pm$ 0.02             | 0.35 $\pm$ 0.04         | 0.34 $\pm$ 0.05          | 0.36 $\pm$ 0.08                           | 0.39 $\pm$ 0.04     |
| ru         | NaiveBayes                      | BorderlineSMOTE               | 0.31 $\pm$ 0.01 | 0.36 $\pm$ 0.02             | 0.40 $\pm$ 0.01         | 0.42 $\pm$ 0.03          | 0.42 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| ru         | NaiveBayes                      | No Upsampling                 | 0.27 $\pm$ 0.00 | 0.36 $\pm$ 0.01             | 0.44 $\pm$ 0.04         | 0.60 $\pm$ 0.04          | 0.49 $\pm$ 0.04                           | 0.53 $\pm$ 0.01     |
| ru         | NaiveBayes                      | Random Oversampling           | 0.31 $\pm$ 0.01 | 0.37 $\pm$ 0.03             | 0.39 $\pm$ 0.01         | 0.42 $\pm$ 0.03          | 0.43 $\pm$ 0.02                           | 0.43 $\pm$ 0.03     |
| ru         | NaiveBayes                      | SMOTE                         | 0.31 $\pm$ 0.00 | 0.36 $\pm$ 0.01             | 0.38 $\pm$ 0.02         | 0.44 $\pm$ 0.03          | 0.42 $\pm$ 0.02                           | 0.45 $\pm$ 0.02     |
| ru         | RandomForest                    | BorderlineSMOTE               | 0.49 $\pm$ 0.05 | 0.51 $\pm$ 0.01             | 0.57 $\pm$ 0.03         | 0.55 $\pm$ 0.12          | 0.53 $\pm$ 0.05                           | 0.50 $\pm$ 0.08     |
| ru         | RandomForest                    | No Upsampling                 | 0.45 $\pm$ 0.09 | 0.41 $\pm$ 0.10             | 0.35 $\pm$ 0.02         | 0.40 $\pm$ 0.06          | 0.38 $\pm$ 0.10                           | 0.45 $\pm$ 0.10     |
| ru         | RandomForest                    | Random Oversampling           | 0.44 $\pm$ 0.01 | 0.54 $\pm$ 0.05             | 0.43 $\pm$ 0.02         | 0.56 $\pm$ 0.03          | 0.50 $\pm$ 0.02                           | 0.57 $\pm$ 0.04     |
| ru         | RandomForest                    | SMOTE                         | 0.55 $\pm$ 0.07 | 0.53 $\pm$ 0.06             | 0.46 $\pm$ 0.05         | 0.53 $\pm$ 0.04          | 0.49 $\pm$ 0.04                           | 0.52 $\pm$ 0.05     |
| ru         | RandomForest                    | SVMSMOTE                      | 0               | 0.49 $\pm$ 0.04             | 0                       | 0                        | 0                                         | **0.61 $\pm$ 0.07** |
| ru         | RidgeClassifier                 | BorderlineSMOTE               | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | No Upsampling                 | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | Random Oversampling           | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | SMOTE                         | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | SVMSMOTE                      | 0               | 0.43 $\pm$ 0.05             | 0                       | 0                        | 0                                         | 0.53 $\pm$ 0.05     |
| ru         | SVM                             | BorderlineSMOTE               | 0.34 $\pm$ 0.08 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.10 $\pm$ 0.07          | 0.05 $\pm$ 0.07                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | No Upsampling                 | 0.10 $\pm$ 0.03 | 0.05 $\pm$ 0.07             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.00     |
| ru         | SVM                             | RakelD Partitioning of labels | 0.16 $\pm$ 0.04 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.05 $\pm$ 0.07          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.02     |
| ru         | SVM                             | Random Oversampling           | 0.43 $\pm$ 0.08 | 0.17 $\pm$ 0.05             | 0.22 $\pm$ 0.12         | 0.28 $\pm$ 0.20          | 0.21 $\pm$ 0.14                           | 0.15 $\pm$ 0.07     |
| ru         | SVM                             | SMOTE                         | 0.33 $\pm$ 0.10 | 0.05 $\pm$ 0.07             | 0.05 $\pm$ 0.07         | 0.10 $\pm$ 0.07          | 0.05 $\pm$ 0.07                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | SVMSMOTE                      | 0               | 0.17 $\pm$ 0.03             | 0                       | 0                        | 0                                         | 0.00 $\pm$ 0.00     |
| ru         | XGBoost                         | BorderlineSMOTE               | 0.33 $\pm$ 0.06 | 0.37 $\pm$ 0.05             | 0.40 $\pm$ 0.02         | 0.46 $\pm$ 0.04          | 0.42 $\pm$ 0.02                           | 0.48 $\pm$ 0.01     |
| ru         | XGBoost                         | No Upsampling                 | 0.32 $\pm$ 0.03 | 0.35 $\pm$ 0.06             | 0.38 $\pm$ 0.03         | 0.50 $\pm$ 0.08          | 0.45 $\pm$ 0.01                           | 0.52 $\pm$ 0.05     |
| ru         | XGBoost                         | Random Oversampling           | 0.36 $\pm$ 0.05 | 0.35 $\pm$ 0.03             | 0.40 $\pm$ 0.07         | 0.48 $\pm$ 0.06          | 0.41 $\pm$ 0.07                           | 0.51 $\pm$ 0.03     |
| ru         | XGBoost                         | SMOTE                         | 0.36 $\pm$ 0.03 | 0.39 $\pm$ 0.07             | 0.40 $\pm$ 0.03         | 0.44 $\pm$ 0.05          | 0.42 $\pm$ 0.01                           | 0.49 $\pm$ 0.03     |
| ru         | XGBoost                         | SVMSMOTE                      | 0               | 0.38 $\pm$ 0.07             | 0                       | 0                        | 0                                         | 0.47 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_type                      | model_subtype                 | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:--------------------------------|:------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | Binary Relevance kNN            | Natively Multilabel           | 0.02 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| en         | ComplementNB                    | RakelD Partitioning of labels | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.08 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.09 $\pm$ 0.01     |
| en         | Dummy Classifier                | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| en         | KNN                             | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.00     |
| en         | KNN                             | SMOTE                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | SVMSMOTE                      | 0               | 0.00 $\pm$ 0.00             | 0                       | 0                        | 0.00 $\pm$ 0.00                           | 0                   |
| en         | LinearSVM                       | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | No Upsampling                 | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | RakelD Partitioning of labels | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.00          | 0.08 $\pm$ 0.02                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | Random Oversampling           | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | SMOTE                         | 0.02 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | BorderlineSMOTE               | 0.03 $\pm$ 0.00 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | No Upsampling                 | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegression              | Random Oversampling           | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.00             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | SMOTE                         | 0.03 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | SVMSMOTE                      | 0.03 $\pm$ 0.00 | 0.05 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.08 $\pm$ 0.01                           | 0.10 $\pm$ 0.03     |
| en         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | No Upsampling                 | 0.05 $\pm$ 0.01 | 0.08 $\pm$ 0.00             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.08 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | Random Oversampling           | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.02             | 0.07 $\pm$ 0.00         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.00     |
| en         | LogisticRegressionElasticNet    | SMOTE                         | 0.05 $\pm$ 0.00 | 0.07 $\pm$ 0.00             | 0.08 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.07 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | No Upsampling                 | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.05 $\pm$ 0.02     |
| en         | LogisticRegressionLasso         | Random Oversampling           | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | SMOTE                         | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | SVMSMOTE                      | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| en         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.10 $\pm$ 0.01         | 0.09 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | No Upsampling                 | 0.06 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.01         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.06 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegressionRidge         | Random Oversampling           | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.09 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SMOTE                         | 0.05 $\pm$ 0.02 | 0.07 $\pm$ 0.01             | 0.10 $\pm$ 0.01         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SVMSMOTE                      | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.10 $\pm$ 0.02                           | 0.11 $\pm$ 0.02     |
| en         | Multi-label ARAM                | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| en         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.03 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| en         | NaiveBayes                      | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| en         | NaiveBayes                      | No Upsampling                 | 0.03 $\pm$ 0.01 | 0.09 $\pm$ 0.01             | **0.13 $\pm$ 0.02**     | 0.12 $\pm$ 0.02          | 0.12 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| en         | NaiveBayes                      | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.04 $\pm$ 0.00         | 0.02 $\pm$ 0.00          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | NaiveBayes                      | SMOTE                         | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.00         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| en         | RandomForest                    | BorderlineSMOTE               | 0.07 $\pm$ 0.01 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.11 $\pm$ 0.01                           | 0.12 $\pm$ 0.01     |
| en         | RandomForest                    | No Upsampling                 | 0.07 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.11 $\pm$ 0.03         | 0.10 $\pm$ 0.03          | 0.12 $\pm$ 0.02                           | 0.12 $\pm$ 0.01     |
| en         | RandomForest                    | Random Oversampling           | 0.07 $\pm$ 0.02 | 0.09 $\pm$ 0.01             | 0.11 $\pm$ 0.02         | 0.11 $\pm$ 0.00          | 0.11 $\pm$ 0.01                           | 0.11 $\pm$ 0.02     |
| en         | RandomForest                    | SMOTE                         | 0.06 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.11 $\pm$ 0.00                           | 0.12 $\pm$ 0.01     |
| en         | RandomForest                    | SVMSMOTE                      | 0.08 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.12 $\pm$ 0.03          | 0.11 $\pm$ 0.02                           | 0.12 $\pm$ 0.01     |
| en         | RidgeClassifier                 | BorderlineSMOTE               | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | No Upsampling                 | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | Random Oversampling           | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SMOTE                         | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.00          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.03     |
| en         | SVM                             | BorderlineSMOTE               | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | SVM                             | No Upsampling                 | 0.03 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | SVM                             | RakelD Partitioning of labels | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| en         | SVM                             | Random Oversampling           | 0.06 $\pm$ 0.02 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.03 $\pm$ 0.02     |
| en         | SVM                             | SMOTE                         | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | SVM                             | SVMSMOTE                      | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| en         | XGBoost                         | BorderlineSMOTE               | 0.04 $\pm$ 0.00 | 0.04 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| en         | XGBoost                         | No Upsampling                 | 0.06 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| en         | XGBoost                         | Random Oversampling           | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.07 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| en         | XGBoost                         | SMOTE                         | 0.04 $\pm$ 0.00 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.02     |
| en         | XGBoost                         | SVMSMOTE                      | 0.04 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.01          | 0.08 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| fr         | Binary Relevance kNN            | Natively Multilabel           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | ComplementNB                    | RakelD Partitioning of labels | 0.00 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| fr         | Dummy Classifier                | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | KNN                             | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | KNN                             | No Upsampling                 | 0.04 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| fr         | KNN                             | Random Oversampling           | 0.03 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.01     |
| fr         | KNN                             | SMOTE                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | LinearSVM                       | BorderlineSMOTE               | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LinearSVM                       | No Upsampling                 | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LinearSVM                       | RakelD Partitioning of labels | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.07 $\pm$ 0.03     |
| fr         | LinearSVM                       | Random Oversampling           | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LinearSVM                       | SMOTE                         | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LogisticRegression              | BorderlineSMOTE               | 0.02 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LogisticRegression              | No Upsampling                 | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| fr         | LogisticRegression              | Random Oversampling           | 0.02 $\pm$ 0.02 | 0.07 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.03 $\pm$ 0.00                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegression              | SMOTE                         | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.06 $\pm$ 0.03     |
| fr         | LogisticRegression              | SVMSMOTE                      | 0.02 $\pm$ 0.01 | 0.06 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.08 $\pm$ 0.03     |
| fr         | LogisticRegressionElasticNet    | No Upsampling                 | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.02             | 0.03 $\pm$ 0.00         | 0.04 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| fr         | LogisticRegressionElasticNet    | Random Oversampling           | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.04 $\pm$ 0.01         | 0.01 $\pm$ 0.00          | 0.04 $\pm$ 0.04                           | 0.09 $\pm$ 0.02     |
| fr         | LogisticRegressionElasticNet    | SMOTE                         | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| fr         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.05 $\pm$ 0.02                           | 0.09 $\pm$ 0.03     |
| fr         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | Random Oversampling           | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | SMOTE                         | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | SVMSMOTE                      | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.02 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.04 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| fr         | LogisticRegressionRidge         | Random Oversampling           | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | SMOTE                         | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | SVMSMOTE                      | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | Multi-label ARAM                | Natively Multilabel           | 0.04 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.03 $\pm$ 0.02     |
| fr         | NaiveBayes                      | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| fr         | NaiveBayes                      | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | **0.10 $\pm$ 0.02**                       | 0.06 $\pm$ 0.01     |
| fr         | NaiveBayes                      | Random Oversampling           | 0.00 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | NaiveBayes                      | SMOTE                         | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| fr         | RandomForest                    | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.00     |
| fr         | RandomForest                    | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.02                           | 0.05 $\pm$ 0.02     |
| fr         | RandomForest                    | Random Oversampling           | 0.04 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| fr         | RandomForest                    | SMOTE                         | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| fr         | RandomForest                    | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.02             | 0.04 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.03     |
| fr         | RidgeClassifier                 | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | Random Oversampling           | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | SMOTE                         | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | SVM                             | BorderlineSMOTE               | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| fr         | SVM                             | RakelD Partitioning of labels | 0.00 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.01 $\pm$ 0.02         | 0.01 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| fr         | SVM                             | Random Oversampling           | 0.02 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | SVM                             | SMOTE                         | 0.04 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | SVMSMOTE                      | 0.00 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.01          | 0.00 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| fr         | XGBoost                         | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.01     |
| fr         | XGBoost                         | No Upsampling                 | 0.03 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.05 $\pm$ 0.02                           | 0.06 $\pm$ 0.01     |
| fr         | XGBoost                         | Random Oversampling           | 0.03 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| fr         | XGBoost                         | SMOTE                         | 0.03 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| fr         | XGBoost                         | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.01 $\pm$ 0.00          | 0.05 $\pm$ 0.03                           | 0.06 $\pm$ 0.01     |
| ge         | Binary Relevance kNN            | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | ComplementNB                    | RakelD Partitioning of labels | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | Dummy Classifier                | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | No Upsampling                 | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | SMOTE                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0                        | 0.00 $\pm$ 0.00                           | 0                   |
| ge         | LinearSVM                       | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LinearSVM                       | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LinearSVM                       | RakelD Partitioning of labels | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LinearSVM                       | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LinearSVM                       | SMOTE                         | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegression              | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegression              | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegression              | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegression              | SMOTE                         | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegression              | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | No Upsampling                 | 0.01 $\pm$ 0.02 | 0.03 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | Random Oversampling           | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | SMOTE                         | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| ge         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionLasso         | No Upsampling                 | 0.02 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | Random Oversampling           | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | SMOTE                         | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionLasso         | SVMSMOTE                      | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | SMOTE                         | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | Multi-label ARAM                | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ge         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.03                           | 0.01 $\pm$ 0.01     |
| ge         | NaiveBayes                      | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | NaiveBayes                      | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.05 $\pm$ 0.02     |
| ge         | NaiveBayes                      | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ge         | NaiveBayes                      | SMOTE                         | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.02     |
| ge         | RandomForest                    | BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.03 $\pm$ 0.03         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.03                           | 0.02 $\pm$ 0.01     |
| ge         | RandomForest                    | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.03 $\pm$ 0.04                           | 0.03 $\pm$ 0.01     |
| ge         | RandomForest                    | Random Oversampling           | 0.01 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | **0.06 $\pm$ 0.02**      | 0.03 $\pm$ 0.04                           | 0.04 $\pm$ 0.03     |
| ge         | RandomForest                    | SMOTE                         | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.04                           | 0.02 $\pm$ 0.01     |
| ge         | RandomForest                    | SVMSMOTE                      | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.02 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | SMOTE                         | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | SVM                             | BorderlineSMOTE               | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.02                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | No Upsampling                 | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | RakelD Partitioning of labels | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.02     |
| ge         | SVM                             | Random Oversampling           | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | SMOTE                         | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ge         | XGBoost                         | BorderlineSMOTE               | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | XGBoost                         | No Upsampling                 | 0.02 $\pm$ 0.03 | 0.05 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.02 $\pm$ 0.00     |
| ge         | XGBoost                         | Random Oversampling           | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ge         | XGBoost                         | SMOTE                         | 0.04 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| ge         | XGBoost                         | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.04 $\pm$ 0.04                           | 0.02 $\pm$ 0.01     |
| it         | Binary Relevance kNN            | Natively Multilabel           | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | ComplementNB                    | RakelD Partitioning of labels | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| it         | Dummy Classifier                | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | No Upsampling                 | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.01 $\pm$ 0.01     |
| it         | KNN                             | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| it         | KNN                             | SMOTE                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0                           | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0                                         | 0.00 $\pm$ 0.00     |
| it         | LinearSVM                       | BorderlineSMOTE               | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | No Upsampling                 | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | RakelD Partitioning of labels | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.03     |
| it         | LinearSVM                       | Random Oversampling           | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | SMOTE                         | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegression              | BorderlineSMOTE               | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.02                           | 0.06 $\pm$ 0.03     |
| it         | LogisticRegression              | No Upsampling                 | 0.03 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.03     |
| it         | LogisticRegression              | Random Oversampling           | 0.03 $\pm$ 0.01 | 0.02 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.03          | 0.05 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| it         | LogisticRegression              | SMOTE                         | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.06 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegression              | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.05 $\pm$ 0.03          | 0.04 $\pm$ 0.03                           | 0.07 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.05 $\pm$ 0.03                           | 0.06 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | No Upsampling                 | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.06 $\pm$ 0.03                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | Random Oversampling           | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.03                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | SMOTE                         | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.02                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.04 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | No Upsampling                 | 0.05 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | Random Oversampling           | 0.05 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | SMOTE                         | 0.05 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | SVMSMOTE                      | 0.04 $\pm$ 0.03 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.03 $\pm$ 0.00     |
| it         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.05 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.05 $\pm$ 0.02                           | 0.08 $\pm$ 0.01     |
| it         | LogisticRegressionRidge         | No Upsampling                 | 0.05 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.06 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.07 $\pm$ 0.03         | 0.08 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | **0.09 $\pm$ 0.02** |
| it         | LogisticRegressionRidge         | Random Oversampling           | 0.05 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegressionRidge         | SMOTE                         | 0.05 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | LogisticRegressionRidge         | SVMSMOTE                      | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | Multi-label ARAM                | Natively Multilabel           | 0.06 $\pm$ 0.02 | 0.06 $\pm$ 0.02             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| it         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| it         | NaiveBayes                      | BorderlineSMOTE               | 0.01 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.06 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| it         | NaiveBayes                      | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.07 $\pm$ 0.01         | **0.09 $\pm$ 0.01**      | 0.08 $\pm$ 0.01                           | **0.09 $\pm$ 0.01** |
| it         | NaiveBayes                      | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.00     |
| it         | NaiveBayes                      | SMOTE                         | 0.01 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.00          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| it         | RandomForest                    | BorderlineSMOTE               | 0.04 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.07 $\pm$ 0.02         | **0.09 $\pm$ 0.01**      | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| it         | RandomForest                    | No Upsampling                 | 0.04 $\pm$ 0.02 | 0.05 $\pm$ 0.02             | 0.07 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| it         | RandomForest                    | Random Oversampling           | 0.03 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | **0.09 $\pm$ 0.02**      | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.02     |
| it         | RandomForest                    | SMOTE                         | 0.04 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.06 $\pm$ 0.00         | 0.07 $\pm$ 0.01          | 0.05 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | RandomForest                    | SVMSMOTE                      | 0.04 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.07 $\pm$ 0.01         | **0.09 $\pm$ 0.02**      | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | RidgeClassifier                 | BorderlineSMOTE               | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | No Upsampling                 | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | Random Oversampling           | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | SMOTE                         | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | SVM                             | BorderlineSMOTE               | 0.03 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| it         | SVM                             | No Upsampling                 | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | SVM                             | RakelD Partitioning of labels | 0.04 $\pm$ 0.04 | 0.03 $\pm$ 0.04             | 0.04 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| it         | SVM                             | Random Oversampling           | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.00     |
| it         | SVM                             | SMOTE                         | 0.04 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| it         | SVM                             | SVMSMOTE                      | 0.02 $\pm$ 0.00 | 0.01 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| it         | XGBoost                         | BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.06 $\pm$ 0.03          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| it         | XGBoost                         | No Upsampling                 | 0.03 $\pm$ 0.00 | 0.01 $\pm$ 0.00             | 0.04 $\pm$ 0.00         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.00                           | 0.05 $\pm$ 0.01     |
| it         | XGBoost                         | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.03 $\pm$ 0.00         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| it         | XGBoost                         | SMOTE                         | 0.03 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| it         | XGBoost                         | SVMSMOTE                      | 0.03 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.06 $\pm$ 0.02          | 0.02 $\pm$ 0.00                           | 0.06 $\pm$ 0.01     |
| po         | Binary Relevance kNN            | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | ComplementNB                    | RakelD Partitioning of labels | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.05 $\pm$ 0.00          | 0.02 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| po         | Dummy Classifier                | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | KNN                             | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| po         | KNN                             | SMOTE                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | SVMSMOTE                      | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0                                         | 0                   |
| po         | LinearSVM                       | BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | No Upsampling                 | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | RakelD Partitioning of labels | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | Random Oversampling           | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | SMOTE                         | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegression              | BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegression              | No Upsampling                 | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegression              | Random Oversampling           | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegression              | SMOTE                         | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegression              | SVMSMOTE                      | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.00          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.00         | 0.04 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.05 $\pm$ 0.00     |
| po         | LogisticRegressionElasticNet    | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.05 $\pm$ 0.00                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | SMOTE                         | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.00     |
| po         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| po         | LogisticRegressionLasso         | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | SMOTE                         | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | SVMSMOTE                      | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.01     |
| po         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | No Upsampling                 | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.03 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | Random Oversampling           | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | SMOTE                         | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | SVMSMOTE                      | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | Multi-label ARAM                | Natively Multilabel           | 0.05 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.04 $\pm$ 0.02     |
| po         | NaiveBayes                      | BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| po         | NaiveBayes                      | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | 0.07 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| po         | NaiveBayes                      | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | NaiveBayes                      | SMOTE                         | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.03 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| po         | RandomForest                    | BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| po         | RandomForest                    | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.05 $\pm$ 0.01     |
| po         | RandomForest                    | Random Oversampling           | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.03             | 0.03 $\pm$ 0.03         | 0.04 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | **0.09 $\pm$ 0.02** |
| po         | RandomForest                    | SMOTE                         | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.03          | 0.04 $\pm$ 0.03                           | 0.07 $\pm$ 0.02     |
| po         | RandomForest                    | SVMSMOTE                      | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | No Upsampling                 | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | Random Oversampling           | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | SMOTE                         | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | SVMSMOTE                      | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | SVM                             | BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | RakelD Partitioning of labels | 0.02 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| po         | SVM                             | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | SMOTE                         | 0.02 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | SVMSMOTE                      | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | XGBoost                         | BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.03          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.03     |
| po         | XGBoost                         | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.03 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.05 $\pm$ 0.02     |
| po         | XGBoost                         | Random Oversampling           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| po         | XGBoost                         | SMOTE                         | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| po         | XGBoost                         | SVMSMOTE                      | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ru         | Binary Relevance kNN            | Natively Multilabel           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| ru         | ComplementNB                    | RakelD Partitioning of labels | 0.01 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| ru         | Dummy Classifier                | No Upsampling                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | BorderlineSMOTE               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | No Upsampling                 | 0.06 $\pm$ 0.04 | 0.04 $\pm$ 0.02             | 0.05 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.03 $\pm$ 0.03     |
| ru         | KNN                             | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | SMOTE                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | LinearSVM                       | BorderlineSMOTE               | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LinearSVM                       | No Upsampling                 | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LinearSVM                       | RakelD Partitioning of labels | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.09 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| ru         | LinearSVM                       | Random Oversampling           | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LinearSVM                       | SMOTE                         | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LogisticRegression              | BorderlineSMOTE               | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.02                           | 0.10 $\pm$ 0.01     |
| ru         | LogisticRegression              | No Upsampling                 | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.00         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.04                           | 0.10 $\pm$ 0.02     |
| ru         | LogisticRegression              | Random Oversampling           | 0.04 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.03          | 0.08 $\pm$ 0.03                           | 0.10 $\pm$ 0.01     |
| ru         | LogisticRegression              | SMOTE                         | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.10 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.03                           | 0.11 $\pm$ 0.01     |
| ru         | LogisticRegression              | SVMSMOTE                      | 0               | 0.08 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.12 $\pm$ 0.01     |
| ru         | LogisticRegressionElasticNet    | BorderlineSMOTE               | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.03          | 0.09 $\pm$ 0.03                           | 0.12 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | No Upsampling                 | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.10 $\pm$ 0.03                           | 0.11 $\pm$ 0.01     |
| ru         | LogisticRegressionElasticNet    | Random Oversampling           | 0.03 $\pm$ 0.00 | 0.04 $\pm$ 0.01             | 0.07 $\pm$ 0.01         | 0.09 $\pm$ 0.04          | 0.09 $\pm$ 0.04                           | 0.11 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | SMOTE                         | 0.03 $\pm$ 0.00 | 0.05 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.03          | 0.10 $\pm$ 0.03                           | **0.13 $\pm$ 0.02** |
| ru         | LogisticRegressionElasticNet    | SVMSMOTE                      | 0               | 0.05 $\pm$ 0.00             | 0                       | 0                        | 0                                         | **0.13 $\pm$ 0.02** |
| ru         | LogisticRegressionLasso         | BorderlineSMOTE               | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.08 $\pm$ 0.03                           | 0.08 $\pm$ 0.03     |
| ru         | LogisticRegressionLasso         | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.03         | 0.06 $\pm$ 0.03          | 0.08 $\pm$ 0.02                           | 0.05 $\pm$ 0.03     |
| ru         | LogisticRegressionLasso         | Random Oversampling           | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.04          | 0.07 $\pm$ 0.03                           | 0.06 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | SMOTE                         | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.07 $\pm$ 0.03                           | 0.06 $\pm$ 0.04     |
| ru         | LogisticRegressionLasso         | SVMSMOTE                      | 0               | 0.05 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.07 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | BorderlineSMOTE               | 0.03 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | No Upsampling                 | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | RakelD Partitioning of labels | 0.04 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.08 $\pm$ 0.00         | 0.09 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.11 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | Random Oversampling           | 0.04 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | SMOTE                         | 0.04 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | SVMSMOTE                      | 0               | 0.07 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.10 $\pm$ 0.01     |
| ru         | Multi-label ARAM                | Natively Multilabel           | 0.07 $\pm$ 0.01 | 0.08 $\pm$ 0.02             | 0.08 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| ru         | Multilabel k Nearest Neighbours | Natively Multilabel           | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| ru         | NaiveBayes                      | BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.01     |
| ru         | NaiveBayes                      | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.06 $\pm$ 0.03             | 0.08 $\pm$ 0.04         | **0.13 $\pm$ 0.03**      | 0.10 $\pm$ 0.02                           | 0.10 $\pm$ 0.03     |
| ru         | NaiveBayes                      | Random Oversampling           | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.04 $\pm$ 0.03         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ru         | NaiveBayes                      | SMOTE                         | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.03             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.03          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| ru         | RandomForest                    | BorderlineSMOTE               | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.07 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| ru         | RandomForest                    | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.04 $\pm$ 0.03     |
| ru         | RandomForest                    | Random Oversampling           | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.07 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| ru         | RandomForest                    | SMOTE                         | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.05 $\pm$ 0.03         | 0.06 $\pm$ 0.02          | 0.07 $\pm$ 0.03                           | 0.07 $\pm$ 0.02     |
| ru         | RandomForest                    | SVMSMOTE                      | 0               | 0.03 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.06 $\pm$ 0.04     |
| ru         | RidgeClassifier                 | BorderlineSMOTE               | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | No Upsampling                 | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | Random Oversampling           | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | SMOTE                         | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | SVMSMOTE                      | 0               | 0.07 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.11 $\pm$ 0.00     |
| ru         | SVM                             | BorderlineSMOTE               | 0.04 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | No Upsampling                 | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.00     |
| ru         | SVM                             | RakelD Partitioning of labels | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ru         | SVM                             | Random Oversampling           | 0.06 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ru         | SVM                             | SMOTE                         | 0.02 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | SVMSMOTE                      | 0               | 0.01 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.00 $\pm$ 0.00     |
| ru         | XGBoost                         | BorderlineSMOTE               | 0.02 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.06 $\pm$ 0.03                           | 0.12 $\pm$ 0.02     |
| ru         | XGBoost                         | No Upsampling                 | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.03                           | 0.12 $\pm$ 0.04     |
| ru         | XGBoost                         | Random Oversampling           | 0.04 $\pm$ 0.03 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | **0.13 $\pm$ 0.05** |
| ru         | XGBoost                         | SMOTE                         | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.07 $\pm$ 0.04                           | 0.11 $\pm$ 0.03     |
| ru         | XGBoost                         | SVMSMOTE                      | 0               | 0.03 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.12 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


# Per model sub-type and exp name

In [13]:
display_metrics_and_write_to_file(df=exps_df, grouping_criterion=['model_type','model_subtype', 'model_name'], output_dir='per_model_name_tables')

# English

## f1_micro

|                                                                                                     | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:----------------------------------------------------------------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| ('Binary Relevance kNN', 'Natively Multilabel', 'BRkNNaClassifier')                                 | 0.25 $\pm$ 0.05 | 0.31 $\pm$ 0.04             | 0.28 $\pm$ 0.09         | 0.28 $\pm$ 0.02          | 0.33 $\pm$ 0.05                           | 0.33 $\pm$ 0.04     |
| ('Binary Relevance kNN', 'Natively Multilabel', 'BRkNNbClassifier')                                 | 0.20 $\pm$ 0.01 | 0.22 $\pm$ 0.00             | 0.20 $\pm$ 0.01         | 0.19 $\pm$ 0.01          | 0.22 $\pm$ 0.02                           | 0.20 $\pm$ 0.00     |
| ('ComplementNB', 'RakelD Partitioning of labels', 'RakelD_ComplementNB')                            | 0.55 $\pm$ 0.00 | 0.62 $\pm$ 0.02             | 0.63 $\pm$ 0.00         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.65 $\pm$ 0.01     |
| ('Dummy Classifier', 'No Upsampling', 'DummyMostFrequent')                                          | 0.29 $\pm$ 0.00 | 0.29 $\pm$ 0.00             | 0.29 $\pm$ 0.00         | 0.29 $\pm$ 0.00          | 0.29 $\pm$ 0.00                           | 0.29 $\pm$ 0.00     |
| ('Dummy Classifier', 'No Upsampling', 'DummyProbSampling')                                          | 0.40 $\pm$ 0.01 | 0.38 $\pm$ 0.01             | 0.39 $\pm$ 0.00         | 0.40 $\pm$ 0.01          | 0.40 $\pm$ 0.02                           | 0.38 $\pm$ 0.01     |
| ('Dummy Classifier', 'No Upsampling', 'DummyUniformSampling')                                       | 0.42 $\pm$ 0.02 | 0.43 $\pm$ 0.01             | 0.41 $\pm$ 0.01         | 0.43 $\pm$ 0.01          | 0.43 $\pm$ 0.01                           | 0.42 $\pm$ 0.01     |
| ('KNN', 'BorderlineSMOTE', 'kNN_BorderlineSMOTE')                                                   | 0.41 $\pm$ 0.02 | 0.33 $\pm$ 0.06             | 0.28 $\pm$ 0.01         | 0.29 $\pm$ 0.00          | 0.31 $\pm$ 0.04                           | 0.33 $\pm$ 0.03     |
| ('KNN', 'No Upsampling', 'kNN')                                                                     | 0.34 $\pm$ 0.06 | 0.40 $\pm$ 0.05             | 0.38 $\pm$ 0.06         | 0.35 $\pm$ 0.04          | 0.41 $\pm$ 0.06                           | 0.50 $\pm$ 0.06     |
| ('KNN', 'Random Oversampling', 'kNN_ROS')                                                           | 0.38 $\pm$ 0.06 | 0.47 $\pm$ 0.06             | 0.41 $\pm$ 0.07         | 0.43 $\pm$ 0.04          | 0.40 $\pm$ 0.07                           | 0.52 $\pm$ 0.06     |
| ('KNN', 'SMOTE', 'kNN_SMOTE')                                                                       | 0.41 $\pm$ 0.03 | 0.34 $\pm$ 0.04             | 0.30 $\pm$ 0.01         | 0.30 $\pm$ 0.02          | 0.31 $\pm$ 0.04                           | 0.33 $\pm$ 0.04     |
| ('KNN', 'SVMSMOTE', 'kNN_SVMSMOTE')                                                                 | 0               | 0.38 $\pm$ 0.03             | 0                       | 0                        | 0.34 $\pm$ 0.03                           | 0                   |
| ('LinearSVM', 'BorderlineSMOTE', 'LinearSVM_BorderlineSMOTE')                                       | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LinearSVM', 'BorderlineSMOTE', 'LinearSVM_SVMSMOTE')                                              | 0               | 0.61 $\pm$ 0.01             | 0                       | 0                        | 0.64 $\pm$ 0.01                           | 0                   |
| ('LinearSVM', 'No Upsampling', 'LinearSVM')                                                         | 0.56 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LinearSVM', 'RakelD Partitioning of labels', 'RakelD_LineaSVM')                                   | 0.55 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| ('LinearSVM', 'Random Oversampling', 'LinearSVM_ROS')                                               | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LinearSVM', 'SMOTE', 'LinearSVM_SMOTE')                                                           | 0.56 $\pm$ 0.02 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'BorderlineSMOTE', 'LogisticRegression_BorderlineSMOTE')                     | 0.55 $\pm$ 0.00 | 0.60 $\pm$ 0.02             | 0.60 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'No Upsampling', 'LogisticRegression')                                       | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.60 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegression', 'Random Oversampling', 'LogisticRegression_ROS')                             | 0.56 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'SMOTE', 'LogisticRegression_SMOTE')                                         | 0.56 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegression', 'SVMSMOTE', 'LogisticRegression_SVMSMOTE')                                   | 0.57 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.61 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegressionElasticNet', 'BorderlineSMOTE', 'LogisticRegressionElasticNet_BorderlineSMOTE') | 0.57 $\pm$ 0.02 | 0.62 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.63 $\pm$ 0.02          | 0.65 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegressionElasticNet', 'No Upsampling', 'LogisticRegressionElasticNet')                   | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.66 $\pm$ 0.01     |
| ('LogisticRegressionElasticNet', 'Random Oversampling', 'LogisticRegressionElasticNet_ROS')         | 0.57 $\pm$ 0.02 | 0.62 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.63 $\pm$ 0.02          | 0.65 $\pm$ 0.01                           | 0.67 $\pm$ 0.00     |
| ('LogisticRegressionElasticNet', 'SMOTE', 'LogisticRegressionElasticNet_SMOTE')                     | 0.58 $\pm$ 0.02 | 0.62 $\pm$ 0.02             | 0.62 $\pm$ 0.02         | 0.63 $\pm$ 0.02          | 0.65 $\pm$ 0.01                           | 0.67 $\pm$ 0.00     |
| ('LogisticRegressionElasticNet', 'SVMSMOTE', 'LogisticRegressionElasticNet_SVMSMOTE')               | 0.58 $\pm$ 0.01 | 0.63 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.63 $\pm$ 0.01          | 0.64 $\pm$ 0.01                           | 0.67 $\pm$ 0.01     |
| ('LogisticRegressionLasso', 'BorderlineSMOTE', 'LogisticRegressionLasso_BorderlineSMOTE')           | 0.53 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.59 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.60 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| ('LogisticRegressionLasso', 'No Upsampling', 'LogisticRegressionLasso')                             | 0.53 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.59 $\pm$ 0.02         | 0.57 $\pm$ 0.02          | 0.60 $\pm$ 0.01                           | 0.62 $\pm$ 0.00     |
| ('LogisticRegressionLasso', 'Random Oversampling', 'LogisticRegressionLasso_ROS')                   | 0.53 $\pm$ 0.02 | 0.59 $\pm$ 0.02             | 0.59 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.60 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('LogisticRegressionLasso', 'SMOTE', 'LogisticRegressionLasso_SMOTE')                               | 0.53 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.59 $\pm$ 0.02         | 0.58 $\pm$ 0.02          | 0.61 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('LogisticRegressionLasso', 'SVMSMOTE', 'LogisticRegressionLasso_SVMSMOTE')                         | 0.53 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.60 $\pm$ 0.03         | 0.60 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('LogisticRegressionRidge', 'BorderlineSMOTE', 'LogisticRegressionRidge_BorderlineSMOTE')           | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'No Upsampling', 'LogisticRegressionRidge')                             | 0.55 $\pm$ 0.01 | 0.60 $\pm$ 0.02             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'RakelD Partitioning of labels', 'RakelD_LogisticRegression')           | 0.55 $\pm$ 0.01 | 0.59 $\pm$ 0.02             | 0.61 $\pm$ 0.01         | 0.61 $\pm$ 0.02          | 0.64 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'Random Oversampling', 'LogisticRegressionRidge_ROS')                   | 0.56 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.62 $\pm$ 0.02         | 0.62 $\pm$ 0.02          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'SMOTE', 'LogisticRegressionRidge_SMOTE')                               | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.02     |
| ('LogisticRegressionRidge', 'SVMSMOTE', 'LogisticRegressionRidge_SVMSMOTE')                         | 0.57 $\pm$ 0.01 | 0.61 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.66 $\pm$ 0.02     |
| ('Multi-label ARAM', 'Natively Multilabel', 'MLARAM')                                               | 0.31 $\pm$ 0.02 | 0.31 $\pm$ 0.03             | 0.31 $\pm$ 0.02         | 0.33 $\pm$ 0.03          | 0.33 $\pm$ 0.03                           | 0.26 $\pm$ 0.08     |
| ('Multilabel k Nearest Neighbours', 'Natively Multilabel', 'MLkNN')                                 | 0.47 $\pm$ 0.04 | 0.49 $\pm$ 0.04             | 0.51 $\pm$ 0.01         | 0.54 $\pm$ 0.02          | 0.55 $\pm$ 0.01                           | 0.59 $\pm$ 0.00     |
| ('NaiveBayes', 'BorderlineSMOTE', 'ComplementNaiveBayes_BorderlineSMOTE')                           | 0.62 $\pm$ 0.01 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | 0.68 $\pm$ 0.01     |
| ('NaiveBayes', 'BorderlineSMOTE', 'NaiveBayes_BorderlineSMOTE')                                     | 0.61 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | 0.68 $\pm$ 0.01     |
| ('NaiveBayes', 'No Upsampling', 'ComplementNaiveBayes')                                             | 0.61 $\pm$ 0.01 | 0.66 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.65 $\pm$ 0.02          | 0.64 $\pm$ 0.03                           | 0.64 $\pm$ 0.02     |
| ('NaiveBayes', 'No Upsampling', 'NaiveBayes')                                                       | 0.37 $\pm$ 0.01 | 0.51 $\pm$ 0.01             | 0.54 $\pm$ 0.02         | 0.56 $\pm$ 0.02          | 0.57 $\pm$ 0.02                           | 0.58 $\pm$ 0.02     |
| ('NaiveBayes', 'Random Oversampling', 'ComplementNaiveBayes_ROS')                                   | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.69 $\pm$ 0.01** |
| ('NaiveBayes', 'Random Oversampling', 'NaiveBayes_ROS')                                             | 0.62 $\pm$ 0.00 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.68 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | 0.68 $\pm$ 0.01     |
| ('NaiveBayes', 'SMOTE', 'ComplementNaiveBayes_SMOTE')                                               | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.69 $\pm$ 0.01** |
| ('NaiveBayes', 'SMOTE', 'NaiveBayes_SMOTE')                                                         | 0.62 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.01         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.69 $\pm$ 0.01** |
| ('RandomForest', 'BorderlineSMOTE', 'RandomForest_BorderlineSMOTE')                                 | 0.52 $\pm$ 0.02 | 0.54 $\pm$ 0.01             | 0.56 $\pm$ 0.03         | 0.59 $\pm$ 0.01          | 0.61 $\pm$ 0.02                           | 0.64 $\pm$ 0.01     |
| ('RandomForest', 'No Upsampling', 'RandomForest')                                                   | 0.50 $\pm$ 0.02 | 0.53 $\pm$ 0.02             | 0.56 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.60 $\pm$ 0.02                           | 0.61 $\pm$ 0.01     |
| ('RandomForest', 'Random Oversampling', 'RandomForest_ROS')                                         | 0.56 $\pm$ 0.02 | 0.58 $\pm$ 0.02             | 0.60 $\pm$ 0.03         | 0.62 $\pm$ 0.02          | 0.63 $\pm$ 0.02                           | 0.64 $\pm$ 0.02     |
| ('RandomForest', 'SMOTE', 'RandomForest_SMOTE')                                                     | 0.52 $\pm$ 0.03 | 0.55 $\pm$ 0.02             | 0.57 $\pm$ 0.03         | 0.58 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.02     |
| ('RandomForest', 'SVMSMOTE', 'RandomForest_SVMSMOTE')                                               | 0.53 $\pm$ 0.02 | 0.55 $\pm$ 0.01             | 0.57 $\pm$ 0.03         | 0.60 $\pm$ 0.02          | 0.61 $\pm$ 0.02                           | 0.64 $\pm$ 0.01     |
| ('RidgeClassifier', 'BorderlineSMOTE', 'RidgeClassifier_BorderlineSMOTE')                           | 0.55 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'No Upsampling', 'RidgeClassifier')                                             | 0.55 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'Random Oversampling', 'RidgeClassifier_ROS')                                   | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'SMOTE', 'RidgeClassifier_SMOTE')                                               | 0.56 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.02         | 0.62 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.66 $\pm$ 0.01     |
| ('RidgeClassifier', 'SVMSMOTE', 'RidgeClassifier_SVMSMOTE')                                         | 0.57 $\pm$ 0.01 | 0.60 $\pm$ 0.01             | 0.61 $\pm$ 0.01         | 0.62 $\pm$ 0.01          | 0.63 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| ('SVM', 'BorderlineSMOTE', 'SVM_rbf_BorderlineSMOTE')                                               | 0.46 $\pm$ 0.01 | 0.37 $\pm$ 0.01             | 0.35 $\pm$ 0.01         | 0.36 $\pm$ 0.02          | 0.37 $\pm$ 0.02                           | 0.37 $\pm$ 0.01     |
| ('SVM', 'No Upsampling', 'SVM_rbf')                                                                 | 0.38 $\pm$ 0.01 | 0.38 $\pm$ 0.02             | 0.34 $\pm$ 0.01         | 0.37 $\pm$ 0.01          | 0.40 $\pm$ 0.02                           | 0.41 $\pm$ 0.02     |
| ('SVM', 'RakelD Partitioning of labels', 'RakelD_SVM')                                              | 0.41 $\pm$ 0.03 | 0.36 $\pm$ 0.02             | 0.31 $\pm$ 0.00         | 0.37 $\pm$ 0.03          | 0.40 $\pm$ 0.02                           | 0.31 $\pm$ 0.04     |
| ('SVM', 'Random Oversampling', 'SVM_rbf_ROS')                                                       | 0.49 $\pm$ 0.00 | 0.44 $\pm$ 0.02             | 0.40 $\pm$ 0.01         | 0.41 $\pm$ 0.02          | 0.41 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| ('SVM', 'SMOTE', 'SVM_rbf_SMOTE')                                                                   | 0.48 $\pm$ 0.01 | 0.37 $\pm$ 0.01             | 0.35 $\pm$ 0.01         | 0.35 $\pm$ 0.01          | 0.38 $\pm$ 0.03                           | 0.37 $\pm$ 0.01     |
| ('SVM', 'SVMSMOTE', 'SVM_rbf_SVMSMOTE')                                                             | 0.50 $\pm$ 0.01 | 0.42 $\pm$ 0.00             | 0.39 $\pm$ 0.02         | 0.38 $\pm$ 0.01          | 0.34 $\pm$ 0.01                           | 0.35 $\pm$ 0.02     |
| ('XGBoost', 'BorderlineSMOTE', 'XGBoost_narrow_BorderlineSMOTE')                                    | 0.53 $\pm$ 0.01 | 0.56 $\pm$ 0.01             | 0.57 $\pm$ 0.03         | 0.57 $\pm$ 0.01          | 0.60 $\pm$ 0.01                           | 0.64 $\pm$ 0.00     |
| ('XGBoost', 'No Upsampling', 'XGBoost_narrow')                                                      | 0.52 $\pm$ 0.02 | 0.57 $\pm$ 0.02             | 0.56 $\pm$ 0.02         | 0.58 $\pm$ 0.00          | 0.60 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| ('XGBoost', 'Random Oversampling', 'XGBoost_narrow_ROS')                                            | 0.53 $\pm$ 0.02 | 0.57 $\pm$ 0.01             | 0.57 $\pm$ 0.02         | 0.59 $\pm$ 0.01          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |
| ('XGBoost', 'SMOTE', 'XGBoost_narrow_SMOTE')                                                        | 0.53 $\pm$ 0.02 | 0.57 $\pm$ 0.01             | 0.57 $\pm$ 0.02         | 0.58 $\pm$ 0.00          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.01     |
| ('XGBoost', 'SVMSMOTE', 'XGBoost_narrow_SVMSMOTE')                                                  | 0.53 $\pm$ 0.02 | 0.56 $\pm$ 0.01             | 0.56 $\pm$ 0.02         | 0.58 $\pm$ 0.01          | 0.61 $\pm$ 0.01                           | 0.64 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_type                      | model_subtype                 | model_name                                   | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:--------------------------------|:------------------------------|:---------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.23 $\pm$ 0.06 | 0.42 $\pm$ 0.03             | 0.45 $\pm$ 0.08         | 0.43 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | 0.68 $\pm$ 0.02     |
| en         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.16 $\pm$ 0.01 | 0.23 $\pm$ 0.05             | 0.16 $\pm$ 0.01         | 0.19 $\pm$ 0.01          | 0.19 $\pm$ 0.02                           | 0.19 $\pm$ 0.01     |
| en         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.55 $\pm$ 0.02 | 0.63 $\pm$ 0.01             | 0.65 $\pm$ 0.01         | 0.64 $\pm$ 0.01          | 0.65 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.23 $\pm$ 0.00 | 0.23 $\pm$ 0.00             | 0.23 $\pm$ 0.00         | 0.23 $\pm$ 0.00          | 0.23 $\pm$ 0.00                           | 0.23 $\pm$ 0.00     |
| en         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.40 $\pm$ 0.01 | 0.38 $\pm$ 0.01             | 0.40 $\pm$ 0.01         | 0.40 $\pm$ 0.02          | 0.40 $\pm$ 0.01                           | 0.39 $\pm$ 0.01     |
| en         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.39 $\pm$ 0.01 | 0.40 $\pm$ 0.01             | 0.39 $\pm$ 0.01         | 0.41 $\pm$ 0.01          | 0.41 $\pm$ 0.01                           | 0.40 $\pm$ 0.01     |
| en         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.36 $\pm$ 0.02 | 0.39 $\pm$ 0.08             | 0.36 $\pm$ 0.07         | 0.45 $\pm$ 0.08          | 0.40 $\pm$ 0.11                           | 0.50 $\pm$ 0.05     |
| en         | KNN                             | No Upsampling                 | kNN                                          | 0.42 $\pm$ 0.08 | 0.51 $\pm$ 0.05             | 0.54 $\pm$ 0.04         | 0.55 $\pm$ 0.04          | 0.56 $\pm$ 0.06                           | 0.66 $\pm$ 0.02     |
| en         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.45 $\pm$ 0.05 | 0.48 $\pm$ 0.01             | 0.53 $\pm$ 0.06         | 0.52 $\pm$ 0.01          | 0.54 $\pm$ 0.02                           | 0.61 $\pm$ 0.02     |
| en         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.36 $\pm$ 0.06 | 0.33 $\pm$ 0.11             | 0.42 $\pm$ 0.08         | 0.51 $\pm$ 0.05          | 0.45 $\pm$ 0.07                           | 0.50 $\pm$ 0.05     |
| en         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0               | 0.36 $\pm$ 0.09             | 0                       | 0                        | 0.42 $\pm$ 0.08                           | 0                   |
| en         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.56 $\pm$ 0.00 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0               | 0.65 $\pm$ 0.01             | 0                       | 0                        | 0.67 $\pm$ 0.02                           | 0                   |
| en         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.58 $\pm$ 0.00 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.57 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.66 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.67 $\pm$ 0.01     |
| en         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.58 $\pm$ 0.00 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.55 $\pm$ 0.02 | 0.65 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.56 $\pm$ 0.00 | 0.64 $\pm$ 0.01             | 0.67 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.58 $\pm$ 0.00 | 0.64 $\pm$ 0.02             | 0.67 $\pm$ 0.03         | 0.66 $\pm$ 0.02          | 0.67 $\pm$ 0.03                           | 0.68 $\pm$ 0.02     |
| en         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.58 $\pm$ 0.00 | 0.64 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.56 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.03                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.58 $\pm$ 0.01 | 0.62 $\pm$ 0.01             | 0.65 $\pm$ 0.02         | 0.66 $\pm$ 0.01          | 0.66 $\pm$ 0.03                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.59 $\pm$ 0.02 | 0.66 $\pm$ 0.02             | 0.66 $\pm$ 0.03         | 0.66 $\pm$ 0.01          | 0.66 $\pm$ 0.02                           | 0.67 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.60 $\pm$ 0.02 | 0.67 $\pm$ 0.01             | 0.67 $\pm$ 0.03         | 0.67 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.59 $\pm$ 0.02 | 0.65 $\pm$ 0.01             | 0.66 $\pm$ 0.02         | 0.65 $\pm$ 0.02          | 0.65 $\pm$ 0.02                           | 0.67 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.59 $\pm$ 0.02 | 0.66 $\pm$ 0.02             | 0.66 $\pm$ 0.03         | 0.66 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.68 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.59 $\pm$ 0.01 | 0.65 $\pm$ 0.01             | 0.66 $\pm$ 0.02         | 0.67 $\pm$ 0.02          | 0.66 $\pm$ 0.02                           | 0.68 $\pm$ 0.02     |
| en         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.60 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.64 $\pm$ 0.03         | 0.61 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.62 $\pm$ 0.00 | 0.64 $\pm$ 0.03             | 0.65 $\pm$ 0.02         | 0.61 $\pm$ 0.01          | 0.63 $\pm$ 0.01                           | 0.63 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.60 $\pm$ 0.01 | 0.64 $\pm$ 0.03             | 0.64 $\pm$ 0.03         | 0.61 $\pm$ 0.02          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.59 $\pm$ 0.01 | 0.64 $\pm$ 0.03             | 0.64 $\pm$ 0.02         | 0.61 $\pm$ 0.01          | 0.62 $\pm$ 0.01                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.62 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.03         | 0.63 $\pm$ 0.02          | 0.63 $\pm$ 0.02                           | 0.63 $\pm$ 0.00     |
| en         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.59 $\pm$ 0.00 | 0.67 $\pm$ 0.01             | 0.69 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.68 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.62 $\pm$ 0.01 | 0.68 $\pm$ 0.01             | 0.69 $\pm$ 0.03         | 0.68 $\pm$ 0.03          | 0.68 $\pm$ 0.03                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.61 $\pm$ 0.01 | 0.66 $\pm$ 0.02             | 0.67 $\pm$ 0.03         | 0.66 $\pm$ 0.03          | 0.69 $\pm$ 0.02                           | 0.69 $\pm$ 0.03     |
| en         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.61 $\pm$ 0.01 | 0.67 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.59 $\pm$ 0.00 | 0.68 $\pm$ 0.01             | 0.69 $\pm$ 0.03         | 0.68 $\pm$ 0.02          | 0.68 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.60 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.02         | 0.68 $\pm$ 0.03          | 0.68 $\pm$ 0.02                           | 0.70 $\pm$ 0.02     |
| en         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.23 $\pm$ 0.02 | 0.24 $\pm$ 0.02             | 0.23 $\pm$ 0.02         | 0.25 $\pm$ 0.02          | 0.25 $\pm$ 0.02                           | 0.19 $\pm$ 0.06     |
| en         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.52 $\pm$ 0.01 | 0.52 $\pm$ 0.01             | 0.56 $\pm$ 0.03         | 0.55 $\pm$ 0.01          | 0.59 $\pm$ 0.02                           | 0.63 $\pm$ 0.02     |
| en         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.59 $\pm$ 0.02 | 0.65 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.64 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.64 $\pm$ 0.02     |
| en         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.58 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.64 $\pm$ 0.01          | 0.64 $\pm$ 0.02                           | 0.64 $\pm$ 0.02     |
| en         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.57 $\pm$ 0.01 | 0.68 $\pm$ 0.03             | 0.73 $\pm$ 0.04         | 0.74 $\pm$ 0.01          | 0.74 $\pm$ 0.05                           | 0.75 $\pm$ 0.02     |
| en         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.52 $\pm$ 0.00 | 0.69 $\pm$ 0.04             | 0.73 $\pm$ 0.05         | 0.71 $\pm$ 0.05          | 0.70 $\pm$ 0.01                           | 0.70 $\pm$ 0.02     |
| en         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.58 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.02         | 0.64 $\pm$ 0.01          | 0.65 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| en         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.58 $\pm$ 0.01 | 0.64 $\pm$ 0.02             | 0.65 $\pm$ 0.02         | 0.64 $\pm$ 0.02          | 0.65 $\pm$ 0.02                           | 0.65 $\pm$ 0.01     |
| en         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.59 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.65 $\pm$ 0.01          | 0.65 $\pm$ 0.02                           | 0.65 $\pm$ 0.02     |
| en         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.58 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.66 $\pm$ 0.02         | 0.65 $\pm$ 0.01          | 0.66 $\pm$ 0.02                           | 0.65 $\pm$ 0.02     |
| en         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.71 $\pm$ 0.01 | 0.73 $\pm$ 0.03             | 0.71 $\pm$ 0.05         | 0.73 $\pm$ 0.01          | 0.75 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.71 $\pm$ 0.01 | 0.73 $\pm$ 0.01             | 0.74 $\pm$ 0.04         | 0.75 $\pm$ 0.01          | **0.77 $\pm$ 0.01**                       | 0.74 $\pm$ 0.00     |
| en         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.67 $\pm$ 0.01 | 0.75 $\pm$ 0.01             | 0.74 $\pm$ 0.02         | 0.73 $\pm$ 0.01          | 0.74 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.70 $\pm$ 0.00 | 0.74 $\pm$ 0.02             | 0.72 $\pm$ 0.02         | 0.73 $\pm$ 0.01          | 0.76 $\pm$ 0.02                           | 0.73 $\pm$ 0.03     |
| en         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.72 $\pm$ 0.02 | 0.75 $\pm$ 0.03             | 0.72 $\pm$ 0.03         | 0.73 $\pm$ 0.01          | 0.74 $\pm$ 0.03                           | 0.74 $\pm$ 0.03     |
| en         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.59 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.59 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.60 $\pm$ 0.01 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.59 $\pm$ 0.00 | 0.66 $\pm$ 0.01             | 0.68 $\pm$ 0.03         | 0.67 $\pm$ 0.02          | 0.67 $\pm$ 0.02                           | 0.69 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.58 $\pm$ 0.01 | 0.65 $\pm$ 0.02             | 0.67 $\pm$ 0.02         | 0.68 $\pm$ 0.02          | 0.67 $\pm$ 0.03                           | 0.69 $\pm$ 0.02     |
| en         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.60 $\pm$ 0.02 | 0.60 $\pm$ 0.06             | 0.54 $\pm$ 0.04         | 0.54 $\pm$ 0.05          | 0.45 $\pm$ 0.03                           | 0.54 $\pm$ 0.04     |
| en         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.54 $\pm$ 0.02 | 0.54 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.04          | 0.48 $\pm$ 0.01                           | 0.54 $\pm$ 0.04     |
| en         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.48 $\pm$ 0.02 | 0.49 $\pm$ 0.03             | 0.41 $\pm$ 0.11         | 0.48 $\pm$ 0.03          | 0.43 $\pm$ 0.07                           | 0.53 $\pm$ 0.03     |
| en         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.62 $\pm$ 0.01 | 0.63 $\pm$ 0.03             | 0.61 $\pm$ 0.06         | 0.63 $\pm$ 0.04          | 0.56 $\pm$ 0.02                           | 0.65 $\pm$ 0.02     |
| en         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.58 $\pm$ 0.01 | 0.61 $\pm$ 0.04             | 0.58 $\pm$ 0.05         | 0.53 $\pm$ 0.04          | 0.47 $\pm$ 0.02                           | 0.55 $\pm$ 0.04     |
| en         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.52 $\pm$ 0.03 | 0.61 $\pm$ 0.02             | 0.55 $\pm$ 0.05         | 0.60 $\pm$ 0.04          | 0.55 $\pm$ 0.04                           | 0.61 $\pm$ 0.05     |
| en         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.60 $\pm$ 0.02 | 0.64 $\pm$ 0.01             | 0.63 $\pm$ 0.01         | 0.64 $\pm$ 0.01          | 0.66 $\pm$ 0.02                           | 0.70 $\pm$ 0.02     |
| en         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.59 $\pm$ 0.02 | 0.65 $\pm$ 0.02             | 0.64 $\pm$ 0.02         | 0.67 $\pm$ 0.01          | 0.69 $\pm$ 0.01                           | 0.74 $\pm$ 0.01     |
| en         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.58 $\pm$ 0.02 | 0.64 $\pm$ 0.02             | 0.63 $\pm$ 0.02         | 0.66 $\pm$ 0.01          | 0.68 $\pm$ 0.00                           | 0.71 $\pm$ 0.01     |
| en         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.60 $\pm$ 0.02 | 0.63 $\pm$ 0.02             | 0.64 $\pm$ 0.01         | 0.65 $\pm$ 0.00          | 0.67 $\pm$ 0.01                           | 0.69 $\pm$ 0.02     |
| en         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.60 $\pm$ 0.02 | 0.64 $\pm$ 0.01             | 0.63 $\pm$ 0.02         | 0.65 $\pm$ 0.01          | 0.67 $\pm$ 0.02                           | 0.71 $\pm$ 0.02     |
| fr         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.11 $\pm$ 0.12 | 0.14 $\pm$ 0.06             | 0.10 $\pm$ 0.02         | 0.07 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.09 $\pm$ 0.07     |
| fr         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.08 $\pm$ 0.04 | 0.12 $\pm$ 0.02             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| fr         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.35 $\pm$ 0.03 | 0.37 $\pm$ 0.01             | 0.43 $\pm$ 0.01         | 0.45 $\pm$ 0.03          | 0.42 $\pm$ 0.01                           | 0.48 $\pm$ 0.04     |
| fr         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.27 $\pm$ 0.02 | 0.31 $\pm$ 0.03             | 0.26 $\pm$ 0.02         | 0.27 $\pm$ 0.01          | 0.30 $\pm$ 0.04                           | 0.27 $\pm$ 0.02     |
| fr         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.25 $\pm$ 0.02 | 0.28 $\pm$ 0.02             | 0.27 $\pm$ 0.03         | 0.27 $\pm$ 0.01          | 0.28 $\pm$ 0.01                           | 0.29 $\pm$ 0.01     |
| fr         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.28 $\pm$ 0.01 | 0.28 $\pm$ 0.01             | 0.28 $\pm$ 0.01         | 0.27 $\pm$ 0.01          | 0.26 $\pm$ 0.03                           | 0.27 $\pm$ 0.02     |
| fr         | KNN                             | No Upsampling                 | kNN                                          | 0.30 $\pm$ 0.01 | 0.19 $\pm$ 0.03             | 0.18 $\pm$ 0.03         | 0.26 $\pm$ 0.03          | 0.08 $\pm$ 0.08                           | 0.17 $\pm$ 0.15     |
| fr         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.35 $\pm$ 0.02 | 0.31 $\pm$ 0.01             | 0.30 $\pm$ 0.04         | 0.28 $\pm$ 0.08          | 0.14 $\pm$ 0.03                           | 0.40 $\pm$ 0.06     |
| fr         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.28 $\pm$ 0.01 | 0.28 $\pm$ 0.01             | 0.28 $\pm$ 0.01         | 0.28 $\pm$ 0.01          | 0.27 $\pm$ 0.05                           | 0.28 $\pm$ 0.01     |
| fr         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.39 $\pm$ 0.01 | 0.44 $\pm$ 0.03             | 0.47 $\pm$ 0.06         | 0.52 $\pm$ 0.06          | 0.44 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| fr         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.06     |
| fr         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.41 $\pm$ 0.01 | 0.46 $\pm$ 0.03             | 0.49 $\pm$ 0.05         | 0.53 $\pm$ 0.03          | 0.46 $\pm$ 0.05                           | 0.51 $\pm$ 0.07     |
| fr         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.41 $\pm$ 0.02 | 0.43 $\pm$ 0.02             | 0.50 $\pm$ 0.07         | 0.50 $\pm$ 0.05          | 0.45 $\pm$ 0.05                           | 0.55 $\pm$ 0.09     |
| fr         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.41 $\pm$ 0.02 | 0.48 $\pm$ 0.03             | 0.52 $\pm$ 0.08         | 0.55 $\pm$ 0.04          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.06     |
| fr         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.03             | 0.51 $\pm$ 0.05         | 0.54 $\pm$ 0.05          | 0.46 $\pm$ 0.04                           | 0.52 $\pm$ 0.06     |
| fr         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.47 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.08     |
| fr         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.43 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.48 $\pm$ 0.03         | 0.50 $\pm$ 0.01          | 0.46 $\pm$ 0.04                           | 0.57 $\pm$ 0.07     |
| fr         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.44 $\pm$ 0.03 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.03         | 0.55 $\pm$ 0.01          | 0.47 $\pm$ 0.04                           | 0.57 $\pm$ 0.08     |
| fr         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.42 $\pm$ 0.01 | 0.41 $\pm$ 0.04             | 0.46 $\pm$ 0.03         | 0.50 $\pm$ 0.04          | 0.45 $\pm$ 0.04                           | 0.58 $\pm$ 0.06     |
| fr         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.43 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.47 $\pm$ 0.03         | 0.50 $\pm$ 0.02          | 0.47 $\pm$ 0.04                           | **0.59 $\pm$ 0.07** |
| fr         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.43 $\pm$ 0.03 | 0.44 $\pm$ 0.04             | 0.50 $\pm$ 0.05         | 0.53 $\pm$ 0.02          | 0.49 $\pm$ 0.05                           | 0.56 $\pm$ 0.08     |
| fr         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.46 $\pm$ 0.02 | 0.39 $\pm$ 0.01             | 0.42 $\pm$ 0.04         | 0.41 $\pm$ 0.04          | 0.46 $\pm$ 0.03                           | 0.48 $\pm$ 0.05     |
| fr         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.43 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.41 $\pm$ 0.06          | 0.45 $\pm$ 0.03                           | 0.47 $\pm$ 0.04     |
| fr         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.46 $\pm$ 0.02 | 0.40 $\pm$ 0.02             | 0.43 $\pm$ 0.02         | 0.41 $\pm$ 0.05          | 0.46 $\pm$ 0.02                           | 0.48 $\pm$ 0.04     |
| fr         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.43 $\pm$ 0.02 | 0.39 $\pm$ 0.01             | 0.43 $\pm$ 0.04         | 0.41 $\pm$ 0.04          | 0.45 $\pm$ 0.02                           | 0.45 $\pm$ 0.03     |
| fr         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.49 $\pm$ 0.01 | 0.42 $\pm$ 0.02             | 0.45 $\pm$ 0.02         | 0.40 $\pm$ 0.03          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.41 $\pm$ 0.02 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.08         | 0.54 $\pm$ 0.04          | 0.52 $\pm$ 0.07                           | 0.52 $\pm$ 0.09     |
| fr         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.40 $\pm$ 0.04 | 0.43 $\pm$ 0.03             | 0.47 $\pm$ 0.07         | 0.54 $\pm$ 0.05          | 0.52 $\pm$ 0.04                           | 0.54 $\pm$ 0.09     |
| fr         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.42 $\pm$ 0.05 | 0.41 $\pm$ 0.02             | 0.44 $\pm$ 0.04         | 0.52 $\pm$ 0.05          | 0.53 $\pm$ 0.05                           | 0.53 $\pm$ 0.07     |
| fr         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.44 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.53 $\pm$ 0.09         | 0.55 $\pm$ 0.03          | 0.54 $\pm$ 0.08                           | 0.53 $\pm$ 0.08     |
| fr         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.40 $\pm$ 0.04 | 0.45 $\pm$ 0.02             | 0.53 $\pm$ 0.09         | 0.53 $\pm$ 0.02          | 0.51 $\pm$ 0.07                           | 0.53 $\pm$ 0.09     |
| fr         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.42 $\pm$ 0.04 | 0.46 $\pm$ 0.02             | 0.50 $\pm$ 0.09         | 0.54 $\pm$ 0.04          | 0.49 $\pm$ 0.09                           | 0.50 $\pm$ 0.08     |
| fr         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.11 $\pm$ 0.01 | 0.10 $\pm$ 0.00             | 0.10 $\pm$ 0.00         | 0.06 $\pm$ 0.03          | 0.08 $\pm$ 0.03                           | 0.08 $\pm$ 0.03     |
| fr         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.39 $\pm$ 0.03 | 0.34 $\pm$ 0.02             | 0.33 $\pm$ 0.03         | 0.40 $\pm$ 0.01          | 0.33 $\pm$ 0.08                           | 0.43 $\pm$ 0.05     |
| fr         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.38 $\pm$ 0.02 | 0.41 $\pm$ 0.02             | 0.43 $\pm$ 0.00         | 0.45 $\pm$ 0.03          | 0.44 $\pm$ 0.01                           | 0.49 $\pm$ 0.03     |
| fr         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.38 $\pm$ 0.03 | 0.40 $\pm$ 0.03             | 0.43 $\pm$ 0.01         | 0.45 $\pm$ 0.02          | 0.44 $\pm$ 0.01                           | 0.48 $\pm$ 0.03     |
| fr         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.35 $\pm$ 0.03 | 0.40 $\pm$ 0.03             | 0.44 $\pm$ 0.02         | 0.51 $\pm$ 0.06          | 0.51 $\pm$ 0.01                           | 0.51 $\pm$ 0.04     |
| fr         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.19 $\pm$ 0.06 | 0.16 $\pm$ 0.06             | 0.23 $\pm$ 0.04         | 0.28 $\pm$ 0.05          | 0.25 $\pm$ 0.04                           | 0.28 $\pm$ 0.03     |
| fr         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.38 $\pm$ 0.03 | 0.39 $\pm$ 0.03             | 0.44 $\pm$ 0.02         | 0.44 $\pm$ 0.02          | 0.45 $\pm$ 0.01                           | 0.47 $\pm$ 0.01     |
| fr         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.37 $\pm$ 0.01 | 0.40 $\pm$ 0.02             | 0.42 $\pm$ 0.02         | 0.45 $\pm$ 0.02          | 0.44 $\pm$ 0.00                           | 0.47 $\pm$ 0.01     |
| fr         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.38 $\pm$ 0.02 | 0.41 $\pm$ 0.03             | 0.44 $\pm$ 0.01         | 0.46 $\pm$ 0.03          | 0.44 $\pm$ 0.02                           | 0.49 $\pm$ 0.02     |
| fr         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.38 $\pm$ 0.02 | 0.40 $\pm$ 0.03             | 0.43 $\pm$ 0.01         | 0.44 $\pm$ 0.03          | 0.44 $\pm$ 0.01                           | 0.49 $\pm$ 0.03     |
| fr         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.48 $\pm$ 0.07 | 0.49 $\pm$ 0.12             | 0.48 $\pm$ 0.12         | 0.57 $\pm$ 0.09          | 0.45 $\pm$ 0.03                           | 0.49 $\pm$ 0.04     |
| fr         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.47 $\pm$ 0.08 | 0.34 $\pm$ 0.09             | 0.40 $\pm$ 0.09         | 0.41 $\pm$ 0.05          | 0.44 $\pm$ 0.08                           | 0.47 $\pm$ 0.04     |
| fr         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.53 $\pm$ 0.07 | 0.41 $\pm$ 0.06             | 0.52 $\pm$ 0.07         | 0.48 $\pm$ 0.04          | 0.44 $\pm$ 0.06                           | 0.57 $\pm$ 0.07     |
| fr         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.47 $\pm$ 0.03 | 0.38 $\pm$ 0.03             | 0.47 $\pm$ 0.06         | 0.57 $\pm$ 0.02          | 0.50 $\pm$ 0.07                           | 0.47 $\pm$ 0.04     |
| fr         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.50 $\pm$ 0.05 | 0.45 $\pm$ 0.07             | 0.47 $\pm$ 0.08         | 0.49 $\pm$ 0.12          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.05     |
| fr         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.02             | 0.52 $\pm$ 0.09         | 0.53 $\pm$ 0.03          | 0.53 $\pm$ 0.09                           | 0.52 $\pm$ 0.08     |
| fr         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.40 $\pm$ 0.03 | 0.45 $\pm$ 0.01             | 0.52 $\pm$ 0.10         | 0.54 $\pm$ 0.04          | 0.51 $\pm$ 0.07                           | 0.51 $\pm$ 0.09     |
| fr         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.35 $\pm$ 0.07 | 0.00 $\pm$ 0.00             | 0.07 $\pm$ 0.05         | 0.03 $\pm$ 0.05          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.14 $\pm$ 0.03 | 0.00 $\pm$ 0.00             | 0.10 $\pm$ 0.07         | 0.00 $\pm$ 0.00          | 0.09 $\pm$ 0.02                           | 0.05 $\pm$ 0.04     |
| fr         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.12 $\pm$ 0.06 | 0.05 $\pm$ 0.06             | 0.07 $\pm$ 0.06         | 0.06 $\pm$ 0.05          | 0.13 $\pm$ 0.05                           | 0.12 $\pm$ 0.04     |
| fr         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.30 $\pm$ 0.03 | 0.17 $\pm$ 0.10             | 0.16 $\pm$ 0.06         | 0.19 $\pm$ 0.06          | 0.16 $\pm$ 0.05                           | 0.19 $\pm$ 0.03     |
| fr         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.30 $\pm$ 0.05 | 0.05 $\pm$ 0.07             | 0.12 $\pm$ 0.02         | 0.05 $\pm$ 0.04          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.29 $\pm$ 0.04 | 0.15 $\pm$ 0.11             | 0.20 $\pm$ 0.13         | 0.15 $\pm$ 0.08          | 0.05 $\pm$ 0.07                           | 0.00 $\pm$ 0.00     |
| fr         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.50 $\pm$ 0.01 | 0.38 $\pm$ 0.02             | 0.37 $\pm$ 0.05         | 0.43 $\pm$ 0.04          | 0.42 $\pm$ 0.03                           | 0.53 $\pm$ 0.07     |
| fr         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.44 $\pm$ 0.04 | 0.36 $\pm$ 0.04             | 0.38 $\pm$ 0.04         | 0.41 $\pm$ 0.04          | 0.37 $\pm$ 0.01                           | 0.52 $\pm$ 0.08     |
| fr         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.44 $\pm$ 0.03 | 0.36 $\pm$ 0.05             | 0.39 $\pm$ 0.03         | 0.44 $\pm$ 0.05          | 0.39 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| fr         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.48 $\pm$ 0.01 | 0.38 $\pm$ 0.01             | 0.37 $\pm$ 0.00         | 0.41 $\pm$ 0.03          | 0.42 $\pm$ 0.03                           | 0.51 $\pm$ 0.04     |
| fr         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.47 $\pm$ 0.02 | 0.38 $\pm$ 0.01             | 0.39 $\pm$ 0.04         | 0.38 $\pm$ 0.05          | 0.42 $\pm$ 0.04                           | 0.50 $\pm$ 0.05     |
| ge         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.31 $\pm$ 0.07 | 0.30 $\pm$ 0.06             | 0.10 $\pm$ 0.04         | 0.15 $\pm$ 0.02          | 0.18 $\pm$ 0.05                           | 0.10 $\pm$ 0.05     |
| ge         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.18 $\pm$ 0.03 | 0.18 $\pm$ 0.02             | 0.19 $\pm$ 0.05         | 0.17 $\pm$ 0.05          | 0.18 $\pm$ 0.06                           | 0.17 $\pm$ 0.12     |
| ge         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.44 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.42 $\pm$ 0.03         | 0.48 $\pm$ 0.02          | 0.44 $\pm$ 0.06                           | 0.43 $\pm$ 0.02     |
| ge         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.11 $\pm$ 0.03 | 0.11 $\pm$ 0.03             | 0.11 $\pm$ 0.03         | 0.11 $\pm$ 0.03          | 0.11 $\pm$ 0.03                           | 0.11 $\pm$ 0.03     |
| ge         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.39 $\pm$ 0.02 | 0.37 $\pm$ 0.04             | 0.40 $\pm$ 0.02         | 0.36 $\pm$ 0.03          | 0.38 $\pm$ 0.04                           | 0.41 $\pm$ 0.03     |
| ge         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.39 $\pm$ 0.02 | 0.37 $\pm$ 0.03             | 0.38 $\pm$ 0.04         | 0.39 $\pm$ 0.02          | 0.37 $\pm$ 0.04                           | 0.38 $\pm$ 0.03     |
| ge         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.42 $\pm$ 0.04 | 0.38 $\pm$ 0.05             | 0.40 $\pm$ 0.09         | 0.27 $\pm$ 0.05          | 0.32 $\pm$ 0.06                           | 0.24 $\pm$ 0.01     |
| ge         | KNN                             | No Upsampling                 | kNN                                          | 0.41 $\pm$ 0.01 | 0.34 $\pm$ 0.06             | 0.39 $\pm$ 0.07         | 0.28 $\pm$ 0.09          | 0.19 $\pm$ 0.01                           | 0.15 $\pm$ 0.04     |
| ge         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.47 $\pm$ 0.02 | 0.41 $\pm$ 0.08             | 0.45 $\pm$ 0.07         | 0.34 $\pm$ 0.06          | 0.24 $\pm$ 0.06                           | 0.20 $\pm$ 0.06     |
| ge         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.35 $\pm$ 0.02 | 0.35 $\pm$ 0.06             | 0.32 $\pm$ 0.01         | 0.32 $\pm$ 0.07          | 0.28 $\pm$ 0.06                           | 0.24 $\pm$ 0.02     |
| ge         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0.37 $\pm$ 0.05 | 0.35 $\pm$ 0.03             | 0.43 $\pm$ 0.06         | 0                        | 0.32 $\pm$ 0.10                           | 0                   |
| ge         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0                        | 0.46 $\pm$ 0.05                           | 0                   |
| ge         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.42 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.42 $\pm$ 0.03 | 0.46 $\pm$ 0.03             | 0.48 $\pm$ 0.06         | 0.48 $\pm$ 0.06          | 0.46 $\pm$ 0.02                           | 0.52 $\pm$ 0.03     |
| ge         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.01             | 0.43 $\pm$ 0.05         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.41 $\pm$ 0.03 | 0.50 $\pm$ 0.02             | 0.45 $\pm$ 0.03         | 0.46 $\pm$ 0.05          | 0.47 $\pm$ 0.05                           | 0.51 $\pm$ 0.06     |
| ge         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.41 $\pm$ 0.03 | 0.48 $\pm$ 0.03             | 0.43 $\pm$ 0.04         | 0.45 $\pm$ 0.04          | 0.46 $\pm$ 0.04                           | 0.48 $\pm$ 0.03     |
| ge         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.41 $\pm$ 0.04 | 0.48 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.05                           | 0.50 $\pm$ 0.02     |
| ge         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.41 $\pm$ 0.03 | 0.50 $\pm$ 0.02             | 0.45 $\pm$ 0.03         | 0.46 $\pm$ 0.05          | 0.46 $\pm$ 0.05                           | 0.50 $\pm$ 0.03     |
| ge         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.41 $\pm$ 0.03 | 0.48 $\pm$ 0.04             | 0.43 $\pm$ 0.03         | 0.49 $\pm$ 0.03          | 0.45 $\pm$ 0.06                           | 0.48 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.42 $\pm$ 0.05 | 0.47 $\pm$ 0.02             | 0.45 $\pm$ 0.04         | 0.52 $\pm$ 0.05          | 0.48 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.42 $\pm$ 0.04 | 0.47 $\pm$ 0.02             | 0.48 $\pm$ 0.06         | 0.51 $\pm$ 0.05          | 0.48 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.43 $\pm$ 0.03 | 0.47 $\pm$ 0.01             | 0.45 $\pm$ 0.04         | 0.53 $\pm$ 0.05          | 0.49 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.43 $\pm$ 0.05 | 0.47 $\pm$ 0.01             | 0.45 $\pm$ 0.05         | 0.51 $\pm$ 0.05          | 0.49 $\pm$ 0.04                           | 0.56 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.41 $\pm$ 0.04 | 0.49 $\pm$ 0.02             | 0.46 $\pm$ 0.05         | 0.53 $\pm$ 0.05          | 0.47 $\pm$ 0.03                           | 0.55 $\pm$ 0.04     |
| ge         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.45 $\pm$ 0.02 | 0.44 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.49 $\pm$ 0.05                           | 0.57 $\pm$ 0.03     |
| ge         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.48 $\pm$ 0.02 | 0.45 $\pm$ 0.04             | 0.45 $\pm$ 0.04         | 0.50 $\pm$ 0.03          | 0.49 $\pm$ 0.05                           | 0.55 $\pm$ 0.04     |
| ge         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.46 $\pm$ 0.02 | 0.46 $\pm$ 0.04             | 0.45 $\pm$ 0.01         | 0.50 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.57 $\pm$ 0.03     |
| ge         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.45 $\pm$ 0.01 | 0.46 $\pm$ 0.05             | 0.46 $\pm$ 0.00         | 0.49 $\pm$ 0.04          | 0.50 $\pm$ 0.05                           | 0.58 $\pm$ 0.03     |
| ge         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.45 $\pm$ 0.02 | 0.48 $\pm$ 0.04             | 0.46 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.48 $\pm$ 0.04                           | 0.55 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.45 $\pm$ 0.04                           | 0.55 $\pm$ 0.03     |
| ge         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.38 $\pm$ 0.05 | 0.48 $\pm$ 0.03             | 0.40 $\pm$ 0.03         | 0.45 $\pm$ 0.02          | 0.45 $\pm$ 0.04                           | 0.54 $\pm$ 0.04     |
| ge         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.42 $\pm$ 0.03 | 0.47 $\pm$ 0.05             | 0.43 $\pm$ 0.03         | 0.46 $\pm$ 0.04          | 0.44 $\pm$ 0.01                           | 0.54 $\pm$ 0.07     |
| ge         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.42 $\pm$ 0.05 | 0.49 $\pm$ 0.04             | 0.42 $\pm$ 0.04         | 0.45 $\pm$ 0.02          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.45 $\pm$ 0.02          | 0.46 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.41 $\pm$ 0.05 | 0.49 $\pm$ 0.05             | 0.43 $\pm$ 0.05         | 0.47 $\pm$ 0.00          | 0.46 $\pm$ 0.01                           | 0.55 $\pm$ 0.01     |
| ge         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.08 $\pm$ 0.03 | 0.12 $\pm$ 0.07             | 0.12 $\pm$ 0.07         | 0.12 $\pm$ 0.07          | 0.13 $\pm$ 0.07                           | 0.12 $\pm$ 0.07     |
| ge         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.41 $\pm$ 0.04 | 0.43 $\pm$ 0.04             | 0.41 $\pm$ 0.02         | 0.44 $\pm$ 0.04          | 0.40 $\pm$ 0.08                           | 0.45 $\pm$ 0.02     |
| ge         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.46 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.03         | 0.51 $\pm$ 0.02          | 0.49 $\pm$ 0.04                           | 0.53 $\pm$ 0.03     |
| ge         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.46 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.51 $\pm$ 0.02          | 0.50 $\pm$ 0.05                           | 0.53 $\pm$ 0.02     |
| ge         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.44 $\pm$ 0.02 | 0.48 $\pm$ 0.02             | 0.47 $\pm$ 0.01         | 0.51 $\pm$ 0.02          | 0.48 $\pm$ 0.03                           | 0.52 $\pm$ 0.03     |
| ge         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.32 $\pm$ 0.11 | 0.31 $\pm$ 0.10             | 0.47 $\pm$ 0.04         | 0.43 $\pm$ 0.02          | 0.48 $\pm$ 0.08                           | 0.48 $\pm$ 0.04     |
| ge         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.45 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.50 $\pm$ 0.02          | 0.49 $\pm$ 0.05                           | 0.52 $\pm$ 0.03     |
| ge         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.45 $\pm$ 0.02 | 0.47 $\pm$ 0.01             | 0.47 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.48 $\pm$ 0.04                           | 0.52 $\pm$ 0.03     |
| ge         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.45 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.01         | 0.51 $\pm$ 0.02          | 0.50 $\pm$ 0.04                           | 0.53 $\pm$ 0.02     |
| ge         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.46 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.51 $\pm$ 0.03          | 0.49 $\pm$ 0.04                           | 0.54 $\pm$ 0.03     |
| ge         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.44 $\pm$ 0.07 | 0.44 $\pm$ 0.03             | 0.43 $\pm$ 0.03         | 0.45 $\pm$ 0.05          | 0.57 $\pm$ 0.10                           | **0.64 $\pm$ 0.07** |
| ge         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.43 $\pm$ 0.01 | 0.40 $\pm$ 0.02             | 0.44 $\pm$ 0.00         | 0.45 $\pm$ 0.03          | 0.46 $\pm$ 0.03                           | 0.52 $\pm$ 0.04     |
| ge         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.46 $\pm$ 0.01 | 0.47 $\pm$ 0.03             | 0.45 $\pm$ 0.03         | 0.49 $\pm$ 0.04          | 0.49 $\pm$ 0.04                           | 0.54 $\pm$ 0.04     |
| ge         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.41 $\pm$ 0.01 | 0.44 $\pm$ 0.04             | 0.44 $\pm$ 0.03         | 0.45 $\pm$ 0.02          | 0.58 $\pm$ 0.13                           | **0.64 $\pm$ 0.11** |
| ge         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.41 $\pm$ 0.03 | 0.45 $\pm$ 0.03             | 0.43 $\pm$ 0.02         | 0.49 $\pm$ 0.03          | 0.52 $\pm$ 0.09                           | 0.60 $\pm$ 0.08     |
| ge         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.41 $\pm$ 0.03 | 0.49 $\pm$ 0.04             | 0.41 $\pm$ 0.03         | 0.47 $\pm$ 0.04          | 0.47 $\pm$ 0.06                           | 0.55 $\pm$ 0.01     |
| ge         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.42 $\pm$ 0.03 | 0.48 $\pm$ 0.05             | 0.44 $\pm$ 0.04         | 0.48 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.54 $\pm$ 0.02     |
| ge         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.36 $\pm$ 0.05 | 0.43 $\pm$ 0.07             | 0.34 $\pm$ 0.05         | 0.17 $\pm$ 0.03          | 0.28 $\pm$ 0.08                           | 0.26 $\pm$ 0.02     |
| ge         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.33 $\pm$ 0.10 | 0.25 $\pm$ 0.04             | 0.31 $\pm$ 0.05         | 0.31 $\pm$ 0.06          | 0.25 $\pm$ 0.03                           | 0.30 $\pm$ 0.02     |
| ge         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.27 $\pm$ 0.03 | 0.23 $\pm$ 0.04             | 0.17 $\pm$ 0.02         | 0.19 $\pm$ 0.02          | 0.30 $\pm$ 0.10                           | 0.30 $\pm$ 0.06     |
| ge         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.39 $\pm$ 0.09 | 0.40 $\pm$ 0.03             | 0.41 $\pm$ 0.06         | 0.35 $\pm$ 0.08          | 0.25 $\pm$ 0.03                           | 0.33 $\pm$ 0.02     |
| ge         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.38 $\pm$ 0.10 | 0.41 $\pm$ 0.09             | 0.37 $\pm$ 0.09         | 0.20 $\pm$ 0.06          | 0.31 $\pm$ 0.10                           | 0.26 $\pm$ 0.02     |
| ge         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.39 $\pm$ 0.04 | 0.55 $\pm$ 0.08             | 0.32 $\pm$ 0.11         | 0.24 $\pm$ 0.05          | 0.23 $\pm$ 0.07                           | 0.24 $\pm$ 0.01     |
| ge         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.45 $\pm$ 0.01 | 0.45 $\pm$ 0.01             | 0.47 $\pm$ 0.02         | 0.47 $\pm$ 0.03          | 0.49 $\pm$ 0.03                           | 0.55 $\pm$ 0.06     |
| ge         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.44 $\pm$ 0.05 | 0.45 $\pm$ 0.03             | 0.49 $\pm$ 0.02         | 0.45 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.53 $\pm$ 0.03     |
| ge         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.48 $\pm$ 0.03 | 0.48 $\pm$ 0.03             | 0.46 $\pm$ 0.03         | 0.46 $\pm$ 0.01          | 0.47 $\pm$ 0.03                           | 0.56 $\pm$ 0.02     |
| ge         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.47 $\pm$ 0.02 | 0.47 $\pm$ 0.01             | 0.46 $\pm$ 0.03         | 0.48 $\pm$ 0.05          | 0.50 $\pm$ 0.04                           | 0.56 $\pm$ 0.06     |
| ge         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.44 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.46 $\pm$ 0.01          | 0.50 $\pm$ 0.05                           | 0.55 $\pm$ 0.04     |
| it         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.17 $\pm$ 0.07 | 0.17 $\pm$ 0.04             | 0.04 $\pm$ 0.03         | 0.09 $\pm$ 0.05          | 0.08 $\pm$ 0.03                           | 0.04 $\pm$ 0.06     |
| it         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.19 $\pm$ 0.04 | 0.18 $\pm$ 0.01             | 0.12 $\pm$ 0.03         | 0.15 $\pm$ 0.07          | 0.12 $\pm$ 0.05                           | 0.13 $\pm$ 0.06     |
| it         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.40 $\pm$ 0.00 | 0.43 $\pm$ 0.02             | 0.45 $\pm$ 0.02         | 0.50 $\pm$ 0.02          | 0.40 $\pm$ 0.03                           | 0.46 $\pm$ 0.02     |
| it         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.32 $\pm$ 0.02 | 0.31 $\pm$ 0.03             | 0.32 $\pm$ 0.02         | 0.31 $\pm$ 0.03          | 0.34 $\pm$ 0.03                           | 0.34 $\pm$ 0.03     |
| it         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.34 $\pm$ 0.03 | 0.32 $\pm$ 0.03             | 0.31 $\pm$ 0.03         | 0.31 $\pm$ 0.04          | 0.32 $\pm$ 0.03                           | 0.33 $\pm$ 0.03     |
| it         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.33 $\pm$ 0.01 | 0.33 $\pm$ 0.01             | 0.34 $\pm$ 0.02         | 0.32 $\pm$ 0.02          | 0.32 $\pm$ 0.02                           | 0.33 $\pm$ 0.01     |
| it         | KNN                             | No Upsampling                 | kNN                                          | 0.36 $\pm$ 0.07 | 0.20 $\pm$ 0.05             | 0.17 $\pm$ 0.01         | 0.22 $\pm$ 0.14          | 0.14 $\pm$ 0.08                           | 0.11 $\pm$ 0.05     |
| it         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.44 $\pm$ 0.03 | 0.34 $\pm$ 0.03             | 0.36 $\pm$ 0.09         | 0.32 $\pm$ 0.02          | 0.30 $\pm$ 0.01                           | 0.33 $\pm$ 0.09     |
| it         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.33 $\pm$ 0.01 | 0.33 $\pm$ 0.02             | 0.32 $\pm$ 0.02         | 0.32 $\pm$ 0.02          | 0.35 $\pm$ 0.00                           | 0.30 $\pm$ 0.04     |
| it         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0.34 $\pm$ 0.02 | 0                           | 0.30 $\pm$ 0.04         | 0.32 $\pm$ 0.02          | 0                                         | 0.31 $\pm$ 0.04     |
| it         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0.42 $\pm$ 0.01 | 0                           | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0                                         | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.41 $\pm$ 0.00 | 0.48 $\pm$ 0.01             | 0.46 $\pm$ 0.02         | 0.51 $\pm$ 0.04          | 0.52 $\pm$ 0.02                           | 0.52 $\pm$ 0.02     |
| it         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.42 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.55 $\pm$ 0.05          | 0.49 $\pm$ 0.02                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.42 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.55 $\pm$ 0.06          | 0.49 $\pm$ 0.01                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.01             | 0.47 $\pm$ 0.02         | 0.54 $\pm$ 0.05          | 0.46 $\pm$ 0.02                           | 0.57 $\pm$ 0.02     |
| it         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.54 $\pm$ 0.06          | 0.50 $\pm$ 0.02                           | 0.56 $\pm$ 0.02     |
| it         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.42 $\pm$ 0.01 | 0.47 $\pm$ 0.02             | 0.47 $\pm$ 0.02         | 0.56 $\pm$ 0.06          | 0.50 $\pm$ 0.02                           | 0.57 $\pm$ 0.03     |
| it         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.41 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.47 $\pm$ 0.02         | 0.51 $\pm$ 0.02          | 0.49 $\pm$ 0.02                           | 0.55 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.43 $\pm$ 0.01 | 0.49 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.57 $\pm$ 0.05          | 0.52 $\pm$ 0.03                           | 0.55 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.44 $\pm$ 0.01 | 0.52 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.59 $\pm$ 0.04          | 0.53 $\pm$ 0.02                           | 0.56 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.43 $\pm$ 0.02 | 0.48 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.57 $\pm$ 0.04          | 0.50 $\pm$ 0.01                           | 0.55 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.43 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.05          | 0.51 $\pm$ 0.02                           | 0.54 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.44 $\pm$ 0.01 | 0.49 $\pm$ 0.03             | 0.48 $\pm$ 0.02         | 0.56 $\pm$ 0.05          | 0.52 $\pm$ 0.01                           | 0.56 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.46 $\pm$ 0.01 | 0.45 $\pm$ 0.03             | 0.43 $\pm$ 0.04         | 0.51 $\pm$ 0.03          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.46 $\pm$ 0.02 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.05         | 0.52 $\pm$ 0.04          | 0.48 $\pm$ 0.02                           | 0.51 $\pm$ 0.04     |
| it         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.46 $\pm$ 0.01 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.05         | 0.51 $\pm$ 0.04          | 0.47 $\pm$ 0.03                           | 0.51 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.45 $\pm$ 0.01 | 0.45 $\pm$ 0.02             | 0.44 $\pm$ 0.04         | 0.50 $\pm$ 0.04          | 0.49 $\pm$ 0.03                           | 0.51 $\pm$ 0.03     |
| it         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.45 $\pm$ 0.02 | 0.45 $\pm$ 0.02             | 0.45 $\pm$ 0.04         | 0.50 $\pm$ 0.03          | 0.48 $\pm$ 0.01                           | 0.51 $\pm$ 0.02     |
| it         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.45 $\pm$ 0.01 | 0.51 $\pm$ 0.03             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.57 $\pm$ 0.04                           | **0.62 $\pm$ 0.08** |
| it         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.46 $\pm$ 0.01 | 0.52 $\pm$ 0.02             | 0.51 $\pm$ 0.04         | 0.59 $\pm$ 0.08          | 0.55 $\pm$ 0.02                           | **0.62 $\pm$ 0.06** |
| it         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.46 $\pm$ 0.02 | 0.49 $\pm$ 0.02             | 0.50 $\pm$ 0.03         | 0.59 $\pm$ 0.05          | 0.53 $\pm$ 0.05                           | 0.58 $\pm$ 0.04     |
| it         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.45 $\pm$ 0.02 | 0.51 $\pm$ 0.03             | 0.50 $\pm$ 0.02         | 0.59 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.45 $\pm$ 0.01 | 0.52 $\pm$ 0.03             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.57 $\pm$ 0.06                           | 0.61 $\pm$ 0.07     |
| it         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.43 $\pm$ 0.01 | 0.48 $\pm$ 0.02             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.09          | 0.54 $\pm$ 0.03                           | 0.61 $\pm$ 0.05     |
| it         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.06 $\pm$ 0.00 | 0.06 $\pm$ 0.00             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.01                           | 0.06 $\pm$ 0.00     |
| it         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.39 $\pm$ 0.01 | 0.40 $\pm$ 0.03             | 0.39 $\pm$ 0.02         | 0.39 $\pm$ 0.02          | 0.38 $\pm$ 0.02                           | 0.40 $\pm$ 0.02     |
| it         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.41 $\pm$ 0.00 | 0.46 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.52 $\pm$ 0.02          | 0.48 $\pm$ 0.02                           | 0.51 $\pm$ 0.03     |
| it         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.41 $\pm$ 0.01 | 0.45 $\pm$ 0.01             | 0.49 $\pm$ 0.02         | 0.51 $\pm$ 0.02          | 0.49 $\pm$ 0.02                           | 0.50 $\pm$ 0.03     |
| it         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.40 $\pm$ 0.01 | 0.48 $\pm$ 0.01             | 0.51 $\pm$ 0.02         | 0.58 $\pm$ 0.00          | 0.58 $\pm$ 0.02                           | 0.58 $\pm$ 0.02     |
| it         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.12 $\pm$ 0.05 | 0.27 $\pm$ 0.02             | 0.30 $\pm$ 0.05         | 0.42 $\pm$ 0.11          | 0.40 $\pm$ 0.08                           | 0.44 $\pm$ 0.06     |
| it         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.41 $\pm$ 0.01 | 0.46 $\pm$ 0.01             | 0.48 $\pm$ 0.02         | 0.51 $\pm$ 0.01          | 0.48 $\pm$ 0.01                           | 0.51 $\pm$ 0.03     |
| it         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.41 $\pm$ 0.01 | 0.46 $\pm$ 0.01             | 0.49 $\pm$ 0.02         | 0.50 $\pm$ 0.01          | 0.47 $\pm$ 0.02                           | 0.51 $\pm$ 0.02     |
| it         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.41 $\pm$ 0.01 | 0.45 $\pm$ 0.01             | 0.48 $\pm$ 0.02         | 0.53 $\pm$ 0.02          | 0.48 $\pm$ 0.02                           | 0.51 $\pm$ 0.03     |
| it         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.41 $\pm$ 0.01 | 0.45 $\pm$ 0.01             | 0.49 $\pm$ 0.02         | 0.52 $\pm$ 0.02          | 0.48 $\pm$ 0.02                           | 0.50 $\pm$ 0.03     |
| it         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.50 $\pm$ 0.05 | 0.51 $\pm$ 0.02             | 0.57 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.57 $\pm$ 0.06                           | 0.61 $\pm$ 0.07     |
| it         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.44 $\pm$ 0.01 | 0.53 $\pm$ 0.03             | 0.53 $\pm$ 0.09         | 0.56 $\pm$ 0.09          | 0.52 $\pm$ 0.02                           | 0.59 $\pm$ 0.03     |
| it         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.47 $\pm$ 0.03 | 0.55 $\pm$ 0.03             | 0.49 $\pm$ 0.04         | 0.55 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | **0.62 $\pm$ 0.06** |
| it         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.51 $\pm$ 0.04 | 0.49 $\pm$ 0.01             | 0.49 $\pm$ 0.00         | 0.54 $\pm$ 0.07          | 0.60 $\pm$ 0.04                           | 0.59 $\pm$ 0.05     |
| it         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.52 $\pm$ 0.02 | 0.51 $\pm$ 0.03             | 0.48 $\pm$ 0.06         | 0.52 $\pm$ 0.06          | 0.57 $\pm$ 0.07                           | **0.62 $\pm$ 0.02** |
| it         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.03             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.44 $\pm$ 0.02 | 0.51 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.60 $\pm$ 0.07          | 0.56 $\pm$ 0.05                           | 0.60 $\pm$ 0.07     |
| it         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.42 $\pm$ 0.01 | 0.46 $\pm$ 0.04             | 0.47 $\pm$ 0.01         | 0.56 $\pm$ 0.07          | 0.51 $\pm$ 0.02                           | 0.59 $\pm$ 0.05     |
| it         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.44 $\pm$ 0.04 | 0.42 $\pm$ 0.07             | 0.22 $\pm$ 0.06         | 0.25 $\pm$ 0.06          | 0.21 $\pm$ 0.06                           | 0.29 $\pm$ 0.13     |
| it         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.28 $\pm$ 0.03 | 0.23 $\pm$ 0.08             | 0.18 $\pm$ 0.07         | 0.23 $\pm$ 0.04          | 0.18 $\pm$ 0.05                           | 0.24 $\pm$ 0.02     |
| it         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.13 $\pm$ 0.05 | 0.12 $\pm$ 0.04             | 0.10 $\pm$ 0.03         | 0.13 $\pm$ 0.04          | 0.14 $\pm$ 0.03                           | 0.14 $\pm$ 0.05     |
| it         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.48 $\pm$ 0.04 | 0.35 $\pm$ 0.06             | 0.30 $\pm$ 0.04         | 0.33 $\pm$ 0.05          | 0.31 $\pm$ 0.03                           | 0.31 $\pm$ 0.03     |
| it         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.45 $\pm$ 0.03 | 0.39 $\pm$ 0.05             | 0.22 $\pm$ 0.06         | 0.24 $\pm$ 0.06          | 0.21 $\pm$ 0.07                           | 0.25 $\pm$ 0.10     |
| it         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.36 $\pm$ 0.03 | 0.43 $\pm$ 0.07             | 0.32 $\pm$ 0.02         | 0.26 $\pm$ 0.05          | 0.17 $\pm$ 0.03                           | 0.17 $\pm$ 0.04     |
| it         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.44 $\pm$ 0.03 | 0.44 $\pm$ 0.04             | 0.44 $\pm$ 0.02         | 0.47 $\pm$ 0.03          | 0.50 $\pm$ 0.05                           | 0.53 $\pm$ 0.03     |
| it         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.43 $\pm$ 0.05 | 0.43 $\pm$ 0.03             | 0.43 $\pm$ 0.02         | 0.46 $\pm$ 0.02          | 0.51 $\pm$ 0.05                           | 0.55 $\pm$ 0.03     |
| it         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.42 $\pm$ 0.03 | 0.42 $\pm$ 0.04             | 0.45 $\pm$ 0.02         | 0.49 $\pm$ 0.04          | 0.48 $\pm$ 0.05                           | 0.55 $\pm$ 0.04     |
| it         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.43 $\pm$ 0.02 | 0.43 $\pm$ 0.04             | 0.46 $\pm$ 0.02         | 0.50 $\pm$ 0.03          | 0.52 $\pm$ 0.05                           | 0.54 $\pm$ 0.04     |
| it         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.43 $\pm$ 0.05 | 0.44 $\pm$ 0.03             | 0.45 $\pm$ 0.03         | 0.49 $\pm$ 0.04          | 0.52 $\pm$ 0.07                           | 0.57 $\pm$ 0.03     |
| po         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.38 $\pm$ 0.07 | 0.34 $\pm$ 0.05             | 0.31 $\pm$ 0.05         | 0.32 $\pm$ 0.07          | 0.20 $\pm$ 0.11                           | 0.20 $\pm$ 0.10     |
| po         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.23 $\pm$ 0.03 | 0.22 $\pm$ 0.01             | 0.24 $\pm$ 0.03         | 0.19 $\pm$ 0.04          | 0.15 $\pm$ 0.05                           | 0.15 $\pm$ 0.05     |
| po         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.46 $\pm$ 0.04 | 0.47 $\pm$ 0.04             | 0.53 $\pm$ 0.01         | 0.57 $\pm$ 0.02          | 0.51 $\pm$ 0.05                           | 0.53 $\pm$ 0.04     |
| po         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.23 $\pm$ 0.01 | 0.23 $\pm$ 0.01             | 0.23 $\pm$ 0.01         | 0.23 $\pm$ 0.01          | 0.23 $\pm$ 0.01                           | 0.23 $\pm$ 0.01     |
| po         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.43 $\pm$ 0.04 | 0.42 $\pm$ 0.03             | 0.42 $\pm$ 0.04         | 0.43 $\pm$ 0.02          | 0.40 $\pm$ 0.03                           | 0.42 $\pm$ 0.03     |
| po         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.43 $\pm$ 0.02 | 0.42 $\pm$ 0.03             | 0.43 $\pm$ 0.03         | 0.41 $\pm$ 0.02          | 0.40 $\pm$ 0.00                           | 0.40 $\pm$ 0.04     |
| po         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.39 $\pm$ 0.06 | 0.38 $\pm$ 0.09             | 0.27 $\pm$ 0.02         | 0.27 $\pm$ 0.04          | 0.23 $\pm$ 0.09                           | 0.32 $\pm$ 0.10     |
| po         | KNN                             | No Upsampling                 | kNN                                          | 0.46 $\pm$ 0.05 | 0.38 $\pm$ 0.04             | 0.47 $\pm$ 0.06         | 0.46 $\pm$ 0.02          | 0.29 $\pm$ 0.04                           | 0.30 $\pm$ 0.09     |
| po         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.48 $\pm$ 0.05 | 0.45 $\pm$ 0.03             | 0.50 $\pm$ 0.04         | 0.54 $\pm$ 0.04          | 0.33 $\pm$ 0.04                           | 0.38 $\pm$ 0.08     |
| po         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.39 $\pm$ 0.06 | 0.37 $\pm$ 0.11             | 0.27 $\pm$ 0.02         | 0.20 $\pm$ 0.03          | 0.23 $\pm$ 0.05                           | 0.19 $\pm$ 0.03     |
| po         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0.42 $\pm$ 0.03 | 0.38 $\pm$ 0.04             | 0.34 $\pm$ 0.04         | 0.25 $\pm$ 0.06          | 0                                         | 0                   |
| po         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.58 $\pm$ 0.03          | 0                                         | 0                   |
| po         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.46 $\pm$ 0.05 | 0.52 $\pm$ 0.04             | 0.57 $\pm$ 0.04         | 0.60 $\pm$ 0.03          | 0.55 $\pm$ 0.03                           | 0.59 $\pm$ 0.04     |
| po         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.48 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.05         | 0.59 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.03             | 0.61 $\pm$ 0.05         | 0.59 $\pm$ 0.04          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.48 $\pm$ 0.04 | 0.53 $\pm$ 0.04             | 0.57 $\pm$ 0.04         | 0.58 $\pm$ 0.04          | 0.56 $\pm$ 0.04                           | 0.60 $\pm$ 0.04     |
| po         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.48 $\pm$ 0.04 | 0.53 $\pm$ 0.03             | 0.58 $\pm$ 0.04         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.04             | 0.59 $\pm$ 0.04         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.48 $\pm$ 0.04 | 0.52 $\pm$ 0.04             | 0.55 $\pm$ 0.04         | 0.59 $\pm$ 0.03          | 0.58 $\pm$ 0.03                           | 0.63 $\pm$ 0.06     |
| po         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.49 $\pm$ 0.04 | 0.52 $\pm$ 0.03             | 0.55 $\pm$ 0.05         | 0.61 $\pm$ 0.08          | 0.58 $\pm$ 0.03                           | 0.63 $\pm$ 0.05     |
| po         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.02             | 0.55 $\pm$ 0.05         | 0.60 $\pm$ 0.08          | 0.57 $\pm$ 0.04                           | 0.61 $\pm$ 0.04     |
| po         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.49 $\pm$ 0.05 | 0.52 $\pm$ 0.03             | 0.54 $\pm$ 0.05         | 0.60 $\pm$ 0.07          | 0.58 $\pm$ 0.04                           | 0.62 $\pm$ 0.05     |
| po         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.49 $\pm$ 0.04 | 0.52 $\pm$ 0.03             | 0.55 $\pm$ 0.05         | 0.61 $\pm$ 0.08          | 0.58 $\pm$ 0.03                           | 0.62 $\pm$ 0.04     |
| po         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.50 $\pm$ 0.05 | 0.54 $\pm$ 0.03             | 0.58 $\pm$ 0.05         | 0.61 $\pm$ 0.04          | 0.58 $\pm$ 0.04                           | 0.63 $\pm$ 0.06     |
| po         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.51 $\pm$ 0.04 | 0.51 $\pm$ 0.04             | 0.52 $\pm$ 0.04         | 0.58 $\pm$ 0.03          | 0.57 $\pm$ 0.02                           | 0.59 $\pm$ 0.04     |
| po         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.53 $\pm$ 0.06 | 0.51 $\pm$ 0.03             | 0.53 $\pm$ 0.04         | 0.57 $\pm$ 0.04          | 0.57 $\pm$ 0.02                           | 0.58 $\pm$ 0.03     |
| po         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.50 $\pm$ 0.05 | 0.52 $\pm$ 0.03             | 0.51 $\pm$ 0.03         | 0.57 $\pm$ 0.03          | 0.57 $\pm$ 0.02                           | 0.58 $\pm$ 0.04     |
| po         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.51 $\pm$ 0.06 | 0.52 $\pm$ 0.04             | 0.52 $\pm$ 0.03         | 0.57 $\pm$ 0.04          | 0.57 $\pm$ 0.01                           | 0.59 $\pm$ 0.04     |
| po         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.52 $\pm$ 0.04 | 0.53 $\pm$ 0.04             | 0.52 $\pm$ 0.06         | 0.56 $\pm$ 0.02          | 0.56 $\pm$ 0.02                           | 0.58 $\pm$ 0.06     |
| po         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.50 $\pm$ 0.04 | 0.55 $\pm$ 0.05             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.63 $\pm$ 0.03                           | 0.64 $\pm$ 0.04     |
| po         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.48 $\pm$ 0.04 | 0.55 $\pm$ 0.04             | 0.60 $\pm$ 0.06         | 0.56 $\pm$ 0.04          | 0.64 $\pm$ 0.04                           | 0.65 $\pm$ 0.04     |
| po         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.49 $\pm$ 0.05 | 0.56 $\pm$ 0.06             | 0.59 $\pm$ 0.02         | 0.56 $\pm$ 0.02          | 0.61 $\pm$ 0.07                           | 0.61 $\pm$ 0.03     |
| po         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.49 $\pm$ 0.04 | 0.54 $\pm$ 0.05             | 0.60 $\pm$ 0.05         | 0.58 $\pm$ 0.02          | 0.61 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.49 $\pm$ 0.04 | 0.55 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.59 $\pm$ 0.01          | 0.62 $\pm$ 0.02                           | 0.64 $\pm$ 0.03     |
| po         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.50 $\pm$ 0.05 | 0.54 $\pm$ 0.02             | 0.59 $\pm$ 0.03         | 0.59 $\pm$ 0.03          | 0.60 $\pm$ 0.05                           | 0.63 $\pm$ 0.03     |
| po         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.06 $\pm$ 0.00 | 0.06 $\pm$ 0.00             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.00                           | 0.06 $\pm$ 0.00     |
| po         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.46 $\pm$ 0.03 | 0.47 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.51 $\pm$ 0.03          | 0.53 $\pm$ 0.01                           | 0.55 $\pm$ 0.05     |
| po         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.50 $\pm$ 0.04 | 0.55 $\pm$ 0.03             | 0.57 $\pm$ 0.02         | 0.60 $\pm$ 0.04          | 0.59 $\pm$ 0.04                           | 0.58 $\pm$ 0.04     |
| po         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.49 $\pm$ 0.03 | 0.53 $\pm$ 0.03             | 0.58 $\pm$ 0.03         | 0.60 $\pm$ 0.03          | 0.59 $\pm$ 0.04                           | 0.59 $\pm$ 0.04     |
| po         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.49 $\pm$ 0.02 | 0.55 $\pm$ 0.03             | 0.58 $\pm$ 0.04         | 0.64 $\pm$ 0.06          | 0.60 $\pm$ 0.05                           | 0.61 $\pm$ 0.04     |
| po         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.35 $\pm$ 0.05 | 0.36 $\pm$ 0.05             | 0.36 $\pm$ 0.04         | 0.38 $\pm$ 0.05          | 0.37 $\pm$ 0.04                           | 0.39 $\pm$ 0.07     |
| po         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.50 $\pm$ 0.03 | 0.54 $\pm$ 0.04             | 0.58 $\pm$ 0.02         | 0.60 $\pm$ 0.04          | 0.58 $\pm$ 0.04                           | 0.59 $\pm$ 0.03     |
| po         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.49 $\pm$ 0.03 | 0.54 $\pm$ 0.03             | 0.58 $\pm$ 0.02         | 0.62 $\pm$ 0.04          | 0.59 $\pm$ 0.04                           | 0.59 $\pm$ 0.03     |
| po         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.50 $\pm$ 0.03 | 0.54 $\pm$ 0.03             | 0.57 $\pm$ 0.02         | 0.61 $\pm$ 0.04          | 0.59 $\pm$ 0.04                           | 0.58 $\pm$ 0.04     |
| po         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.50 $\pm$ 0.04 | 0.54 $\pm$ 0.03             | 0.57 $\pm$ 0.02         | 0.60 $\pm$ 0.04          | 0.58 $\pm$ 0.04                           | 0.59 $\pm$ 0.04     |
| po         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.57 $\pm$ 0.02 | 0.60 $\pm$ 0.05             | 0.59 $\pm$ 0.07         | 0.57 $\pm$ 0.03          | 0.70 $\pm$ 0.04                           | 0.70 $\pm$ 0.02     |
| po         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.48 $\pm$ 0.06 | 0.54 $\pm$ 0.08             | 0.56 $\pm$ 0.06         | 0.56 $\pm$ 0.05          | 0.52 $\pm$ 0.02                           | 0.60 $\pm$ 0.01     |
| po         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.57 $\pm$ 0.03 | 0.60 $\pm$ 0.02             | 0.58 $\pm$ 0.07         | 0.59 $\pm$ 0.01          | 0.65 $\pm$ 0.03                           | 0.70 $\pm$ 0.03     |
| po         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.55 $\pm$ 0.03 | 0.59 $\pm$ 0.04             | 0.55 $\pm$ 0.03         | 0.59 $\pm$ 0.07          | 0.66 $\pm$ 0.01                           | **0.72 $\pm$ 0.04** |
| po         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.58 $\pm$ 0.02 | 0.56 $\pm$ 0.05             | 0.58 $\pm$ 0.08         | 0.59 $\pm$ 0.08          | 0.67 $\pm$ 0.07                           | 0.71 $\pm$ 0.05     |
| po         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.05         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.49 $\pm$ 0.04 | 0.56 $\pm$ 0.04             | 0.60 $\pm$ 0.04         | 0.58 $\pm$ 0.01          | 0.62 $\pm$ 0.03                           | 0.64 $\pm$ 0.03     |
| po         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.49 $\pm$ 0.05 | 0.54 $\pm$ 0.04             | 0.57 $\pm$ 0.03         | 0.59 $\pm$ 0.04          | 0.63 $\pm$ 0.06                           | 0.62 $\pm$ 0.05     |
| po         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.52 $\pm$ 0.05 | 0.39 $\pm$ 0.05             | 0.39 $\pm$ 0.06         | 0.30 $\pm$ 0.02          | 0.26 $\pm$ 0.03                           | 0.32 $\pm$ 0.08     |
| po         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.35 $\pm$ 0.04 | 0.34 $\pm$ 0.03             | 0.28 $\pm$ 0.04         | 0.30 $\pm$ 0.02          | 0.32 $\pm$ 0.03                           | 0.34 $\pm$ 0.03     |
| po         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.34 $\pm$ 0.10 | 0.32 $\pm$ 0.01             | 0.28 $\pm$ 0.04         | 0.22 $\pm$ 0.07          | 0.25 $\pm$ 0.01                           | 0.26 $\pm$ 0.05     |
| po         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.52 $\pm$ 0.05 | 0.42 $\pm$ 0.03             | 0.45 $\pm$ 0.09         | 0.38 $\pm$ 0.04          | 0.36 $\pm$ 0.04                           | 0.39 $\pm$ 0.03     |
| po         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.50 $\pm$ 0.05 | 0.39 $\pm$ 0.04             | 0.38 $\pm$ 0.05         | 0.31 $\pm$ 0.01          | 0.27 $\pm$ 0.04                           | 0.29 $\pm$ 0.04     |
| po         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.46 $\pm$ 0.04 | 0.44 $\pm$ 0.06             | 0.35 $\pm$ 0.04         | 0.32 $\pm$ 0.01          | 0.23 $\pm$ 0.01                           | 0.23 $\pm$ 0.01     |
| po         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.54 $\pm$ 0.03 | 0.52 $\pm$ 0.03             | 0.52 $\pm$ 0.05         | 0.58 $\pm$ 0.05          | 0.59 $\pm$ 0.03                           | 0.66 $\pm$ 0.02     |
| po         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.52 $\pm$ 0.01 | 0.52 $\pm$ 0.02             | 0.53 $\pm$ 0.05         | 0.56 $\pm$ 0.06          | 0.59 $\pm$ 0.03                           | 0.67 $\pm$ 0.01     |
| po         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.51 $\pm$ 0.01 | 0.52 $\pm$ 0.03             | 0.53 $\pm$ 0.04         | 0.55 $\pm$ 0.05          | 0.59 $\pm$ 0.02                           | 0.63 $\pm$ 0.02     |
| po         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.54 $\pm$ 0.03 | 0.51 $\pm$ 0.03             | 0.52 $\pm$ 0.04         | 0.56 $\pm$ 0.06          | 0.57 $\pm$ 0.02                           | 0.65 $\pm$ 0.03     |
| po         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.56 $\pm$ 0.03 | 0.51 $\pm$ 0.03             | 0.54 $\pm$ 0.04         | 0.57 $\pm$ 0.04          | 0.59 $\pm$ 0.03                           | 0.66 $\pm$ 0.01     |
| ru         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.08 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.05 $\pm$ 0.04          | 0.04 $\pm$ 0.05                           | 0.01 $\pm$ 0.01     |
| ru         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.06 $\pm$ 0.00 | 0.07 $\pm$ 0.01             | 0.06 $\pm$ 0.00         | 0.06 $\pm$ 0.00          | 0.06 $\pm$ 0.00                           | 0.06 $\pm$ 0.00     |
| ru         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.30 $\pm$ 0.01 | 0.34 $\pm$ 0.05             | 0.45 $\pm$ 0.04         | 0.52 $\pm$ 0.03          | 0.43 $\pm$ 0.06                           | 0.39 $\pm$ 0.02     |
| ru         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.22 $\pm$ 0.00 | 0.21 $\pm$ 0.01             | 0.21 $\pm$ 0.01         | 0.25 $\pm$ 0.03          | 0.23 $\pm$ 0.02                           | 0.20 $\pm$ 0.01     |
| ru         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.23 $\pm$ 0.01 | 0.22 $\pm$ 0.02             | 0.19 $\pm$ 0.01         | 0.21 $\pm$ 0.00          | 0.20 $\pm$ 0.02                           | 0.23 $\pm$ 0.01     |
| ru         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.22 $\pm$ 0.01 | 0.25 $\pm$ 0.02             | 0.22 $\pm$ 0.00         | 0.22 $\pm$ 0.00          | 0.22 $\pm$ 0.00                           | 0.21 $\pm$ 0.01     |
| ru         | KNN                             | No Upsampling                 | kNN                                          | 0.26 $\pm$ 0.04 | 0.12 $\pm$ 0.07             | 0.20 $\pm$ 0.05         | 0.24 $\pm$ 0.03          | 0.11 $\pm$ 0.08                           | 0.20 $\pm$ 0.12     |
| ru         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.34 $\pm$ 0.01 | 0.36 $\pm$ 0.09             | 0.32 $\pm$ 0.01         | 0.36 $\pm$ 0.06          | 0.28 $\pm$ 0.01                           | 0.39 $\pm$ 0.05     |
| ru         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.22 $\pm$ 0.01 | 0.24 $\pm$ 0.01             | 0.22 $\pm$ 0.01         | 0.22 $\pm$ 0.01          | 0.23 $\pm$ 0.01                           | 0.22 $\pm$ 0.01     |
| ru         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.34 $\pm$ 0.05 | 0.40 $\pm$ 0.03             | 0.48 $\pm$ 0.07         | 0.59 $\pm$ 0.03          | 0.48 $\pm$ 0.06                           | 0.51 $\pm$ 0.05     |
| ru         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.34 $\pm$ 0.02 | 0.42 $\pm$ 0.05             | 0.49 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.49 $\pm$ 0.02                           | 0.52 $\pm$ 0.05     |
| ru         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.35 $\pm$ 0.01 | 0.42 $\pm$ 0.05             | 0.51 $\pm$ 0.05         | 0.60 $\pm$ 0.04          | 0.49 $\pm$ 0.03                           | 0.53 $\pm$ 0.04     |
| ru         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.34 $\pm$ 0.03 | 0.42 $\pm$ 0.04             | 0.49 $\pm$ 0.03         | 0.56 $\pm$ 0.02          | 0.46 $\pm$ 0.02                           | 0.47 $\pm$ 0.01     |
| ru         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.35 $\pm$ 0.02 | 0.43 $\pm$ 0.04             | 0.50 $\pm$ 0.03         | 0.58 $\pm$ 0.01          | 0.52 $\pm$ 0.03                           | 0.53 $\pm$ 0.02     |
| ru         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.34 $\pm$ 0.03 | 0.43 $\pm$ 0.04             | 0.49 $\pm$ 0.04         | 0.58 $\pm$ 0.02          | 0.49 $\pm$ 0.05                           | 0.52 $\pm$ 0.06     |
| ru         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0               | 0.42 $\pm$ 0.05             | 0                       | 0                        | 0                                         | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.37 $\pm$ 0.02 | 0.42 $\pm$ 0.02             | 0.48 $\pm$ 0.02         | 0.56 $\pm$ 0.02          | 0.48 $\pm$ 0.01                           | 0.56 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.37 $\pm$ 0.03 | 0.46 $\pm$ 0.01             | 0.48 $\pm$ 0.01         | 0.57 $\pm$ 0.01          | 0.53 $\pm$ 0.04                           | 0.58 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.37 $\pm$ 0.02 | 0.43 $\pm$ 0.01             | 0.49 $\pm$ 0.01         | 0.57 $\pm$ 0.02          | 0.47 $\pm$ 0.03                           | 0.52 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.37 $\pm$ 0.02 | 0.43 $\pm$ 0.02             | 0.46 $\pm$ 0.02         | 0.58 $\pm$ 0.02          | 0.49 $\pm$ 0.03                           | 0.57 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0               | 0.42 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.53 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.38 $\pm$ 0.04 | 0.43 $\pm$ 0.03             | 0.41 $\pm$ 0.03         | 0.48 $\pm$ 0.02          | 0.51 $\pm$ 0.07                           | 0.48 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.37 $\pm$ 0.06 | 0.43 $\pm$ 0.03             | 0.41 $\pm$ 0.01         | 0.48 $\pm$ 0.08          | 0.49 $\pm$ 0.04                           | 0.45 $\pm$ 0.01     |
| ru         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.34 $\pm$ 0.04 | 0.41 $\pm$ 0.02             | 0.41 $\pm$ 0.04         | 0.47 $\pm$ 0.02          | 0.50 $\pm$ 0.10                           | 0.46 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.39 $\pm$ 0.03 | 0.41 $\pm$ 0.03             | 0.40 $\pm$ 0.02         | 0.45 $\pm$ 0.03          | 0.48 $\pm$ 0.07                           | 0.46 $\pm$ 0.01     |
| ru         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0               | 0.40 $\pm$ 0.02             | 0                       | 0                        | 0                                         | 0.47 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.38 $\pm$ 0.02 | 0.44 $\pm$ 0.00             | 0.56 $\pm$ 0.04         | 0.57 $\pm$ 0.03          | 0.56 $\pm$ 0.02                           | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.37 $\pm$ 0.03 | 0.43 $\pm$ 0.02             | 0.56 $\pm$ 0.04         | 0.54 $\pm$ 0.07          | 0.52 $\pm$ 0.05                           | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.33 $\pm$ 0.03 | 0.43 $\pm$ 0.03             | 0.49 $\pm$ 0.01         | 0.46 $\pm$ 0.03          | 0.51 $\pm$ 0.09                           | 0.53 $\pm$ 0.06     |
| ru         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.38 $\pm$ 0.02 | 0.44 $\pm$ 0.02             | 0.53 $\pm$ 0.03         | 0.56 $\pm$ 0.04          | 0.54 $\pm$ 0.01                           | 0.56 $\pm$ 0.07     |
| ru         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.39 $\pm$ 0.03 | 0.45 $\pm$ 0.02             | 0.53 $\pm$ 0.03         | 0.55 $\pm$ 0.04          | 0.55 $\pm$ 0.02                           | 0.54 $\pm$ 0.03     |
| ru         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0               | 0.41 $\pm$ 0.03             | 0                       | 0                        | 0                                         | 0.50 $\pm$ 0.02     |
| ru         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.06 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.00         | 0.05 $\pm$ 0.00          | 0.06 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ru         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.31 $\pm$ 0.06 | 0.32 $\pm$ 0.02             | 0.35 $\pm$ 0.04         | 0.34 $\pm$ 0.05          | 0.36 $\pm$ 0.08                           | 0.39 $\pm$ 0.04     |
| ru         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.31 $\pm$ 0.01 | 0.36 $\pm$ 0.02             | 0.40 $\pm$ 0.01         | 0.43 $\pm$ 0.01          | 0.42 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| ru         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.29 $\pm$ 0.01 | 0.35 $\pm$ 0.02             | 0.39 $\pm$ 0.01         | 0.42 $\pm$ 0.03          | 0.42 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| ru         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.27 $\pm$ 0.00 | 0.36 $\pm$ 0.01             | 0.44 $\pm$ 0.04         | 0.60 $\pm$ 0.04          | 0.49 $\pm$ 0.04                           | 0.53 $\pm$ 0.01     |
| ru         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.05 $\pm$ 0.07         | 0.14 $\pm$ 0.00          | 0.14 $\pm$ 0.00                           | 0.10 $\pm$ 0.07     |
| ru         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.30 $\pm$ 0.01 | 0.37 $\pm$ 0.03             | 0.39 $\pm$ 0.01         | 0.42 $\pm$ 0.04          | 0.43 $\pm$ 0.02                           | 0.43 $\pm$ 0.03     |
| ru         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.31 $\pm$ 0.01 | 0.36 $\pm$ 0.03             | 0.39 $\pm$ 0.02         | 0.42 $\pm$ 0.03          | 0.42 $\pm$ 0.01                           | 0.42 $\pm$ 0.03     |
| ru         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.30 $\pm$ 0.01 | 0.36 $\pm$ 0.01             | 0.39 $\pm$ 0.02         | 0.44 $\pm$ 0.03          | 0.42 $\pm$ 0.02                           | 0.43 $\pm$ 0.02     |
| ru         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.31 $\pm$ 0.00 | 0.36 $\pm$ 0.02             | 0.38 $\pm$ 0.02         | 0.43 $\pm$ 0.01          | 0.42 $\pm$ 0.01                           | 0.45 $\pm$ 0.02     |
| ru         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.49 $\pm$ 0.05 | 0.51 $\pm$ 0.01             | 0.57 $\pm$ 0.03         | 0.55 $\pm$ 0.12          | 0.53 $\pm$ 0.05                           | 0.50 $\pm$ 0.08     |
| ru         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.45 $\pm$ 0.09 | 0.41 $\pm$ 0.10             | 0.35 $\pm$ 0.02         | 0.40 $\pm$ 0.06          | 0.38 $\pm$ 0.10                           | 0.45 $\pm$ 0.10     |
| ru         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.44 $\pm$ 0.01 | 0.54 $\pm$ 0.05             | 0.43 $\pm$ 0.02         | 0.56 $\pm$ 0.03          | 0.50 $\pm$ 0.02                           | 0.57 $\pm$ 0.04     |
| ru         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.55 $\pm$ 0.07 | 0.53 $\pm$ 0.06             | 0.46 $\pm$ 0.05         | 0.53 $\pm$ 0.04          | 0.49 $\pm$ 0.04                           | 0.52 $\pm$ 0.05     |
| ru         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0               | 0.49 $\pm$ 0.04             | 0                       | 0                        | 0                                         | **0.61 $\pm$ 0.07** |
| ru         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.37 $\pm$ 0.01 | 0.44 $\pm$ 0.01             | 0.53 $\pm$ 0.02         | 0.57 $\pm$ 0.05          | 0.56 $\pm$ 0.02                           | 0.56 $\pm$ 0.05     |
| ru         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0               | 0.43 $\pm$ 0.05             | 0                       | 0                        | 0                                         | 0.53 $\pm$ 0.05     |
| ru         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.34 $\pm$ 0.08 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.10 $\pm$ 0.07          | 0.05 $\pm$ 0.07                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.10 $\pm$ 0.03 | 0.05 $\pm$ 0.07             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.00     |
| ru         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.16 $\pm$ 0.04 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.05 $\pm$ 0.07          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.02     |
| ru         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.43 $\pm$ 0.08 | 0.17 $\pm$ 0.05             | 0.22 $\pm$ 0.12         | 0.28 $\pm$ 0.20          | 0.21 $\pm$ 0.14                           | 0.15 $\pm$ 0.07     |
| ru         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.33 $\pm$ 0.10 | 0.05 $\pm$ 0.07             | 0.05 $\pm$ 0.07         | 0.10 $\pm$ 0.07          | 0.05 $\pm$ 0.07                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0               | 0.17 $\pm$ 0.03             | 0                       | 0                        | 0                                         | 0.00 $\pm$ 0.00     |
| ru         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.33 $\pm$ 0.06 | 0.37 $\pm$ 0.05             | 0.40 $\pm$ 0.02         | 0.46 $\pm$ 0.04          | 0.42 $\pm$ 0.02                           | 0.48 $\pm$ 0.01     |
| ru         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.32 $\pm$ 0.03 | 0.35 $\pm$ 0.06             | 0.38 $\pm$ 0.03         | 0.50 $\pm$ 0.08          | 0.45 $\pm$ 0.01                           | 0.52 $\pm$ 0.05     |
| ru         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.36 $\pm$ 0.05 | 0.35 $\pm$ 0.03             | 0.40 $\pm$ 0.07         | 0.48 $\pm$ 0.06          | 0.41 $\pm$ 0.07                           | 0.51 $\pm$ 0.03     |
| ru         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.36 $\pm$ 0.03 | 0.39 $\pm$ 0.07             | 0.40 $\pm$ 0.03         | 0.44 $\pm$ 0.05          | 0.42 $\pm$ 0.01                           | 0.49 $\pm$ 0.03     |
| ru         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0               | 0.38 $\pm$ 0.07             | 0                       | 0                        | 0                                         | 0.47 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_type                      | model_subtype                 | model_name                                   | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:--------------------------------|:------------------------------|:---------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.02 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| en         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.08 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.09 $\pm$ 0.01     |
| en         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| en         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.01 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | No Upsampling                 | kNN                                          | 0.00 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| en         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.00     |
| en         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| en         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0               | 0.00 $\pm$ 0.00             | 0                       | 0                        | 0.00 $\pm$ 0.00                           | 0                   |
| en         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.02 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0               | 0.07 $\pm$ 0.01             | 0                       | 0                        | 0.09 $\pm$ 0.01                           | 0                   |
| en         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.00          | 0.08 $\pm$ 0.02                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.02 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.00          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.03 $\pm$ 0.00 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.00             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.03 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.03 $\pm$ 0.00 | 0.05 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.08 $\pm$ 0.01                           | 0.10 $\pm$ 0.03     |
| en         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.05 $\pm$ 0.01 | 0.08 $\pm$ 0.00             | 0.08 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.08 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.02             | 0.07 $\pm$ 0.00         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.00     |
| en         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.05 $\pm$ 0.00 | 0.07 $\pm$ 0.00             | 0.08 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.07 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.08 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.07 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.05 $\pm$ 0.02     |
| en         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| en         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.00     |
| en         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| en         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.10 $\pm$ 0.01         | 0.09 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.06 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.01         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.06 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| en         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.09 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.05 $\pm$ 0.02 | 0.07 $\pm$ 0.01             | 0.10 $\pm$ 0.01         | 0.09 $\pm$ 0.01          | 0.10 $\pm$ 0.02                           | 0.10 $\pm$ 0.02     |
| en         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.04 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.10 $\pm$ 0.02                           | 0.11 $\pm$ 0.02     |
| en         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| en         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.03 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| en         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| en         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.03 $\pm$ 0.01 | 0.09 $\pm$ 0.01             | **0.13 $\pm$ 0.02**     | 0.12 $\pm$ 0.02          | 0.12 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| en         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.03 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.00         | 0.11 $\pm$ 0.01          | 0.10 $\pm$ 0.01                           | 0.11 $\pm$ 0.01     |
| en         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.03 $\pm$ 0.01         | 0.02 $\pm$ 0.00          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| en         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.00         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| en         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| en         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.07 $\pm$ 0.01 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.11 $\pm$ 0.01                           | 0.12 $\pm$ 0.01     |
| en         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.07 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.11 $\pm$ 0.03         | 0.10 $\pm$ 0.03          | 0.12 $\pm$ 0.02                           | 0.12 $\pm$ 0.01     |
| en         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.07 $\pm$ 0.02 | 0.09 $\pm$ 0.01             | 0.11 $\pm$ 0.02         | 0.11 $\pm$ 0.00          | 0.11 $\pm$ 0.01                           | 0.11 $\pm$ 0.02     |
| en         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.06 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.11 $\pm$ 0.00                           | 0.12 $\pm$ 0.01     |
| en         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.08 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.12 $\pm$ 0.03          | 0.11 $\pm$ 0.02                           | 0.12 $\pm$ 0.01     |
| en         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.05 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.08 $\pm$ 0.01          | 0.09 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| en         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.03 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.00          | 0.09 $\pm$ 0.02                           | 0.10 $\pm$ 0.03     |
| en         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.03 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| en         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.06 $\pm$ 0.02 | 0.05 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.03 $\pm$ 0.02     |
| en         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| en         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| en         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.04 $\pm$ 0.00 | 0.04 $\pm$ 0.00             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| en         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.06 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| en         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.07 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| en         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.04 $\pm$ 0.00 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.02     |
| en         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.04 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.01          | 0.08 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| fr         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.02 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.01     |
| fr         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.00 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| fr         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.01     |
| fr         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | KNN                             | No Upsampling                 | kNN                                          | 0.04 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| fr         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.03 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.01     |
| fr         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.07 $\pm$ 0.03     |
| fr         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.02 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.02     |
| fr         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| fr         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.02 $\pm$ 0.02 | 0.07 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.03 $\pm$ 0.00                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.02 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.06 $\pm$ 0.03     |
| fr         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.02 $\pm$ 0.01 | 0.06 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.08 $\pm$ 0.03     |
| fr         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.02             | 0.03 $\pm$ 0.00         | 0.04 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| fr         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.04 $\pm$ 0.01         | 0.01 $\pm$ 0.00          | 0.04 $\pm$ 0.04                           | 0.09 $\pm$ 0.02     |
| fr         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.09 $\pm$ 0.02     |
| fr         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.05 $\pm$ 0.02                           | 0.09 $\pm$ 0.03     |
| fr         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.02 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.04 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.04 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| fr         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.04 $\pm$ 0.01     |
| fr         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.04 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.03 $\pm$ 0.02     |
| fr         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| fr         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | **0.10 $\pm$ 0.02**                       | 0.06 $\pm$ 0.01     |
| fr         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| fr         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.00 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| fr         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| fr         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.00 $\pm$ 0.01          | 0.03 $\pm$ 0.00                           | 0.03 $\pm$ 0.02     |
| fr         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.00     |
| fr         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.02                           | 0.05 $\pm$ 0.02     |
| fr         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.04 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| fr         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.02 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.01     |
| fr         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.02             | 0.04 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.03     |
| fr         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| fr         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| fr         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| fr         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.00 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.01 $\pm$ 0.02         | 0.01 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| fr         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.02 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| fr         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.04 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| fr         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.00 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.01          | 0.00 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| fr         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.02 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.01     |
| fr         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.03 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.05 $\pm$ 0.02                           | 0.06 $\pm$ 0.01     |
| fr         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.03 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| fr         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.03 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| fr         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.01 $\pm$ 0.00          | 0.05 $\pm$ 0.03                           | 0.06 $\pm$ 0.01     |
| ge         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ge         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | No Upsampling                 | kNN                                          | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0                        | 0.00 $\pm$ 0.00                           | 0                   |
| ge         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0                        | 0.02 $\pm$ 0.01                           | 0                   |
| ge         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.01 $\pm$ 0.02 | 0.03 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| ge         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.02 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ge         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| ge         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ge         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.03                           | 0.01 $\pm$ 0.01     |
| ge         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| ge         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.02             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.05 $\pm$ 0.02     |
| ge         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.02                           | 0.03 $\pm$ 0.03     |
| ge         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ge         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| ge         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.02     |
| ge         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.03 $\pm$ 0.03         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.03                           | 0.02 $\pm$ 0.01     |
| ge         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.02          | 0.03 $\pm$ 0.04                           | 0.03 $\pm$ 0.01     |
| ge         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.01 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | **0.06 $\pm$ 0.02**      | 0.03 $\pm$ 0.04                           | 0.04 $\pm$ 0.03     |
| ge         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.04                           | 0.02 $\pm$ 0.01     |
| ge         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.02 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.00 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ge         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.02                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.02     |
| ge         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.02          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ge         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ge         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.02 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ge         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.02 $\pm$ 0.03 | 0.05 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.02 $\pm$ 0.00     |
| ge         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ge         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.04 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| ge         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.01 $\pm$ 0.01          | 0.04 $\pm$ 0.04                           | 0.02 $\pm$ 0.01     |
| it         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| it         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| it         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | No Upsampling                 | kNN                                          | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.01 $\pm$ 0.01     |
| it         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| it         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0.00 $\pm$ 0.00 | 0                           | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0                                         | 0.00 $\pm$ 0.00     |
| it         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0.03 $\pm$ 0.02 | 0                           | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0                                         | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.03     |
| it         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.02                           | 0.06 $\pm$ 0.03     |
| it         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.03 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.03     |
| it         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.03 $\pm$ 0.01 | 0.02 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.03          | 0.05 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| it         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.06 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.05 $\pm$ 0.03          | 0.04 $\pm$ 0.03                           | 0.07 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.05 $\pm$ 0.03                           | 0.06 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.06 $\pm$ 0.03                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.03                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.06 $\pm$ 0.02                           | 0.06 $\pm$ 0.03     |
| it         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.07 $\pm$ 0.02                           | 0.07 $\pm$ 0.04     |
| it         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.04 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.05 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.05 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.05 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| it         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.04 $\pm$ 0.03 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.03 $\pm$ 0.00     |
| it         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.05 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.05 $\pm$ 0.02                           | 0.08 $\pm$ 0.01     |
| it         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.05 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.06 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.07 $\pm$ 0.03         | 0.08 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | **0.09 $\pm$ 0.02** |
| it         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.05 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.05 $\pm$ 0.02 | 0.04 $\pm$ 0.00             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.06 $\pm$ 0.02 | 0.06 $\pm$ 0.02             | 0.06 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| it         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.02 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.00         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| it         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.01 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| it         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.01 $\pm$ 0.00 | 0.02 $\pm$ 0.02             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.05 $\pm$ 0.03     |
| it         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.00             | 0.07 $\pm$ 0.01         | **0.09 $\pm$ 0.01**      | 0.08 $\pm$ 0.01                           | **0.09 $\pm$ 0.01** |
| it         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.04 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.07 $\pm$ 0.02         | 0.07 $\pm$ 0.01          | 0.07 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| it         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.01 $\pm$ 0.00 | 0.01 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.00     |
| it         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.02 $\pm$ 0.00     |
| it         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.01 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| it         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.01 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.00          | 0.04 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| it         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.04 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.07 $\pm$ 0.02         | **0.09 $\pm$ 0.01**      | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| it         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.04 $\pm$ 0.02 | 0.05 $\pm$ 0.02             | 0.07 $\pm$ 0.01         | 0.07 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| it         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.03 $\pm$ 0.02 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | **0.09 $\pm$ 0.02**      | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.02     |
| it         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.04 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.06 $\pm$ 0.00         | 0.07 $\pm$ 0.01          | 0.05 $\pm$ 0.02                           | 0.07 $\pm$ 0.02     |
| it         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.04 $\pm$ 0.02 | 0.06 $\pm$ 0.00             | 0.07 $\pm$ 0.01         | **0.09 $\pm$ 0.02**      | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| it         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.04 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.03 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| it         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.03 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| it         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.04 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| it         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.04 $\pm$ 0.04 | 0.03 $\pm$ 0.04             | 0.04 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.01 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| it         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.03 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.00     |
| it         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.04 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| it         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.02 $\pm$ 0.00 | 0.01 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| it         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.06 $\pm$ 0.03          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| it         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.03 $\pm$ 0.00 | 0.01 $\pm$ 0.00             | 0.04 $\pm$ 0.00         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.00                           | 0.05 $\pm$ 0.01     |
| it         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.03 $\pm$ 0.00         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| it         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.03 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| it         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.03 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.06 $\pm$ 0.02          | 0.02 $\pm$ 0.00                           | 0.06 $\pm$ 0.01     |
| po         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.02 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.05 $\pm$ 0.00          | 0.02 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| po         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | No Upsampling                 | kNN                                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.02 $\pm$ 0.01     |
| po         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | KNN                             | SVMSMOTE                      | kNN_SVMSMOTE                                 | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0                                         | 0                   |
| po         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_SVMSMOTE                           | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0                                         | 0                   |
| po         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.03 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.03 $\pm$ 0.00          | 0.03 $\pm$ 0.02                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.00         | 0.04 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.05 $\pm$ 0.00     |
| po         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.05 $\pm$ 0.00                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.00     |
| po         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0.01 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.02     |
| po         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.01 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| po         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.01     |
| po         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.05 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.03 $\pm$ 0.01 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.05 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.04 $\pm$ 0.02     |
| po         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.04     |
| po         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.00 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.03         | 0.04 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| po         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.04 $\pm$ 0.03         | 0.07 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.07 $\pm$ 0.02     |
| po         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.01     |
| po         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.04 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.03 $\pm$ 0.01     |
| po         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.00 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| po         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.00             | 0.03 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| po         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.01 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| po         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.03 $\pm$ 0.03          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| po         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.02             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.05 $\pm$ 0.01     |
| po         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.03             | 0.03 $\pm$ 0.03         | 0.04 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | **0.09 $\pm$ 0.02** |
| po         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.03          | 0.04 $\pm$ 0.03                           | 0.07 $\pm$ 0.02     |
| po         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.02             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.03          | 0.05 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.02 $\pm$ 0.00 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.07 $\pm$ 0.01     |
| po         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0.02 $\pm$ 0.00 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.05 $\pm$ 0.01          | 0.05 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| po         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.02 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.02 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| po         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.02 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| po         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.03 $\pm$ 0.03          | 0.01 $\pm$ 0.01                           | 0.03 $\pm$ 0.03     |
| po         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.03 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.03                           | 0.05 $\pm$ 0.02     |
| po         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.00 $\pm$ 0.00 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.04 $\pm$ 0.02     |
| po         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.02 $\pm$ 0.02          | 0.02 $\pm$ 0.01                           | 0.04 $\pm$ 0.02     |
| po         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.02 $\pm$ 0.01         | 0.02 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ru         | Binary Relevance kNN            | Natively Multilabel           | BRkNNaClassifier                             | 0.04 $\pm$ 0.03 | 0.02 $\pm$ 0.02             | 0.01 $\pm$ 0.01         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ru         | Binary Relevance kNN            | Natively Multilabel           | BRkNNbClassifier                             | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.02                           | 0.01 $\pm$ 0.01     |
| ru         | ComplementNB                    | RakelD Partitioning of labels | RakelD_ComplementNB                          | 0.01 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| ru         | Dummy Classifier                | No Upsampling                 | DummyMostFrequent                            | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | Dummy Classifier                | No Upsampling                 | DummyProbSampling                            | 0.01 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.01         | 0.02 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.01 $\pm$ 0.01     |
| ru         | Dummy Classifier                | No Upsampling                 | DummyUniformSampling                         | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | BorderlineSMOTE               | kNN_BorderlineSMOTE                          | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | No Upsampling                 | kNN                                          | 0.06 $\pm$ 0.04 | 0.04 $\pm$ 0.02             | 0.05 $\pm$ 0.02         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.02                           | 0.03 $\pm$ 0.03     |
| ru         | KNN                             | Random Oversampling           | kNN_ROS                                      | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.01 $\pm$ 0.02         | 0.02 $\pm$ 0.01          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | KNN                             | SMOTE                         | kNN_SMOTE                                    | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.00 $\pm$ 0.00     |
| ru         | LinearSVM                       | BorderlineSMOTE               | LinearSVM_BorderlineSMOTE                    | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LinearSVM                       | No Upsampling                 | LinearSVM                                    | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LinearSVM                       | RakelD Partitioning of labels | RakelD_LineaSVM                              | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.09 $\pm$ 0.02         | 0.11 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| ru         | LinearSVM                       | Random Oversampling           | LinearSVM_ROS                                | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LinearSVM                       | SMOTE                         | LinearSVM_SMOTE                              | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.02                           | 0.12 $\pm$ 0.02     |
| ru         | LogisticRegression              | BorderlineSMOTE               | LogisticRegression_BorderlineSMOTE           | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.02                           | 0.10 $\pm$ 0.01     |
| ru         | LogisticRegression              | No Upsampling                 | LogisticRegression                           | 0.04 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.09 $\pm$ 0.00         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.04                           | 0.10 $\pm$ 0.02     |
| ru         | LogisticRegression              | Random Oversampling           | LogisticRegression_ROS                       | 0.04 $\pm$ 0.01 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.09 $\pm$ 0.03          | 0.08 $\pm$ 0.03                           | 0.10 $\pm$ 0.01     |
| ru         | LogisticRegression              | SMOTE                         | LogisticRegression_SMOTE                     | 0.04 $\pm$ 0.01 | 0.09 $\pm$ 0.02             | 0.10 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.03                           | 0.11 $\pm$ 0.01     |
| ru         | LogisticRegression              | SVMSMOTE                      | LogisticRegression_SVMSMOTE                  | 0               | 0.08 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.12 $\pm$ 0.01     |
| ru         | LogisticRegressionElasticNet    | BorderlineSMOTE               | LogisticRegressionElasticNet_BorderlineSMOTE | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.03          | 0.09 $\pm$ 0.03                           | 0.12 $\pm$ 0.03     |
| ru         | LogisticRegressionElasticNet    | No Upsampling                 | LogisticRegressionElasticNet                 | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.09 $\pm$ 0.02         | 0.07 $\pm$ 0.03          | 0.10 $\pm$ 0.03                           | 0.11 $\pm$ 0.01     |
| ru         | LogisticRegressionElasticNet    | Random Oversampling           | LogisticRegressionElasticNet_ROS             | 0.03 $\pm$ 0.00 | 0.04 $\pm$ 0.01             | 0.07 $\pm$ 0.01         | 0.09 $\pm$ 0.04          | 0.09 $\pm$ 0.04                           | 0.11 $\pm$ 0.02     |
| ru         | LogisticRegressionElasticNet    | SMOTE                         | LogisticRegressionElasticNet_SMOTE           | 0.03 $\pm$ 0.00 | 0.05 $\pm$ 0.01             | 0.08 $\pm$ 0.02         | 0.09 $\pm$ 0.03          | 0.10 $\pm$ 0.03                           | **0.13 $\pm$ 0.02** |
| ru         | LogisticRegressionElasticNet    | SVMSMOTE                      | LogisticRegressionElasticNet_SVMSMOTE        | 0               | 0.05 $\pm$ 0.00             | 0                       | 0                        | 0                                         | **0.13 $\pm$ 0.02** |
| ru         | LogisticRegressionLasso         | BorderlineSMOTE               | LogisticRegressionLasso_BorderlineSMOTE      | 0.03 $\pm$ 0.02 | 0.04 $\pm$ 0.01             | 0.04 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.08 $\pm$ 0.03                           | 0.08 $\pm$ 0.03     |
| ru         | LogisticRegressionLasso         | No Upsampling                 | LogisticRegressionLasso                      | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.05 $\pm$ 0.03         | 0.06 $\pm$ 0.03          | 0.08 $\pm$ 0.02                           | 0.05 $\pm$ 0.03     |
| ru         | LogisticRegressionLasso         | Random Oversampling           | LogisticRegressionLasso_ROS                  | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.06 $\pm$ 0.04          | 0.07 $\pm$ 0.03                           | 0.06 $\pm$ 0.02     |
| ru         | LogisticRegressionLasso         | SMOTE                         | LogisticRegressionLasso_SMOTE                | 0.02 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.03 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.07 $\pm$ 0.03                           | 0.06 $\pm$ 0.04     |
| ru         | LogisticRegressionLasso         | SVMSMOTE                      | LogisticRegressionLasso_SVMSMOTE             | 0               | 0.05 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.07 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | BorderlineSMOTE               | LogisticRegressionRidge_BorderlineSMOTE      | 0.03 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | No Upsampling                 | LogisticRegressionRidge                      | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.07 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | RakelD Partitioning of labels | RakelD_LogisticRegression                    | 0.04 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.08 $\pm$ 0.00         | 0.09 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.11 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | Random Oversampling           | LogisticRegressionRidge_ROS                  | 0.04 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.10 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.08 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | SMOTE                         | LogisticRegressionRidge_SMOTE                | 0.04 $\pm$ 0.02 | 0.08 $\pm$ 0.01             | 0.09 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.08 $\pm$ 0.01                           | 0.09 $\pm$ 0.01     |
| ru         | LogisticRegressionRidge         | SVMSMOTE                      | LogisticRegressionRidge_SVMSMOTE             | 0               | 0.07 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.10 $\pm$ 0.01     |
| ru         | Multi-label ARAM                | Natively Multilabel           | MLARAM                                       | 0.07 $\pm$ 0.01 | 0.08 $\pm$ 0.02             | 0.08 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.05 $\pm$ 0.03     |
| ru         | Multilabel k Nearest Neighbours | Natively Multilabel           | MLkNN                                        | 0.04 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.02                           | 0.06 $\pm$ 0.02     |
| ru         | NaiveBayes                      | BorderlineSMOTE               | ComplementNaiveBayes_BorderlineSMOTE         | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.05 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ru         | NaiveBayes                      | BorderlineSMOTE               | NaiveBayes_BorderlineSMOTE                   | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.04 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.04 $\pm$ 0.02                           | 0.06 $\pm$ 0.01     |
| ru         | NaiveBayes                      | No Upsampling                 | ComplementNaiveBayes                         | 0.02 $\pm$ 0.01 | 0.06 $\pm$ 0.03             | 0.08 $\pm$ 0.04         | **0.13 $\pm$ 0.03**      | 0.10 $\pm$ 0.02                           | 0.10 $\pm$ 0.03     |
| ru         | NaiveBayes                      | No Upsampling                 | NaiveBayes                                   | 0.00 $\pm$ 0.00 | 0.00 $\pm$ 0.00             | 0.01 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.02 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ru         | NaiveBayes                      | Random Oversampling           | ComplementNaiveBayes_ROS                     | 0.02 $\pm$ 0.02 | 0.03 $\pm$ 0.00             | 0.04 $\pm$ 0.03         | 0.02 $\pm$ 0.01          | 0.03 $\pm$ 0.01                           | 0.03 $\pm$ 0.02     |
| ru         | NaiveBayes                      | Random Oversampling           | NaiveBayes_ROS                               | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.00             | 0.02 $\pm$ 0.02         | 0.03 $\pm$ 0.03          | 0.02 $\pm$ 0.01                           | 0.03 $\pm$ 0.01     |
| ru         | NaiveBayes                      | SMOTE                         | ComplementNaiveBayes_SMOTE                   | 0.00 $\pm$ 0.00 | 0.03 $\pm$ 0.03             | 0.03 $\pm$ 0.01         | 0.05 $\pm$ 0.03          | 0.04 $\pm$ 0.01                           | 0.04 $\pm$ 0.01     |
| ru         | NaiveBayes                      | SMOTE                         | NaiveBayes_SMOTE                             | 0.01 $\pm$ 0.01 | 0.02 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.04 $\pm$ 0.02          | 0.03 $\pm$ 0.01                           | 0.05 $\pm$ 0.01     |
| ru         | RandomForest                    | BorderlineSMOTE               | RandomForest_BorderlineSMOTE                 | 0.03 $\pm$ 0.01 | 0.05 $\pm$ 0.01             | 0.07 $\pm$ 0.01         | 0.07 $\pm$ 0.02          | 0.05 $\pm$ 0.01                           | 0.06 $\pm$ 0.02     |
| ru         | RandomForest                    | No Upsampling                 | RandomForest                                 | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.03 $\pm$ 0.01         | 0.03 $\pm$ 0.01          | 0.04 $\pm$ 0.01                           | 0.04 $\pm$ 0.03     |
| ru         | RandomForest                    | Random Oversampling           | RandomForest_ROS                             | 0.05 $\pm$ 0.01 | 0.06 $\pm$ 0.01             | 0.07 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.01                           | 0.09 $\pm$ 0.02     |
| ru         | RandomForest                    | SMOTE                         | RandomForest_SMOTE                           | 0.04 $\pm$ 0.01 | 0.05 $\pm$ 0.02             | 0.05 $\pm$ 0.03         | 0.06 $\pm$ 0.02          | 0.07 $\pm$ 0.03                           | 0.07 $\pm$ 0.02     |
| ru         | RandomForest                    | SVMSMOTE                      | RandomForest_SVMSMOTE                        | 0               | 0.03 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.06 $\pm$ 0.04     |
| ru         | RidgeClassifier                 | BorderlineSMOTE               | RidgeClassifier_BorderlineSMOTE              | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | No Upsampling                 | RidgeClassifier                              | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | Random Oversampling           | RidgeClassifier_ROS                          | 0.03 $\pm$ 0.03 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | SMOTE                         | RidgeClassifier_SMOTE                        | 0.03 $\pm$ 0.02 | 0.07 $\pm$ 0.02             | 0.10 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.09 $\pm$ 0.01                           | 0.10 $\pm$ 0.01     |
| ru         | RidgeClassifier                 | SVMSMOTE                      | RidgeClassifier_SVMSMOTE                     | 0               | 0.07 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.11 $\pm$ 0.00     |
| ru         | SVM                             | BorderlineSMOTE               | SVM_rbf_BorderlineSMOTE                      | 0.04 $\pm$ 0.02 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | No Upsampling                 | SVM_rbf                                      | 0.01 $\pm$ 0.01 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.01 $\pm$ 0.01                           | 0.02 $\pm$ 0.00     |
| ru         | SVM                             | RakelD Partitioning of labels | RakelD_SVM                                   | 0.01 $\pm$ 0.01 | 0.00 $\pm$ 0.00             | 0.00 $\pm$ 0.00         | 0.00 $\pm$ 0.00          | 0.00 $\pm$ 0.00                           | 0.01 $\pm$ 0.01     |
| ru         | SVM                             | Random Oversampling           | SVM_rbf_ROS                                  | 0.06 $\pm$ 0.02 | 0.02 $\pm$ 0.01             | 0.02 $\pm$ 0.01         | 0.01 $\pm$ 0.01          | 0.03 $\pm$ 0.02                           | 0.02 $\pm$ 0.01     |
| ru         | SVM                             | SMOTE                         | SVM_rbf_SMOTE                                | 0.02 $\pm$ 0.02 | 0.01 $\pm$ 0.01             | 0.00 $\pm$ 0.00         | 0.01 $\pm$ 0.01          | 0.01 $\pm$ 0.01                           | 0.00 $\pm$ 0.00     |
| ru         | SVM                             | SVMSMOTE                      | SVM_rbf_SVMSMOTE                             | 0               | 0.01 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.00 $\pm$ 0.00     |
| ru         | XGBoost                         | BorderlineSMOTE               | XGBoost_narrow_BorderlineSMOTE               | 0.02 $\pm$ 0.02 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.08 $\pm$ 0.02          | 0.06 $\pm$ 0.03                           | 0.12 $\pm$ 0.02     |
| ru         | XGBoost                         | No Upsampling                 | XGBoost_narrow                               | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.06 $\pm$ 0.01         | 0.08 $\pm$ 0.02          | 0.07 $\pm$ 0.03                           | 0.12 $\pm$ 0.04     |
| ru         | XGBoost                         | Random Oversampling           | XGBoost_narrow_ROS                           | 0.04 $\pm$ 0.03 | 0.02 $\pm$ 0.01             | 0.05 $\pm$ 0.02         | 0.06 $\pm$ 0.02          | 0.06 $\pm$ 0.01                           | **0.13 $\pm$ 0.05** |
| ru         | XGBoost                         | SMOTE                         | XGBoost_narrow_SMOTE                         | 0.02 $\pm$ 0.01 | 0.03 $\pm$ 0.01             | 0.05 $\pm$ 0.01         | 0.08 $\pm$ 0.01          | 0.07 $\pm$ 0.04                           | 0.11 $\pm$ 0.03     |
| ru         | XGBoost                         | SVMSMOTE                      | XGBoost_narrow_SVMSMOTE                      | 0               | 0.03 $\pm$ 0.01             | 0                       | 0                        | 0                                         | 0.12 $\pm$ 0.02     |

/tmp/ipykernel_58875/3322418740.py:69: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
